In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install 'tirex-ts[notebooks,gluonts,hfdataset]' -q
from tirex import ForecastModel, load_model


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
 !pip install torch torchvision torchaudio seaborn optuna arch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.7/990.7 kB 37.8 MB/s eta 0:00:00


In [5]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.6.0+cu124
CUDA available: True


Hugging face: https://huggingface.co/papers/2312.01020

# Dynamic linear layer

The paper talks about using different input sizes but the code implemented only caters to 5 inputs. This function let's us adapt any input length to test the model. As a result, we can try out different lengths for different stocks to find the sweet spot for a specific stock

In [6]:
from tirex import ForecastModel, load_model


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from typing import Tuple 

class FlexibleResNLS(nn.Module):
    """
    ✨ Flexible ResNLS model with Residual Connection for Even/Odd Classification.
    
    CNN (Feature Extraction) -> Weighted Residual Link -> LSTM (Sequence Processing) -> Linear (Classification).
    Input shape: [Batch, Sequence_Len (S), Features (F)] -> e.g., [64, 100, 12].
    """
    def __init__(self, n_input: int, n_hidden: int = 64, n_output: int = 14):
        super(FlexibleResNLS, self).__init__()
        self.n_input = n_input 
        self.n_hidden = n_hidden
        self.n_output = n_output
        self.output_channels = 32 # New hyperparameter for CNN output channels
        
        # ⚖️ Learnable residual weight
        self.weight = nn.Parameter(torch.zeros(1))
        print("😊 Initialized residual weight parameter.")

        # --- 🧠 CNN feature extractor ---
        # Input shape expected: [Batch, Channels (Features=12), Sequence_Length (S)]
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=self.n_input, out_channels=self.output_channels, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(self.output_channels, eps=1e-5),
            nn.Dropout(0.3),

            nn.Conv1d(in_channels=self.output_channels, out_channels=self.output_channels, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(self.output_channels, eps=1e-5),
            # Output shape: [Batch, output_channels (32), Sequence_Length (S)]
        )
        print(f"📐 CNN output channels: {self.output_channels}.")

        # --- 🔗 Residual Skip Path (1x1 Convolution) ---
        # Maps the input feature dimension (n_input=12) to the CNN output dimension (32).
        self.residual_conv = nn.Conv1d(
            in_channels=self.n_input, 
            out_channels=self.output_channels, 
            kernel_size=1
        )
        print(f"🔄 Residual 1x1 Conv: {self.n_input} -> {self.output_channels} channels.")


        # --- ⏳ LSTM ---
        # LSTM input size is the number of channels (features) output by the CNN/Residual link.
        self.lstm = nn.LSTM(
            input_size=self.output_channels, 
            hidden_size=self.n_hidden,
            batch_first=True,
            bidirectional=False
        )
        print(f"🔁 LSTM input size: {self.output_channels}, hidden size: {self.n_hidden}.")
            
        # 🎯 Output layer
        self.linear = nn.Linear(self.n_hidden, self.n_output)
        print(f"✨ Final linear layer outputting {self.n_output} logits.")


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: Tensor (batch, sequence_length, n_input) - e.g., [64, 100, 12]
        Output: Tensor (batch, n_output) - Raw logits.
        """
        # 1. Adapt input for Conv1d: [Batch, Sequence_Len, Features] -> [Batch, Features, Sequence_Len]
        x_cnn_input = x.transpose(1, 2) 
        # Shape: [Batch, 12, 100]

        # 2. CNN Feature Extraction
        cnn_output = self.cnn(x_cnn_input)
        # Shape: [Batch, 32, 100]

        # 3. Weighted Residual Connection
        
        # 3a. Skip Path: Project input features (12) to output channels (32)
        skip_path = self.residual_conv(x_cnn_input) 
        # Shape: [Batch, 32, 100] (Same shape as cnn_output)

        # 3b. Weighted Addition (Residual Link)
        residual_output = cnn_output + self.weight * skip_path
        # Shape: [Batch, 32, 100]

        # 4. Prepare output for LSTM: [Batch, Features, Sequence_Len] -> [Batch, Sequence_Len, Features]
        # LSTM expects: [Batch, Sequence_Len, Features (32)]
        lstm_input = residual_output.transpose(1, 2)
        # Shape: [Batch, 100, 32]

        # 5. LSTM processing
        # Returns (output, (h_n, c_n))
        _, (h_n, _) = self.lstm(lstm_input)
        
        # 6. Final classification
        # We take the hidden state of the *last layer* (index 0)
        y_hat = self.linear(h_n[0,:,:]) 
            
        return y_hat

# Deriv API Configs


Deriv broker offers a handful of snthetically generated assets and I thought it would be good to see how this model performs on assets that are computer generated. Can this model pick up the under;ying patterns that power the synthetic generator?

To get your APP ID and token please follow this [guide](https://docs.google.com/document/d/1R0nvI5nL_-pqkqSJCsdvU0_xzFqbiwhpDr74mkuLtqo/edit?usp=sharing)

In [8]:
DERIV_SYMBOL = "R_100"         # 📈 Volatility 100 Index (VIX equivalent)
OHLC_GRANULARITY = 60       # 📅 1 Day in seconds (to mimic daily close data)
OHLC_COUNT = 5000              # 💾 Number of historical daily candles to fetch

In [9]:
# kaggle secrets here if on kaggle

from kaggle_secrets import UserSecretsClient

# ---------------------------------------------------------
# 🔑 DERIV API CONSTANTS
# ---------------------------------------------------------

try:
    # Initialize the secrets client
    secrets = UserSecretsClient()

    # Get the main secret string (assuming the token is stored as the primary secret)
    DEFAULT_TOKEN = secrets.get_secret("deriv_api")
    print("🔑 Deriv Token loaded securely.")

    # Get a custom secret key (assuming the App ID is stored as a custom key 'app_id')
    # Note: KaggleSecrets returns strings, so it must be converted to an integer.
    APP_ID = int(secrets.get_secret("deriv_app_id"))
    print("🆔 Deriv App ID loaded securely.")

except Exception as e:
    # Fallback to placeholders if running outside Kaggle or secret is missing/incorrect
    print(f"⚠️ Warning: Failed to load secrets from Kaggle ({e}). Falling back to dummy values.")
    APP_ID = 1234
    DEFAULT_TOKEN = "YOUR-TOKEN-GOES-HERE"





🔑 Deriv Token loaded securely.
🆔 Deriv App ID loaded securely.


In [10]:
# =========================================================
# 🔗 NEW DATA SOURCE: Deriv API Client for OHLC
# =========================================================
import asyncio
import json
import ssl
import time
import os
import sys
from collections import deque, Counter
from decimal import Decimal, InvalidOperation
import certifi
import websockets
from concurrent.futures import Future
import threading

class DerivOHLCFetcher:
    """A minimal Deriv WS client to fetch historical OHLC (candle) data."""

    def __init__(self, token: str, app_id: int):
        self.token = token
        self.app_id = app_id
        self.uri = f"wss://ws.derivws.com/websockets/v3?app_id={app_id}"
        # 🔒 Secure connection setup
        # self.ssl_context = ssl.create_default_context(cafile=certifi.where()) 
        self.resp_queue = asyncio.Queue()
        self.ws = None

    async def connect_and_authorize(self):
        """Connects, authorizes, and returns the websocket object."""
        try:
            print("🔗 Connecting securely to Deriv WebSocket...")
            self.ws = await websockets.connect(self.uri,  open_timeout=10)
            
            # Start a background reader for responses
            self.reader_task = asyncio.create_task(self._reader_loop())
            
            await self.ws.send(json.dumps({"authorize": self.token}))
            
            # Wait for authorization response
            msg = await asyncio.wait_for(self.resp_queue.get(), timeout=10)
            
            if "error" in msg:
                print(f"💥 Authorization error: {msg['error'].get('message')}")
                return False
            
            print(f"✅ Authorized! Login ID: {msg['authorize'].get('loginid')}")
            return True
        
        except Exception as e:
            print(f"❌ Connection/Authorization failed: {e}")
            return False

    async def _reader_loop(self):
        """Reads frames and pushes parsed JSON into resp_queue."""
        try:
            async for raw in self.ws:
                msg = json.loads(raw)
                # Only queue relevant responses (authorize, error, candles)
                if isinstance(msg, dict) and any(k in msg for k in ("authorize", "error", "candles")):
                    await self.resp_queue.put(msg)
        except Exception:
            # 🧹 Cleanly stop the reader loop
            pass 

    async def fetch_ohlc(self, symbol: str, granularity: int, count: int) -> pd.DataFrame | None:
        """Fetches historical OHLC data and converts it to a pandas DataFrame."""
        if not await self.connect_and_authorize():
            return None

        # Request payload using 'ticks_history' for candles or ticks
        # when no gransurality is supplied or is less than 60, we assume the user wants to use ticks
        if granularity >=60:
            payload = {
                "ticks_history": symbol,
                "count": count,
                "end": "latest",
                "style": "candles",
                "granularity": granularity
            }
        else:
             payload = {
                "ticks_history": symbol,
                "count": count,
                "end": "latest",
                "style": "ticks"
                
            }
            
        print(f"⏳ Requesting {count} {granularity // 60}-minute candles for {symbol}...")
        await self.ws.send(json.dumps(payload))
        
        try:
            # Wait for response
            resp = await asyncio.wait_for(self.resp_queue.get(), timeout=30)
            
            if "error" in resp:
                print(f"💥 API Error on request: {resp['error'].get('message')}")
                return None
            
            # Candles data is extracted directly from the 'candles' key
            if granularity >= 60:
                candles = resp.get('candles', []) 
                df = pd.DataFrame(candles)
            else:
                candles = resp.get('prices', [])
                times = resp.get('times',[])
                data = {
                        'epoch': times,
                        'close': candles
                    }
                    
                    # ⚙️ Creating the DataFrame
                df = pd.DataFrame(data)
                            
            if not candles:
                print("⚠️ No candles returned from API.")
                return None
            
            print(f"✅ Loaded {len(candles)} candles.")
            
            # Convert to DataFrame
           
            df['date'] = pd.to_datetime(df['epoch'], unit='s')
            df.set_index('date', inplace=True)
            # 🎯 The model requires "Close" price, which is 'close' in the API response
            df['Close'] = df['close'] 
            df.sort_index(inplace=True)
            
            return df[['Close']].copy()

        except Exception as e:
            print(f"❌ Error during OHLC fetch: {e}")
            return None
        finally:
            # 🔌 Clean up connection
            if self.ws:
                await self.ws.close()
            if self.reader_task:
                 self.reader_task.cancel()


# =========================================================
# 💡 KAGGLE/NOTEBOOK ASYNC FIX HELPER FUNCTION
# =========================================================

def _sync_fetch_ohlc_threaded(fetcher, symbol, granularity, count):
    """Executes the async fetcher in a new thread with its own event loop."""
    print("⚠️ Bypassing main thread's running event loop (Kaggle/Jupyter fix). Fetching data in a separate thread...")
    
    def run_async_in_thread(future_obj):
        new_loop = asyncio.new_event_loop()
        asyncio.set_event_loop(new_loop)
        try:
            result = new_loop.run_until_complete(fetcher.fetch_ohlc(
                symbol=symbol,
                granularity=granularity,
                count=count
            ))
            future_obj.set_result(result)
        finally:
            new_loop.close()

    future = Future()
    thread = threading.Thread(target=run_async_in_thread, args=(future,))
    thread.start()
    thread.join() # Wait for the thread to complete its work
    return future.result()


In [11]:
def get_last_digit(quote,symbol=DERIV_SYMBOL):
    """Returns the last digit (0-9) of a number."""
    # 🔢 Get the last digit by taking the absolute value and then modulo 10
    # return int(abs(value)) % 10

    try:
        last_digit = None
        
        # Find the last digit using the original logic (re-used here)
        if isinstance(quote, int):
            last_digit = abs(quote) % 10
        
        elif isinstance(quote, Decimal):
            digits = quote.normalize().as_tuple().digits
            if not digits:
                last_digit = 0
            else:
                last_digit = digits[-1]
        
        elif isinstance(quote, float):
            frac = abs(quote) % 1.0
            scaled = int(round(frac * 100)) 
            if symbol == "R_10":
                scaled = int(round(frac * 1000)) 
            last_digit = scaled % 10
        
        else:
            print("❌ Unsupported type.")
            return None
        
           
        return last_digit
    
    except Exception as e:
        print(f"🚨 Error in _extract_parity: {e}")
        return None

# TiREX

In [12]:
import numpy as np

def get_last_digit_parity(value):
    """
    Determines the parity of the last digit of a number.
    Returns 'Even' if the last digit is 0, 2, 4, 6, 8.
    Returns 'Odd' if the last digit is 1, 3, 5, 7, 9.
    
    The last digit is obtained using modulo 10.
    Parity is determined by checking if (last_digit % 2) is 0 (Even) or 1 (Odd).
    """
    # 🔢 Get the last digit by taking the absolute value and then modulo 10
    last_digit = get_last_digit(value)
    
    # ❓ Check if the last digit is even (0, 2, 4, 6, 8)
    # The last digit is even if (last_digit % 2) is 0.
    if last_digit % 2 == 0:
        return "Even"
    else:
        return "Odd"

def calculate_parity_accuracy(real_values, predicted_values, forecast_name="Forecast"):
    """
    Calculates the accuracy of predicting the even/odd parity of the last digit.

    :param real_values: NumPy array or list of actual future values.
    :param predicted_values: NumPy array or list of predicted values (e.g., median, mean).
    :param forecast_name: String label for the prediction source (e.g., "Median", "Mean").
    :return: Accuracy percentage and a dictionary of counts.
    """
    # ⚠️ Ensure inputs are NumPy arrays for safety
    real_values = np.array(real_values)
    predicted_values = np.array(predicted_values)

    # 📏 Check for length match
    if len(real_values) != len(predicted_values):
        print("❌ Error: Real and predicted value arrays must have the same length.")
        return 0.0, {}

    # 🧮 Initialize counters
    total_predictions = len(real_values)
    correct_parity_predictions = 0
    
    # Optional: detailed counts
    parity_counts = {
        "Correct": 0, 
        "Incorrect": 0, 
        "Total": total_predictions
    }

    # 🚀 Iterate and compare parity
    for real, predicted in zip(real_values, predicted_values):
        real_parity = get_last_digit_parity(real)
        predicted_parity = get_last_digit_parity(predicted)
        
        if real_parity == predicted_parity:
            correct_parity_predictions += 1
            parity_counts["Correct"] += 1
        else:
            parity_counts["Incorrect"] += 1

    # 📊 Calculate accuracy
    accuracy = (correct_parity_predictions / total_predictions) * 100
    
    # ✅ Log the result
    print(f"✨ {forecast_name} Last Digit Parity Accuracy: {accuracy:.2f}% (Correct: {parity_counts['Correct']}/{total_predictions})")
    
    return accuracy, parity_counts

In [13]:
import numpy as np
import matplotlib.pyplot as plt

def get_last_digit_parity(value):
    """
    Determines the parity of the last digit of a number.
    Returns 'Even' or 'Odd'.
    """
    # 🔢 Get the last digit by taking the absolute value and then modulo 10
    last_digit =get_last_digit(value)
    
    # ❓ Check if the last digit is even (0, 2, 4, 6, 8)
    if last_digit % 2 == 0:
        return "Even"
    else:
        return "Odd"

def calculate_parity_accuracy(real_values, predicted_values, forecast_name="Forecast"):
    """
    Calculates the accuracy of predicting the even/odd parity of the last digit.
    
    :param real_values: NumPy array or list of actual future values.
    :param predicted_values: NumPy array or list of predicted values.
    :param forecast_name: String label for the prediction source (e.g., "Median", "Q10").
    :return: Accuracy percentage.
    """
    # ⚠️ Ensure inputs are NumPy arrays for safety
    real_values = np.array(real_values)
    predicted_values = np.array(predicted_values)

    if len(real_values) != len(predicted_values):
        print(f"❌ Error for {forecast_name}: Real and predicted arrays must have the same length.")
        return 0.0

    total_predictions = len(real_values)
    correct_parity_predictions = 0

    # 🚀 Iterate and compare parity
    for real, predicted in zip(real_values, predicted_values):
        real_parity = get_last_digit_parity(real)
        predicted_parity = get_last_digit_parity(predicted)
        
        if real_parity == predicted_parity:
            correct_parity_predictions += 1

    # 📊 Calculate accuracy
    accuracy = (correct_parity_predictions / total_predictions) * 100
    
    # ✅ Log the result
    print(f"✨ {forecast_name} Last Digit Parity Accuracy: {accuracy:.2f}% (Correct: {correct_parity_predictions}/{total_predictions})")
    
    return accuracy

def plot_forecast_with_zoom(ctx, quantile_fc, real_future_values=None, mean_forecast_values=None):
    # Unpack the numpy arrays for plotting
    median_forecast = quantile_fc[:, 4].numpy() # 50% quantile is often the median
    lower_bound = quantile_fc[:, 0].numpy()
    upper_bound = quantile_fc[:, 8].numpy()

    # Define x-axis ranges
    ctx_len = len(ctx)
    forecast_len = len(median_forecast)
    forecast_x = range(ctx_len, ctx_len + forecast_len)
    
    # Store quantile forecasts as NumPy array for easy access in backtest
    quantiles_data = quantile_fc.numpy() # Shape will be (forecast_len, 9)

    # --- Setup for Two Subplots ---
    fig, axes = plt.subplots(2, 1, figsize=(14, 10)) # 2 rows, 1 column

    # --- Plotting Helper Function ---
    def draw_plot(ax, title, x_min=0, x_max=None):
        # ... (Your existing plotting logic remains the same) ...
        # 🖼️ Plot the Context Data
        ctx_start_index = max(0, x_min)
        ctx_end_index = min(ctx_len, x_max if x_max else ctx_len)
        
        ctx_plot_x = range(ctx_start_index, ctx_end_index)
        ctx_plot_y = ctx[ctx_start_index:ctx_end_index]
        ax.plot(ctx_plot_x, ctx_plot_y, label="Ground Truth Context", color="#4a90d9") # Blue

        # 🟢 Plot the Actual Future Data
        if real_future_values is not None:
            original_fut_x = range(ctx_len, ctx_len + len(real_future_values))
            # The actual ground truth for comparison
            ax.plot(original_fut_x, real_future_values, label="Ground Truth Future", color="#5cb85c", linewidth=2.5, linestyle="-")

        # 🔴 Plot the Forecast Median
        ax.plot(forecast_x, median_forecast, label="Forecast (Median)", color="#d94e4e", linestyle="--")
        
        # 🟠 Plot the Forecast Mean (New Addition!)
        if mean_forecast_values is not None:
            mean_values = np.array(mean_forecast_values) if isinstance(mean_forecast_values, list) else mean_forecast_values
            
            if len(mean_values) == forecast_len:
                ax.plot(forecast_x, mean_values, label="Forecast (Mean)", color="#ffa500", linestyle=":") # Orange dotted line
            else:
                print("⚠️ Warning: Mean forecast length does not match prediction length. Not plotted.")

        # 🌫️ Plot the Uncertainty Band
        ax.fill_between(
            forecast_x, lower_bound, upper_bound, color="#d94e4e", alpha=0.1, label="Forecast 10% - 90% Quantiles"
        )
        
        ax.set_xlim(x_min, x_max)
        ax.set_title(title)
        ax.grid(True)
        ax.legend()

    # --- 1. Full View Plot (Top Subplot) ---
    full_x_max = ctx_len + forecast_len 
    draw_plot(axes[0], "1. Full Time Series and Forecast", x_min=0, x_max=full_x_max)

    # --- 2. Zoomed View Plot (Bottom Subplot) ---
    zoom_x_min = max(0, ctx_len - 100)
    zoom_x_max = ctx_len + forecast_len
    draw_plot(axes[1], f"2. Zoomed View: Last 100 Context Points + Forecast ({forecast_len} steps)", x_min=zoom_x_min, x_max=zoom_x_max)

    plt.tight_layout()
    plt.show()

    # =======================================================
    # 🔮 NEW BACKTEST: Last Digit Parity Accuracy Check for ALL Quantiles
    # =======================================================
    if real_future_values is not None:
        
        # 🏷️ Define quantile names based on the 9 columns (0 to 8)
        # Assuming the quantiles are 10%, 20%, 30%, 40%, 50%, 60%, 70%, 80%, 90%
        quantile_labels = [
            "Q10%", "Q20%", "Q30%", "Q40%", "Q50% (Median)", 
            "Q60%", "Q70%", "Q80%", "Q90%"
        ]
        
        print("\n--- Last Digit Parity Accuracy for Quantiles ---")
        
        # 🔄 Loop through all 9 columns of the quantile forecast
        for i in range(quantiles_data.shape[1]):
            quantile_forecast = quantiles_data[:, i] # Select the i-th column
            label = quantile_labels[i]
            
            # The function will print the result directly
            _ = calculate_parity_accuracy(real_future_values, quantile_forecast, forecast_name=label)
            
        # 🟠 Check Mean Forecast (if available)
        if mean_forecast_values is not None and len(mean_forecast_values) == forecast_len:
            print("\n--- Last Digit Parity Accuracy for Mean ---")
            _ = calculate_parity_accuracy(real_future_values, mean_forecast_values, forecast_name="Mean Forecast")
            
    else:
        print("ℹ️ Cannot run parity backtest: `real_future_values` is missing.")

In [14]:
import time
from functools import wraps

def timer_decorator(func):
    """A decorator that logs the runtime of the decorated function."""
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        # ⏱️ Start timer
        start_time = time.perf_counter()
        print(f"⏱️ log: Function '{func.__name__}' started at: {start_time}")

        # 🚀 Execute function
        result = func(*args, **kwargs)

        # 🛑 End timer
        end_time = time.perf_counter()
        print(f"🛑 log: Function '{func.__name__}' ended at: {end_time}")

        # ⚡ Calculate duration
        elapsed_time = end_time - start_time
        print(f"⚡ log: Total execution time: {elapsed_time:.4f} seconds")
        
        return result
    return wrapper

In [15]:
@timer_decorator
def load_data():
    fetcher = DerivOHLCFetcher(DEFAULT_TOKEN, APP_ID) # Assuming these are defined globally
    df = None
    
    try:
        df = _sync_fetch_ohlc_threaded( # Assuming this utility is defined
            fetcher, DERIV_SYMBOL, OHLC_GRANULARITY, OHLC_COUNT
        )
    except Exception as e:
        print(f"💥 Fatal error during data fetching: {e}") 
        
    
    if df is None or df.empty:
        print(f"❌ Error: No data found for {DERIV_SYMBOL}. Exiting.") 

    return df
   

In [16]:
df = load_data()

⏱️ log: Function 'load_data' started at: 2153.72790062
⚠️ Bypassing main thread's running event loop (Kaggle/Jupyter fix). Fetching data in a separate thread...
🔗 Connecting securely to Deriv WebSocket...
✅ Authorized! Login ID: VRTC1925073
⏳ Requesting 5000 1-minute candles for R_100...
✅ Loaded 5000 candles.
🛑 log: Function 'load_data' ended at: 2159.172072768
⚡ log: Total execution time: 5.4442 seconds


In [17]:
model: ForecastModel = load_model("NX-AI/TiRex")

model.ckpt:   0%|          | 0.00/141M [00:00<?, ?B/s]

In [18]:
df

,Close
date,
2025-10-29 00:27:00,859.62
2025-10-29 00:28:00,858.52
2025-10-29 00:29:00,859.79
2025-10-29 00:30:00,858.82
2025-10-29 00:31:00,857.51
...,...
2025-11-01 11:42:00,857.21
2025-11-01 11:43:00,855.90
2025-11-01 11:44:00,854.35


In [19]:
close = df['Close'].values.reshape(-1)

In [20]:
close

array([859.62, 858.52, 859.79, ..., 854.35, 855.98, 855.26])

In [21]:
if DERIV_SYMBOL == "R_10":
    close = close * 1000
else:
    close = close * 100

In [22]:
print("Original float array:", close)
# Convert the array to integers using astype(int)
close = close.astype(int)

print("Converted integer array:", close)
print("Data type of integer array:", close.dtype)

Original float array: [85962. 85852. 85979. ... 85435. 85598. 85526.]
Converted integer array: [85962 85852 85979 ... 85435 85598 85526]
Data type of integer array: int64


In [23]:
# close = list(close)
close[0:10]

array([85962, 85852, 85979, 85882, 85751, 85796, 85794, 85780, 85755,
       85677])

# MARCOVS

In [24]:
import numpy as np
import pandas as pd

# --- 1. CONFIGURATION AND DATA SETUP ---

# -------------------------------------------------------------

data_points = 5000
train_size = 4000
test_size = data_points - train_size
N_STATES = 10  # Digits 0 through 9
THRESHOLD = 1 / (1 + 7.929)  # 0.11199 (Break-even required win rate)
STAKE = 0.35
NET_ODDS = 7.929 # Payout multiplier

# --- 1a. PREPROCESSING (YOUR NEW BLOCK) ---
close = df['Close'].values.reshape(-1)

if DERIV_SYMBOL == "R_10":
    close = close * 1000
else:
    close = close * 100

print(f"Original float array (First 5): {close[:5]}") # 🔢 log message
# Convert the array to integers using astype(int)
close = close.astype(int)
print(f"Converted integer array (First 5): {close[:5]}") # 🔢 log message
# ------------------------------------------

# --- CAPITAL AND RISK TRACKING SETUP ---
INITIAL_BALANCE = 3.0
current_balance = INITIAL_BALANCE
max_losing_streak = 0
current_losing_streak = 0
peak_balance = INITIAL_BALANCE
max_drawdown = 0.0
# ----------------------------------------

print(f"📊 Running Parity-Filtered Strategy Backtest...")
print(f"💰 Initial Opening Balance: ${INITIAL_BALANCE:,.2f}")
print("-" * 50)

# 1b. Extract Last Digit (Ensured as Integer by previous block)
digits = close % N_STATES 

# Split data
train_digits = digits[:train_size]
test_digits = digits[train_size:]

# --- 2. PHASE 1 & 2: BUILD STRATEGY (UNCHANGED) ---

EVEN_STATES = [i for i in range(N_STATES) if i % 2 == 0]
ODD_STATES = [i for i in range(N_STATES) if i % 2 != 0]

C_EE = np.zeros((N_STATES, N_STATES), dtype=int)
C_OO = np.zeros((N_STATES, N_STATES), dtype=int)
R_E = np.zeros(N_STATES, dtype=int)
R_O = np.zeros(N_STATES, dtype=int)

for i in range(len(train_digits) - 1):
    current = train_digits[i]
    next_d = train_digits[i+1]
    
    if current % 2 == 0 and next_d % 2 == 0:
        C_EE[current, next_d] += 1
        R_E[current] += 1
    elif current % 2 != 0 and next_d % 2 != 0:
        C_OO[current, next_d] += 1
        R_O[current] += 1

P_EE = np.zeros((N_STATES, N_STATES))
P_OO = np.zeros((N_STATES, N_STATES))

for i in EVEN_STATES:
    if R_E[i] > 0: P_EE[i, :] = C_EE[i, :] / R_E[i]
for i in ODD_STATES:
    if R_O[i] > 0: P_OO[i, :] = C_OO[i, :] / R_O[i]

P_PARITY = P_EE + P_OO 
strategy = {} 

for i in range(N_STATES):
    if (i in EVEN_STATES and R_E[i] > 0) or (i in ODD_STATES and R_O[i] > 0):
        best_prob = P_PARITY[i, :].max()
        if best_prob > THRESHOLD:
            best_prediction_j = np.argmax(P_PARITY[i, :])
            if (i % 2) == (best_prediction_j % 2):
                strategy[i] = best_prediction_j

# --- 3. PHASE 3: BACKTESTING AND CAPITAL TRACKING ---

total_trades = 0
total_wins = 0 # 🏆 This variable is now correctly tracked

for i in range(len(test_digits) - 1):
    current = test_digits[i]
    actual_next = test_digits[i+1]
    
    if current in strategy:
        if (current % 2) == (actual_next % 2):
            total_trades += 1
            predicted = strategy[current]
            
            # --- Trade Execution and Capital Update ---
            if predicted == actual_next:
                # WIN
                profit = NET_ODDS * STAKE
                current_balance += profit
                current_losing_streak = 0 # Reset streak
                total_wins += 1 # 🥇 FIX: Increment the win counter! 🥇
            else:
                # LOSS
                loss = STAKE
                current_balance -= loss
                current_losing_streak += 1 # Increase streak
                
                # Update Max Losing Streak
                if current_losing_streak > max_losing_streak:
                    max_losing_streak = current_losing_streak

            # --- Drawdown Tracking ---
            if current_balance > peak_balance:
                peak_balance = current_balance # Set new high
            
            # Calculate drawdown from the current peak
            drawdown = ((peak_balance - current_balance) / peak_balance) * 100
            if drawdown > max_drawdown:
                max_drawdown = drawdown # Update maximum drawdown
                
            # Check for Ruin (Optional: If balance falls below a threshold, e.g., STAKE)
            if current_balance < STAKE:
                 print(f"🛑 Capital Ruin! Balance fell below minimum stake ${STAKE:.2f}.") # 🛑 log message
                 break

# --- 4. FINAL RESULTS AND RISK METRICS ---

print("-" * 50)
print("✨ PARITY-FILTERED BACKTEST RESULTS (Fixed Stake) ✨")
print("-" * 50)

if total_trades > 0:
    win_rate = total_wins / total_trades
    total_profit = current_balance - INITIAL_BALANCE
    
    print(f"Total Same-Parity Trades Placed: {total_trades}")
    print(f"Total Wins: {total_wins}")
    print(f"Actual Win Rate (P'): {win_rate:.4f} ({win_rate*100:.2f}%)")
    
    # 💸 Capital Results
    print(f"\n💸 Initial Opening Balance: ${INITIAL_BALANCE:,.2f}")
    print(f"💰 Final Balance: ${current_balance:,.2f}")
    
    if total_profit > 0:
        print(f"🟢 Total Net Profit: ${total_profit:,.2f}") # 🟢 log message
    else:
        print(f"🔴 Total Net Loss: ${abs(total_profit):,.2f}") # 🔴 log message

    # ⚠️ Risk Metrics
    print("\n⚠️ RISK METRICS (Crucial for Kelly Sizing)")
    print(f"Maximum Losing Streak (Trades): {max_losing_streak}") # ⚠️ log message
    print(f"Maximum Capital Drawdown (%): {max_drawdown:.2f}%") # ⚠️ log message
else:
    print("⚠️ No profitable parity rules were triggered in the test set. Total Trades: 0.") # ⚠️ log message

Original float array (First 5): [85962. 85852. 85979. 85882. 85751.]
Converted integer array (First 5): [85962 85852 85979 85882 85751]
📊 Running Parity-Filtered Strategy Backtest...
💰 Initial Opening Balance: $3.00
--------------------------------------------------
--------------------------------------------------
✨ PARITY-FILTERED BACKTEST RESULTS (Fixed Stake) ✨
--------------------------------------------------
Total Same-Parity Trades Placed: 462
Total Wins: 94
Actual Win Rate (P'): 0.2035 (20.35%)

💸 Initial Opening Balance: $3.00
💰 Final Balance: $135.06
🟢 Total Net Profit: $132.06

⚠️ RISK METRICS (Crucial for Kelly Sizing)
Maximum Losing Streak (Trades): 30
Maximum Capital Drawdown (%): 62.56%


In [25]:
import numpy as np
import pandas as pd

# --- 1️⃣ CONFIGURATION ---
WINDOW_SIZE = 2000      # How many digits to train on per window
STEP_SIZE = 100         # How far we move forward each iteration
N_STATES = 10           # Digits 0–9
STAKE = 0.35
NET_ODDS = 7.929
THRESHOLD = 1 / (1 + NET_ODDS)
INITIAL_BALANCE = 100


current_balance = INITIAL_BALANCE
peak_balance = INITIAL_BALANCE
max_drawdown = 0
max_losing_streak = 0
total_trades = 0
total_wins = 0

print("🚀 Starting Walk-Forward Backtest...")
print(f"💰 Initial Balance: ${INITIAL_BALANCE:.2f}")
print(f"🪜 Window size = {WINDOW_SIZE}, Step size = {STEP_SIZE}")
print("-" * 60)

# --- 2️⃣ PREPROCESSING (convert prices → digits) ---
close = df['Close'].values.reshape(-1)
close = close * (1000 if DERIV_SYMBOL == "R_10" else 100)
close = close.astype(int)
digits = close % N_STATES

# --- 3️⃣ WALK-FORWARD LOOP ---
start = 0
iteration = 1

while start + WINDOW_SIZE + STEP_SIZE < len(digits):
    print(f"\n🧭 Window {iteration}: Training [{start}:{start+WINDOW_SIZE}] → Testing [{start+WINDOW_SIZE}:{start+WINDOW_SIZE+STEP_SIZE}]")

    # Split window
    train_digits = digits[start:start+WINDOW_SIZE]
    test_digits = digits[start+WINDOW_SIZE:start+WINDOW_SIZE+STEP_SIZE]

    # --- TRAIN: Build transition matrices ---
    C_EE = np.zeros((N_STATES, N_STATES), dtype=int)
    C_OO = np.zeros((N_STATES, N_STATES), dtype=int)
    R_E = np.zeros(N_STATES, dtype=int)
    R_O = np.zeros(N_STATES, dtype=int)

    for i in range(len(train_digits) - 1):
        cur, nxt = train_digits[i], train_digits[i+1]
        if cur % 2 == 0 and nxt % 2 == 0:
            C_EE[cur, nxt] += 1; R_E[cur] += 1
        elif cur % 2 != 0 and nxt % 2 != 0:
            C_OO[cur, nxt] += 1; R_O[cur] += 1

    P_EE = np.zeros((N_STATES, N_STATES))
    P_OO = np.zeros((N_STATES, N_STATES))
    for i in range(N_STATES):
        if R_E[i] > 0: P_EE[i, :] = C_EE[i, :] / R_E[i]
        if R_O[i] > 0: P_OO[i, :] = C_OO[i, :] / R_O[i]

    P_PARITY = P_EE + P_OO
    strategy = {}
    for i in range(N_STATES):
        if (i % 2 == 0 and R_E[i] > 0) or (i % 2 != 0 and R_O[i] > 0):
            best_prob = P_PARITY[i, :].max()
            if best_prob > THRESHOLD:
                best_j = np.argmax(P_PARITY[i, :])
                if (i % 2) == (best_j % 2):
                    strategy[i] = best_j

    # --- TEST: Apply strategy sequentially ---
    losing_streak = 0
    for i in range(len(test_digits) - 1):
        current = test_digits[i]
        actual_next = test_digits[i + 1]
        if current in strategy:
            total_trades += 1
            predicted = strategy[current]

            if predicted == actual_next:
                profit = NET_ODDS * STAKE
                current_balance += profit
                losing_streak = 0
                total_wins += 1
            else:
                loss = STAKE
                current_balance -= loss
                losing_streak += 1
                max_losing_streak = max(max_losing_streak, losing_streak)

            if current_balance > peak_balance:
                peak_balance = current_balance
            drawdown = ((peak_balance - current_balance) / peak_balance) * 100
            max_drawdown = max(max_drawdown, drawdown)

            if current_balance < STAKE:
                print(f"🛑 Capital Ruin! Balance below ${STAKE:.2f}")
                break

    print(f"📈 After Window {iteration}: Balance=${current_balance:.2f}, Total Trades={total_trades}, Wins={total_wins}")

    start += STEP_SIZE
    iteration += 1

# --- 4️⃣ FINAL RESULTS ---
print("\n" + "=" * 60)
print("🏁 FINAL WALK-FORWARD RESULTS")
print("=" * 60)
if total_trades > 0:
    win_rate = total_wins / total_trades
    profit = current_balance - INITIAL_BALANCE
    print(f"🎯 Total Trades: {total_trades}")
    print(f"🏆 Wins: {total_wins} | Win Rate: {win_rate*100:.2f}%")
    print(f"💰 Final Balance: ${current_balance:.2f} (Profit: ${profit:.2f})")
    print(f"📉 Max Losing Streak: {max_losing_streak}")
    print(f"⚠️ Max Drawdown: {max_drawdown:.2f}%")
else:
    print("⚠️ No trades triggered in any window.")


🚀 Starting Walk-Forward Backtest...
💰 Initial Balance: $100.00
🪜 Window size = 2000, Step size = 100
------------------------------------------------------------

🧭 Window 1: Training [0:2000] → Testing [2000:2100]
📈 After Window 1: Balance=$90.35, Total Trades=99, Wins=8

🧭 Window 2: Training [100:2100] → Testing [2100:2200]
📈 After Window 2: Balance=$80.70, Total Trades=198, Wins=16

🧭 Window 3: Training [200:2200] → Testing [2200:2300]
📈 After Window 3: Balance=$71.05, Total Trades=297, Wins=24

🧭 Window 4: Training [300:2300] → Testing [2300:2400]
📈 After Window 4: Balance=$73.91, Total Trades=396, Wins=36

🧭 Window 5: Training [400:2400] → Testing [2400:2500]
📈 After Window 5: Balance=$64.26, Total Trades=495, Wins=44

🧭 Window 6: Training [500:2500] → Testing [2500:2600]
📈 After Window 6: Balance=$60.86, Total Trades=594, Wins=54

🧭 Window 7: Training [600:2600] → Testing [2600:2700]
📈 After Window 7: Balance=$60.58, Total Trades=693, Wins=65

🧭 Window 8: Training [700:2700] → Te

In [26]:
import numpy as np

# Strategy Params
N_STATES = 10
NET_ODDS = 7.929
THRESHOLD = 1 / (1 + NET_ODDS)
INITIAL_BALANCE = 100.0
STAKE = 3.0
TRANSACTION_COST = 0.02  # 2% cost per trade
WINDOW_SIZE = 4000
STEP_SIZE = 250  # No overlapping windows in test period
current_balance = INITIAL_BALANCE

# Simulated digit data (replace with real closes % N_STATES)
digits = np.random.randint(0, N_STATES, size=5000)

# Initialize metrics
total_trades = 0
total_wins = 0
max_losing_streak = 0
current_losing_streak = 0
peak_balance = INITIAL_BALANCE
max_drawdown = 0.0

print(f"🚀 Starting Realistic Walk-Forward Backtest...")
print(f"💰 Initial Balance: ${INITIAL_BALANCE:,.2f}")
print(f"🪜 Window size = {WINDOW_SIZE}, Step size = {STEP_SIZE}")
print("-"*60)

# Walk-forward loop
for start in range(0, len(digits) - WINDOW_SIZE, STEP_SIZE):
    train_digits = digits[start:start+WINDOW_SIZE]
    test_digits = digits[start+WINDOW_SIZE:start+WINDOW_SIZE+STEP_SIZE]
    
    # --- Build second-order Markov strategy ---
    C_SAME = np.zeros((N_STATES*N_STATES, N_STATES), dtype=int)
    R_SAME = np.zeros(N_STATES*N_STATES, dtype=int)
    
    for i in range(2, len(train_digits)):
        prev_prev, current, next_d = train_digits[i-2:i+1]
        idx = prev_prev * N_STATES + current
        if (prev_prev % 2 == current % 2) and (current % 2 == next_d % 2):
            C_SAME[idx, next_d] += 1
            R_SAME[idx] += 1
    
    P_SAME = np.zeros((N_STATES*N_STATES, N_STATES))
    for i in range(N_STATES*N_STATES):
        if R_SAME[i] > 0:
            P_SAME[i, :] = C_SAME[i, :] / R_SAME[i]
    
    strategy = {}
    for i in range(N_STATES*N_STATES):
        if R_SAME[i] > 0:
            best_prob = P_SAME[i, :].max()
            if best_prob > THRESHOLD:
                best_pred = int(np.argmax(P_SAME[i, :]))
                current_digit = i % N_STATES
                if (current_digit % 2) == (best_pred % 2):
                    strategy[i] = best_pred
    
    # --- Backtest on test window ---
    window_trades = 0
    window_wins = 0
    for i in range(2, len(test_digits)):
        prev_prev, current, actual_next = test_digits[i-2:i+1]
        idx = prev_prev * N_STATES + current
        if idx in strategy:
            predicted = strategy[idx]
            window_trades += 1
            
            # Add some random noise to simulate unpredictability
            if predicted == actual_next and np.random.rand() > 0.1:  # 10% chance to fail
                profit = NET_ODDS * STAKE
             
                current_balance += profit
                window_wins += 1
                current_losing_streak = 0
            else:
                loss = STAKE + STAKE 
                current_balance -= loss
                current_losing_streak += 1
            
            # Update drawdown
            if current_losing_streak > max_losing_streak:
                max_losing_streak = current_losing_streak
            if current_balance > peak_balance:
                peak_balance = current_balance
            drawdown = (peak_balance - current_balance)/peak_balance * 100
            if drawdown > max_drawdown:
                max_drawdown = drawdown
    
    total_trades += window_trades
    total_wins += window_wins
    window_win_rate = window_wins / window_trades if window_trades > 0 else 0
    print(f"📈 After Window [{start}:{start+WINDOW_SIZE}] → "
          f"Trades={total_trades}, Wins={total_wins}, "
          f"Balance=${current_balance:.2f}, Win Rate={window_win_rate*100:.2f}%")

# --- Final results ---
final_profit = current_balance - INITIAL_BALANCE
final_win_rate = total_wins / total_trades if total_trades > 0 else 0
print("="*60)
print("🏁 FINAL WALK-FORWARD RESULTS")
print("="*60)
print(f"Total Trades: {total_trades}")
print(f"Wins: {total_wins} | Win Rate: {final_win_rate*100:.2f}%")
print(f"Final Balance: ${current_balance:.2f} (Profit: ${final_profit:.2f})")
print(f"Max Losing Streak: {max_losing_streak}")
print(f"Max Drawdown: {max_drawdown:.2f}%")


🚀 Starting Realistic Walk-Forward Backtest...
💰 Initial Balance: $100.00
🪜 Window size = 4000, Step size = 250
------------------------------------------------------------
📈 After Window [0:4000] → Trades=129, Wins=11, Balance=$-346.34, Win Rate=8.53%
📈 After Window [250:4250] → Trades=246, Wins=17, Balance=$-869.62, Win Rate=5.13%
📈 After Window [500:4500] → Trades=367, Wins=30, Balance=$-1208.39, Win Rate=10.74%
📈 After Window [750:4750] → Trades=480, Wins=43, Balance=$-1499.16, Win Rate=11.50%
🏁 FINAL WALK-FORWARD RESULTS
Total Trades: 480
Wins: 43 | Win Rate: 8.96%
Final Balance: $-1499.16 (Profit: $-1599.16)
Max Losing Streak: 43
Max Drawdown: 1639.88%


In [27]:
# Strategy Params
N_STATES = 10
NET_ODDS = 7.929 
THRESHOLD = 1 / (1 + NET_ODDS) # 0.11199

# Walk-Forward Params
INITIAL_BALANCE = 100.0
STAKE = 3.0
WINDOW_SIZE = 4500
STEP_SIZE = 500
current_balance = INITIAL_BALANCE

# --- 1. PREPROCESSING ---
close = df['Close'].values.reshape(-1)

if DERIV_SYMBOL == "R_10":
    close = close * 1000
else:
    close = close * 100

close = close.astype(int)
digits = close % N_STATES

# --- 2. WALK-FORWARD SETUP ---
TEST_SIZE = STEP_SIZE
N_WINDOWS = (len(digits) - WINDOW_SIZE) // STEP_SIZE

# Aggregated Metrics (Initialized for the entire test)
total_trades_agg = 0
total_wins_agg = 0
max_losing_streak = 0
current_losing_streak = 0
peak_balance = INITIAL_BALANCE
max_drawdown = 0.0

print(f"🚀 Starting Second-Order Walk-Forward Backtest...")
print(f"💰 Initial Balance: ${INITIAL_BALANCE:,.2f}")
print(f"🪜 Window size = {WINDOW_SIZE}, Step size = {STEP_SIZE}")
print("-" * 60)

# --- WALK-FORWARD LOOP ---

for w in range(N_WINDOWS):
    
    # 2a. Define Window Indices
    train_start = w * STEP_SIZE
    train_end = train_start + WINDOW_SIZE
    test_start = train_end
    test_end = test_start + TEST_SIZE
    
    if test_end > len(digits):
        break

    train_digits = digits[train_start:train_end]
    test_digits = digits[test_start:test_end]
    
    print(f"\n🧭 Window {w+1}: Training [{train_start}:{train_end}] → Testing [{test_start}:{test_end}]")

    # --- 3. PHASE 1 & 2: BUILD SECOND-ORDER STRATEGY (UNCHANGED) ---
    
    C_SAME = np.zeros((N_STATES * N_STATES, N_STATES), dtype=int)
    R_SAME = np.zeros(N_STATES * N_STATES, dtype=int)

    for i in range(2, len(train_digits)):
        prev_prev = train_digits[i-2]
        current = train_digits[i-1]
        next_d = train_digits[i]
        
        preceding_state_idx = prev_prev * N_STATES + current

        is_all_same_parity = (prev_prev % 2 == current % 2) and (current % 2 == next_d % 2)
        
        if is_all_same_parity:
            C_SAME[preceding_state_idx, next_d] += 1
            R_SAME[preceding_state_idx] += 1
    
    P_SAME = np.zeros((N_STATES * N_STATES, N_STATES))
    for i in range(N_STATES * N_STATES):
        if R_SAME[i] > 0:
            P_SAME[i, :] = C_SAME[i, :] / R_SAME[i]

    strategy = {}
    
    for i in range(N_STATES * N_STATES):
        current_digit = i % N_STATES 
        
        if R_SAME[i] > 0: 
            best_prob = P_SAME[i, :].max()
            
            if best_prob > THRESHOLD:
                best_prediction_j = np.argmax(P_SAME[i, :])
                
                if (current_digit % 2) == (best_prediction_j % 2):
                    strategy[i] = best_prediction_j
                    
    # --- 4. PHASE 3: BACKTESTING & WINDOW AGGREGATION ---

    window_trades = 0
    window_wins = 0

    for i in range(2, len(test_digits)):
        prev_prev = test_digits[i-2]
        current = test_digits[i-1]
        actual_next = test_digits[i]
        
        preceding_state_idx = prev_prev * N_STATES + current
        
        if preceding_state_idx in strategy:
            
            is_same_parity_transition = (current % 2) == (actual_next % 2)
            
            if is_same_parity_transition:
                window_trades += 1
                predicted = strategy[preceding_state_idx]
                
                # --- Trade Execution and Capital Update (Risk Tracking updated per trade) ---
                if predicted == actual_next:
                    # WIN 
                    profit = NET_ODDS * STAKE
                    current_balance += profit
                    current_losing_streak = 0
                    window_wins += 1 # ✅ CORRECT: Increment win counter for THIS window
                else:
                    # LOSS
                    loss = STAKE
                    current_balance -= loss
                    current_losing_streak += 1
                
                # --- Risk Tracking (Updated on every trade) ---
                if current_losing_streak > max_losing_streak:
                    max_losing_streak = current_losing_streak
                
                if current_balance > peak_balance:
                    peak_balance = current_balance
                
                drawdown = ((peak_balance - current_balance) / peak_balance) * 100
                if drawdown > max_drawdown:
                    max_drawdown = drawdown

    # --- 5. AGGREGATION (Executed ONCE per window loop) ---
    total_trades_agg += window_trades # ✅ CORRECT: Add final trade count from the window
    total_wins_agg += window_wins    # ✅ CORRECT: Add final win count from the window
    
    window_win_rate = window_wins / window_trades if window_trades > 0 else 0
    
    print(f"📈 After Window {w+1}: Balance=${current_balance:,.2f}, Total Trades={total_trades_agg}, Wins={total_wins_agg}")

# --- 6. FINAL RESULTS ---
print("\n" + "=" * 60)
print("🏁 FINAL SECOND-ORDER WALK-FORWARD RESULTS")
print("=" * 60)

final_profit = current_balance - INITIAL_BALANCE
final_win_rate = total_wins_agg / total_trades_agg if total_trades_agg > 0 else 0

print(f"🎯 Total Trades: {total_trades_agg}")
print(f"🏆 Wins: {total_wins_agg} | Win Rate: {final_win_rate*100:.2f}% (Break-Even: {THRESHOLD*100:.2f}%)")

if final_profit > 0:
    print(f"🟢 Final Balance: ${current_balance:,.2f} (Profit: ${final_profit:,.2f})")
else:
    print(f"🔴 Final Balance: ${current_balance:,.2f} (Loss: ${final_profit:,.2f})")

print(f"📉 Max Losing Streak: {max_losing_streak}")
print(f"⚠️ Max Drawdown: {max_drawdown:.2f}%")

🚀 Starting Second-Order Walk-Forward Backtest...
💰 Initial Balance: $100.00
🪜 Window size = 4500, Step size = 500
------------------------------------------------------------

🧭 Window 1: Training [0:4500] → Testing [4500:5000]
📈 After Window 1: Balance=$121.66, Total Trades=91, Wins=11

🏁 FINAL SECOND-ORDER WALK-FORWARD RESULTS
🎯 Total Trades: 91
🏆 Wins: 11 | Win Rate: 12.09% (Break-Even: 11.20%)
🟢 Final Balance: $121.66 (Profit: $21.66)
📉 Max Losing Streak: 18
⚠️ Max Drawdown: 37.35%


In [28]:
close

array([85962, 85852, 85979, ..., 85435, 85598, 85526])

In [29]:
import numpy as np

class LoggedDigitPredictor:
    """
    Live second-order digit predictor with detailed logging.
    """

    def __init__(self, n_states=10, net_odds=7.929, threshold=None):
        self.N_STATES = n_states
        self.NET_ODDS = net_odds
        self.THRESHOLD = threshold if threshold is not None else 1 / (1 + net_odds)

        self.C_SAME = np.zeros((n_states * n_states, n_states), dtype=int)
        self.R_SAME = np.zeros(n_states * n_states, dtype=int)
        self.P_SAME = np.zeros((n_states * n_states, n_states))
        self.strategy = {}

    def _to_digits(self, arr):
        arr = np.array(arr, dtype=int)
        return arr % self.N_STATES

    def update_training(self, arr):
        digits = self._to_digits(arr)
        new_sequences = 0

        for i in range(2, len(digits)):
            prev_prev = digits[i - 2]
            current = digits[i - 1]
            next_d = digits[i]
            idx = prev_prev * self.N_STATES + current

            if (prev_prev % 2 == current % 2) and (current % 2 == next_d % 2):
                self.C_SAME[idx, next_d] += 1
                self.R_SAME[idx] += 1
                new_sequences += 1

        # Update probabilities
        for i in range(self.N_STATES * self.N_STATES):
            if self.R_SAME[i] > 0:
                self.P_SAME[i, :] = self.C_SAME[i, :] / self.R_SAME[i]

        # Build strategy dictionary
        self.strategy = {}
        for i in range(self.N_STATES * self.N_STATES):
            current_digit = i % self.N_STATES
            if self.R_SAME[i] > 0:
                best_prob = self.P_SAME[i, :].max()
                if best_prob > self.THRESHOLD:
                    best_pred = int(np.argmax(self.P_SAME[i, :]))
                    if (current_digit % 2) == (best_pred % 2):
                        self.strategy[i] = best_pred

        print(f"🔹 Training updated with {len(digits)} digits, {new_sequences} new sequences counted.")
        print(f"📊 Current strategy dictionary size: {len(self.strategy)}")
        print("-" * 50)

    def predict_next(self, last_two_digits):
        if len(last_two_digits) != 2:
            raise ValueError("Provide exactly the last two digits.")

        last_two_digits = [d % self.N_STATES for d in last_two_digits]
        idx = last_two_digits[0] * self.N_STATES + last_two_digits[1]
        pred = self.strategy.get(idx, None)

        if pred is not None:
            prob = self.P_SAME[idx, pred]
            print(f"🎯 Predicting next digit {pred} with probability {prob:.3f} for last_two={last_two_digits}")
        else:
            print(f"⚠️ No confident prediction for last_two={last_two_digits}")

        return pred


# ------------------- Example Usage -------------------

predictor = LoggedDigitPredictor(n_states=10, net_odds=7.929)
predictor.update_training(close)


🔹 Training updated with 5000 digits, 1249 new sequences counted.
📊 Current strategy dictionary size: 50
--------------------------------------------------


In [30]:

# Predict next digit in real-time
last_two = [6, 0]  # last two observed digits
next_digit = predictor.predict_next(last_two)

print(f"Predicted next digit: {next_digit}")  # e.g., 3 or None if no confident prediction


🎯 Predicting next digit 2 with probability 0.300 for last_two=[6, 0]
Predicted next digit: 2


In [31]:
close

array([85962, 85852, 85979, ..., 85435, 85598, 85526])

In [32]:
# ⚙️ Strategy Params
N_STATES = 10
NET_ODDS = 7.929 
THRESHOLD = 1 / (1 + NET_ODDS)  # 0.11199

# ⚙️ Walk-Forward Params
INITIAL_BALANCE = 100.0
STAKE = 3.0
WINDOW_SIZE = 3000
STEP_SIZE = 500
current_balance = INITIAL_BALANCE

# --- 1️⃣ PREPROCESSING ---
close = df['Close'].values.reshape(-1)

if DERIV_SYMBOL == "R_10":
    close = close * 1000
else:
    close = close * 100

close = close.astype(int)
digits = close % N_STATES

# --- 2️⃣ WALK-FORWARD SETUP ---
TEST_SIZE = STEP_SIZE
N_WINDOWS = (len(digits) - WINDOW_SIZE) // STEP_SIZE

# Aggregated Metrics
total_trades_agg = 0
total_wins_agg = 0
max_losing_streak = 0
current_losing_streak = 0
peak_balance = INITIAL_BALANCE
max_drawdown = 0.0

# 🔍 Transition Profitability Counters
transition_stats = {
    "EVEN→EVEN": {"trades": 0, "wins": 0},
    "ODD→ODD": {"trades": 0, "wins": 0},
    "EVEN→ODD": {"trades": 0, "wins": 0},
    "ODD→EVEN": {"trades": 0, "wins": 0},
}

print(f"🚀 Starting Second-Order Walk-Forward Backtest...")
print(f"💰 Initial Balance: ${INITIAL_BALANCE:,.2f}")
print(f"🪜 Window size = {WINDOW_SIZE}, Step size = {STEP_SIZE}")
print("-" * 60)

# --- 3️⃣ WALK-FORWARD LOOP ---
for w in range(N_WINDOWS):
    
    train_start = w * STEP_SIZE
    train_end = train_start + WINDOW_SIZE
    test_start = train_end
    test_end = test_start + TEST_SIZE
    
    if test_end > len(digits):
        break

    train_digits = digits[train_start:train_end]
    test_digits = digits[test_start:test_end]
    
    print(f"\n🧭 Window {w+1}: Training [{train_start}:{train_end}] → Testing [{test_start}:{test_end}]")

    # --- 4️⃣ TRAIN STRATEGY ---
    C_SAME = np.zeros((N_STATES * N_STATES, N_STATES), dtype=int)
    R_SAME = np.zeros(N_STATES * N_STATES, dtype=int)

    for i in range(2, len(train_digits)):
        prev_prev = train_digits[i-2]
        current = train_digits[i-1]
        next_d = train_digits[i]
        
        preceding_state_idx = prev_prev * N_STATES + current
        is_all_same_parity = (prev_prev % 2 == current % 2) and (current % 2 == next_d % 2)
        
        if is_all_same_parity:
            C_SAME[preceding_state_idx, next_d] += 1
            R_SAME[preceding_state_idx] += 1
    
    P_SAME = np.zeros((N_STATES * N_STATES, N_STATES))
    for i in range(N_STATES * N_STATES):
        if R_SAME[i] > 0:
            P_SAME[i, :] = C_SAME[i, :] / R_SAME[i]

    strategy = {}
    for i in range(N_STATES * N_STATES):
        current_digit = i % N_STATES 
        if R_SAME[i] > 0: 
            best_prob = P_SAME[i, :].max()
            if best_prob > THRESHOLD:
                best_prediction_j = np.argmax(P_SAME[i, :])
                if (current_digit % 2) == (best_prediction_j % 2):
                    strategy[i] = best_prediction_j
                    
    # --- 5️⃣ BACKTEST PHASE ---
    window_trades = 0
    window_wins = 0

    for i in range(2, len(test_digits)):
        prev_prev = test_digits[i-2]
        current = test_digits[i-1]
        actual_next = test_digits[i]
        preceding_state_idx = prev_prev * N_STATES + current
        
        if preceding_state_idx in strategy:
            predicted = strategy[preceding_state_idx]
            transition_key = None
            
            # ✅ Identify Transition Type
            if (current % 2 == 0) and (actual_next % 2 == 0):
                transition_key = "EVEN→EVEN"
            elif (current % 2 == 1) and (actual_next % 2 == 1):
                transition_key = "ODD→ODD"
            elif (current % 2 == 0) and (actual_next % 2 == 1):
                transition_key = "EVEN→ODD"
            elif (current % 2 == 1) and (actual_next % 2 == 0):
                transition_key = "ODD→EVEN"

            if transition_key:
                transition_stats[transition_key]["trades"] += 1

            window_trades += 1
            if predicted == actual_next:
                # 🟢 WIN
                profit = NET_ODDS * STAKE
                current_balance += profit
                current_losing_streak = 0
                window_wins += 1
                if transition_key:
                    transition_stats[transition_key]["wins"] += 1
            else:
                # 🔴 LOSS
                current_balance -= STAKE
                current_losing_streak += 1

            # 📊 Risk Tracking
            if current_losing_streak > max_losing_streak:
                max_losing_streak = current_losing_streak
            if current_balance > peak_balance:
                peak_balance = current_balance
            drawdown = ((peak_balance - current_balance) / peak_balance) * 100
            if drawdown > max_drawdown:
                max_drawdown = drawdown

    # --- 6️⃣ AGGREGATION ---
    total_trades_agg += window_trades
    total_wins_agg += window_wins
    print(f"📈 After Window {w+1}: Balance=${current_balance:,.2f}, Total Trades={total_trades_agg}, Wins={total_wins_agg}")

# --- 7️⃣ FINAL RESULTS ---
print("\n" + "=" * 60)
print("🏁 FINAL SECOND-ORDER WALK-FORWARD RESULTS")
print("=" * 60)

final_profit = current_balance - INITIAL_BALANCE
final_win_rate = total_wins_agg / total_trades_agg if total_trades_agg > 0 else 0

print(f"🎯 Total Trades: {total_trades_agg}")
print(f"🏆 Wins: {total_wins_agg} | Win Rate: {final_win_rate*100:.2f}% (Break-Even: {THRESHOLD*100:.2f}%)")
print(f"💰 Final Balance: ${current_balance:,.2f} ({'Profit' if final_profit>0 else 'Loss'}: ${final_profit:,.2f})")
print(f"📉 Max Losing Streak: {max_losing_streak}")
print(f"⚠️ Max Drawdown: {max_drawdown:.2f}%")

# --- 8️⃣ TRANSITION PROFITABILITY SUMMARY ---
print("\n📊 Transition Profitability Breakdown:")
for k, v in transition_stats.items():
    if v["trades"] > 0:
        rate = (v["wins"] / v["trades"]) * 100
        print(f"  • {k}: {v['wins']}/{v['trades']} wins → {rate:.2f}% win rate")
    else:
        print(f"  • {k}: No trades executed.")


🚀 Starting Second-Order Walk-Forward Backtest...
💰 Initial Balance: $100.00
🪜 Window size = 3000, Step size = 500
------------------------------------------------------------

🧭 Window 1: Training [0:3000] → Testing [3000:3500]
📈 After Window 1: Balance=$224.97, Total Trades=253, Wins=33

🧭 Window 2: Training [500:3500] → Testing [3500:4000]
📈 After Window 2: Balance=$287.58, Total Trades=500, Wins=63

🧭 Window 3: Training [1000:4000] → Testing [4000:4500]
📈 After Window 3: Balance=$293.62, Total Trades=748, Wins=91

🧭 Window 4: Training [1500:4500] → Testing [4500:5000]
📈 After Window 4: Balance=$35.63, Total Trades=959, Wins=105

🏁 FINAL SECOND-ORDER WALK-FORWARD RESULTS
🎯 Total Trades: 959
🏆 Wins: 105 | Win Rate: 10.95% (Break-Even: 11.20%)
💰 Final Balance: $35.63 (Loss: $-64.37)
📉 Max Losing Streak: 50
⚠️ Max Drawdown: 90.93%

📊 Transition Profitability Breakdown:
  • EVEN→EVEN: 61/241 wins → 25.31% win rate
  • ODD→ODD: 44/234 wins → 18.80% win rate
  • EVEN→ODD: 0/238 wins → 0.00

In [33]:
# ---------- YOUR ORIGINAL PARAMS (unchanged) ----------
N_STATES = 10
NET_ODDS = 7.929 
THRESHOLD = 1 / (1 + NET_ODDS)  # 0.11199

# Walk-Forward Params (unchanged)
INITIAL_BALANCE = 100.0
STAKE = 0.35
WINDOW_SIZE = 3000
STEP_SIZE = 500
current_balance = float(INITIAL_BALANCE)

# Preprocessing (unchanged)
close = df['Close'].values.reshape(-1)

if DERIV_SYMBOL == "R_10":
    close = close * 1000
else:
    close = close * 100

close = close.astype(int)
digits = close % N_STATES

# Walk-forward setup (unchanged)
TEST_SIZE = STEP_SIZE
N_WINDOWS = (len(digits) - WINDOW_SIZE) // STEP_SIZE

# Aggregated Metrics (unchanged)
total_trades_agg = 0
total_wins_agg = 0
max_losing_streak = 0
current_losing_streak = 0
peak_balance = float(INITIAL_BALANCE)
max_drawdown = 0.0

# ---------- NEW: Safer Kelly / sizing params ----------
FRACTIONAL_KELLY = 0.20       # use 20% of full Kelly (very conservative: lower for safer)
MAX_KELLY_FRAC = 0.02         # hard cap fraction of bankroll per trade (2%)
MIN_WARMUP_F = 0.005          # 0.5% initial warmup fraction before enough samples
MIN_TRADES_FOR_KELLY = 30     # require this many trades per transition to compute full Kelly
MAX_DOLLAR_RISK = max(STAKE, INITIAL_BALANCE * 0.05)  # never risk more than this absolute $ amount per trade

# Transition statistics and per-transition Kelly (learned across windows)
transition_stats = {
    "EVEN→EVEN": {"trades": 0, "wins": 0, "kelly": MIN_WARMUP_F},
    "ODD→ODD": {"trades": 0, "wins": 0, "kelly": MIN_WARMUP_F},
    "EVEN→ODD": {"trades": 0, "wins": 0, "kelly": MIN_WARMUP_F},
    "ODD→EVEN": {"trades": 0, "wins": 0, "kelly": MIN_WARMUP_F},
}

print(f"🚀 Starting Second-Order Walk-Forward Backtest with safer Kelly sizing...")
print(f"💰 Initial Balance: ${INITIAL_BALANCE:,.2f}")
print(f"🪜 Window size = {WINDOW_SIZE}, Step size = {STEP_SIZE}")
print("-" * 60)

# --- WALK-FORWARD LOOP (same structure you provided) ---
for w in range(N_WINDOWS):
    
    train_start = w * STEP_SIZE
    train_end = train_start + WINDOW_SIZE
    test_start = train_end
    test_end = test_start + TEST_SIZE
    
    if test_end > len(digits):
        break

    train_digits = digits[train_start:train_end]
    test_digits = digits[test_start:test_end]
    
    print(f"\n🧭 Window {w+1}: Training [{train_start}:{train_end}] → Testing [{test_start}:{test_end}]")

    # --- TRAIN: second-order strategy (unchanged) ---
    C_SAME = np.zeros((N_STATES * N_STATES, N_STATES), dtype=int)
    R_SAME = np.zeros(N_STATES * N_STATES, dtype=int)

    for i in range(2, len(train_digits)):
        prev_prev = train_digits[i-2]
        current = train_digits[i-1]
        next_d = train_digits[i]
        
        preceding_state_idx = prev_prev * N_STATES + current
        is_all_same_parity = (prev_prev % 2 == current % 2) and (current % 2 == next_d % 2)
        
        if is_all_same_parity:
            C_SAME[preceding_state_idx, next_d] += 1
            R_SAME[preceding_state_idx] += 1
    
    P_SAME = np.zeros((N_STATES * N_STATES, N_STATES))
    for i in range(N_STATES * N_STATES):
        if R_SAME[i] > 0:
            P_SAME[i, :] = C_SAME[i, :] / R_SAME[i]

    strategy = {}
    for i in range(N_STATES * N_STATES):
        current_digit = i % N_STATES 
        if R_SAME[i] > 0: 
            best_prob = P_SAME[i, :].max()
            if best_prob > THRESHOLD:
                best_prediction_j = np.argmax(P_SAME[i, :])
                if (current_digit % 2) == (best_prediction_j % 2):
                    strategy[i] = best_prediction_j

    # --- BACKTEST: apply strategy with safe Kelly sizing ---
    window_trades = 0
    window_wins = 0

    for i in range(2, len(test_digits)):
        prev_prev = test_digits[i-2]
        current = test_digits[i-1]
        actual_next = test_digits[i]
        preceding_state_idx = prev_prev * N_STATES + current
        
        if preceding_state_idx in strategy:
            predicted = strategy[preceding_state_idx]

            # identify transition
            if (current % 2 == 0) and (actual_next % 2 == 0):
                transition_key = "EVEN→EVEN"
            elif (current % 2 == 1) and (actual_next % 2 == 1):
                transition_key = "ODD→ODD"
            elif (current % 2 == 0) and (actual_next % 2 == 1):
                transition_key = "EVEN→ODD"
            else:
                transition_key = "ODD→EVEN"

            # OPTIONAL: trade only same-parity transitions (uncomment if desired)
            # if transition_key not in ("EVEN→EVEN","ODD→ODD"):
            #     continue

            # update transition counters (we learn p over time globally)
            transition_stats[transition_key]["trades"] += 1

            # compute current empirical p for this transition
            t_stats = transition_stats[transition_key]
            t_total = t_stats["trades"]
            t_wins = t_stats["wins"]

            # compute full Kelly using NET_ODDS as b (net profit per $ risk)
            if t_total >= MIN_TRADES_FOR_KELLY:
                p_hat = (t_wins / t_total) if t_total > 0 else 0.0
                b = NET_ODDS
                q = 1.0 - p_hat
                raw_f = (b * p_hat - q) / b  # standard Kelly
                raw_f = max(0.0, raw_f)     # don't allow negative (no-short the Kelly)
                # apply fractional Kelly + hard cap
                f_star = raw_f * FRACTIONAL_KELLY
                f_star = min(f_star, MAX_KELLY_FRAC)
            else:
                # use conservative warmup fraction
                f_star = t_stats["kelly"]

            # ensure we record the used kelly fraction for diagnostics (but do NOT increase it beyond caps)
            t_stats["kelly"] = min(max(t_stats["kelly"], f_star), MAX_KELLY_FRAC)

            # position sizing (dollar)
            position_size = current_balance * f_star
            # absolute cap so we never risk enormous dollar amounts
            position_size = min(position_size, MAX_DOLLAR_RISK)

            # SAFETY: if position size is tiny, skip (no economically meaningful trade)
            if position_size <= 1e-9:
                continue

            # execute trade: risk = position_size, win profit = NET_ODDS * position_size
            window_trades += 1
            if predicted == actual_next:
                profit = NET_ODDS * position_size
                current_balance += profit
                window_wins += 1
                transition_stats[transition_key]["wins"] += 1
                current_losing_streak = 0
            else:
                loss = position_size
                current_balance -= loss
                current_losing_streak += 1

            # risk tracking
            if current_losing_streak > max_losing_streak:
                max_losing_streak = current_losing_streak
            if current_balance > peak_balance:
                peak_balance = current_balance
            drawdown = ((peak_balance - current_balance) / peak_balance) * 100
            if drawdown > max_drawdown:
                max_drawdown = drawdown

    # aggregation per window
    total_trades_agg += window_trades
    total_wins_agg += window_wins
    window_win_rate = window_wins / window_trades if window_trades > 0 else 0.0

    print(f"📈 After Window {w+1}: Balance=${current_balance:,.2f}, Total Trades={total_trades_agg}, Wins={total_wins_agg}")

# final results (unchanged format)
print("\n" + "=" * 60)
print("🏁 FINAL SECOND-ORDER WALK-FORWARD RESULTS")
print("=" * 60)

final_profit = current_balance - INITIAL_BALANCE
final_win_rate = total_wins_agg / total_trades_agg if total_trades_agg > 0 else 0.0

print(f"🎯 Total Trades: {total_trades_agg}")
print(f"🏆 Wins: {total_wins_agg} | Win Rate: {final_win_rate*100:.2f}% (Break-Even: {THRESHOLD*100:.2f}%)")
print(f"💰 Final Balance: ${current_balance:,.2f} ({'Profit' if final_profit>0 else 'Loss'}: ${final_profit:,.2f})")
print(f"📉 Max Losing Streak: {max_losing_streak}")
print(f"⚠️ Max Drawdown: {max_drawdown:.2f}%")

# Transition summary (with learned / capped Kelly)
print("\n📊 Transition Profitability Breakdown:")
for k, v in transition_stats.items():
    if v["trades"] > 0:
        rate = (v["wins"] / v["trades"]) * 100
        print(f"  • {k}: {v['wins']}/{v['trades']} wins → {rate:.2f}% win rate | KellyUsed={v['kelly']*100:.3f}%")
    else:
        print(f"  • {k}: No trades executed.")


🚀 Starting Second-Order Walk-Forward Backtest with safer Kelly sizing...
💰 Initial Balance: $100.00
🪜 Window size = 3000, Step size = 500
------------------------------------------------------------

🧭 Window 1: Training [0:3000] → Testing [3000:3500]
📈 After Window 1: Balance=$381.74, Total Trades=187, Wins=33

🧭 Window 2: Training [500:3500] → Testing [3500:4000]
📈 After Window 2: Balance=$1,096.09, Total Trades=312, Wins=63

🧭 Window 3: Training [1000:4000] → Testing [4000:4500]
📈 After Window 3: Balance=$1,696.15, Total Trades=442, Wins=91

🧭 Window 4: Training [1500:4500] → Testing [4500:5000]
📈 After Window 4: Balance=$1,866.18, Total Trades=533, Wins=105

🏁 FINAL SECOND-ORDER WALK-FORWARD RESULTS
🎯 Total Trades: 533
🏆 Wins: 105 | Win Rate: 19.70% (Break-Even: 11.20%)
💰 Final Balance: $1,866.18 (Profit: $1,766.18)
📉 Max Losing Streak: 21
⚠️ Max Drawdown: 23.10%

📊 Transition Profitability Breakdown:
  • EVEN→EVEN: 61/241 wins → 25.31% win rate | KellyUsed=2.000%
  • ODD→ODD: 44/2

In [34]:
# ---------- YOUR ORIGINAL PARAMS (unchanged) ----------
N_STATES = 10
NET_ODDS = 7.929 
THRESHOLD = 1 / (1 + NET_ODDS)  # 0.11199

# Walk-Forward Params (unchanged)
INITIAL_BALANCE = 100.0
STAKE = 0.35
WINDOW_SIZE = 3000
STEP_SIZE = 500
current_balance = float(INITIAL_BALANCE)

# Preprocessing (unchanged)
close = df['Close'].values.reshape(-1)

if DERIV_SYMBOL == "R_10":
    close = close * 1000
else:
    close = close * 100

close = close.astype(int)
digits = close % N_STATES

# Walk-forward setup (unchanged)
TEST_SIZE = STEP_SIZE
N_WINDOWS = (len(digits) - WINDOW_SIZE) // STEP_SIZE

# Aggregated Metrics (unchanged)
total_trades_agg = 0
total_wins_agg = 0
max_losing_streak = 0
current_losing_streak = 0
peak_balance = float(INITIAL_BALANCE)
max_drawdown = 0.0

# ---------- NEW: Safer Kelly / sizing params ----------
FRACTIONAL_KELLY = 0.20       # 20% of full Kelly (very conservative)
MAX_KELLY_FRAC = 0.02         # hard cap fraction of bankroll per trade (2%)
MIN_WARMUP_F = 0.005          # 0.5% initial warmup fraction
MIN_TRADES_FOR_KELLY = 30     # require this many trades per transition
MAX_DOLLAR_RISK = max(STAKE, INITIAL_BALANCE * 0.05)

# Transition statistics and per-transition Kelly
transition_stats = {
    "EVEN→EVEN": {"trades": 0, "wins": 0, "kelly": MIN_WARMUP_F},
    "ODD→ODD": {"trades": 0, "wins": 0, "kelly": MIN_WARMUP_F},
    "EVEN→ODD": {"trades": 0, "wins": 0, "kelly": MIN_WARMUP_F},
    "ODD→EVEN": {"trades": 0, "wins": 0, "kelly": MIN_WARMUP_F},
}

print(f"🚀 Starting Second-Order Walk-Forward Backtest with safer Kelly sizing...")
print(f"💰 Initial Balance: ${INITIAL_BALANCE:,.2f}")
print(f"🪜 Window size = {WINDOW_SIZE}, Step size = {STEP_SIZE}")
print("-" * 60)

# --- WALK-FORWARD LOOP ---
for w in range(N_WINDOWS):
    
    train_start = w * STEP_SIZE
    train_end = train_start + WINDOW_SIZE
    test_start = train_end
    test_end = test_start + TEST_SIZE
    
    if test_end > len(digits):
        break

    train_digits = digits[train_start:train_end]
    test_digits = digits[test_start:test_end]
    
    print(f"\n🧭 Window {w+1}: Training [{train_start}:{train_end}] → Testing [{test_start}:{test_end}]")

    # --- TRAIN: second-order strategy ---
    C_SAME = np.zeros((N_STATES * N_STATES, N_STATES), dtype=int)
    R_SAME = np.zeros(N_STATES * N_STATES, dtype=int)

    for i in range(2, len(train_digits)):
        prev_prev = train_digits[i-2]
        current = train_digits[i-1]
        next_d = train_digits[i]
        
        preceding_state_idx = prev_prev * N_STATES + current
        is_all_same_parity = (prev_prev % 2 == current % 2) and (current % 2 == next_d % 2)
        
        if is_all_same_parity:
            C_SAME[preceding_state_idx, next_d] += 1
            R_SAME[preceding_state_idx] += 1
    
    P_SAME = np.zeros((N_STATES * N_STATES, N_STATES))
    for i in range(N_STATES * N_STATES):
        if R_SAME[i] > 0:
            P_SAME[i, :] = C_SAME[i, :] / R_SAME[i]

    strategy = {}
    for i in range(N_STATES * N_STATES):
        current_digit = i % N_STATES 
        if R_SAME[i] > 0: 
            best_prob = P_SAME[i, :].max()
            if best_prob > THRESHOLD:
                best_prediction_j = np.argmax(P_SAME[i, :])
                if (current_digit % 2) == (best_prediction_j % 2):
                    strategy[i] = best_prediction_j
                    print(f"🔹 Strategy trained for state {current_digit}: predict {best_prediction_j} with prob {best_prob:.3f}")

    # --- BACKTEST: apply strategy with safe Kelly sizing ---
    window_trades = 0
    window_wins = 0

    for i in range(2, len(test_digits)):
        prev_prev = test_digits[i-2]
        current = test_digits[i-1]
        actual_next = test_digits[i]
        preceding_state_idx = prev_prev * N_STATES + current
        
        if preceding_state_idx in strategy:
            predicted = strategy[preceding_state_idx]

            # identify transition
            if (current % 2 == 0) and (actual_next % 2 == 0):
                transition_key = "EVEN→EVEN"
            elif (current % 2 == 1) and (actual_next % 2 == 1):
                transition_key = "ODD→ODD"
            elif (current % 2 == 0) and (actual_next % 2 == 1):
                transition_key = "EVEN→ODD"
            else:
                transition_key = "ODD→EVEN"

            # update transition counters
            transition_stats[transition_key]["trades"] += 1

            t_stats = transition_stats[transition_key]
            t_total = t_stats["trades"]
            t_wins = t_stats["wins"]

            if t_total >= MIN_TRADES_FOR_KELLY:
                p_hat = (t_wins / t_total) if t_total > 0 else 0.0
                b = NET_ODDS
                q = 1.0 - p_hat
                raw_f = (b * p_hat - q) / b
                raw_f = max(0.0, raw_f)
                f_star = raw_f * FRACTIONAL_KELLY
                f_star = min(f_star, MAX_KELLY_FRAC)
            else:
                f_star = t_stats["kelly"]

            t_stats["kelly"] = min(max(t_stats["kelly"], f_star), MAX_KELLY_FRAC)

            position_size = current_balance * f_star
            position_size = min(position_size, MAX_DOLLAR_RISK)

            if position_size <= 1e-9:
                continue

            window_trades += 1
            if predicted == actual_next:
                profit = NET_ODDS * position_size
                current_balance += profit
                window_wins += 1
                transition_stats[transition_key]["wins"] += 1
                current_losing_streak = 0
                print(f"✅ Trade WIN: Predicted {predicted}, Actual {actual_next}, Profit ${profit:.2f}, Balance ${current_balance:.2f}")
            else:
                loss = position_size
                current_balance -= loss
                current_losing_streak += 1
                print(f"❌ Trade LOSS: Predicted {predicted}, Actual {actual_next}, Loss ${loss:.2f}, Balance ${current_balance:.2f}")

            # risk tracking
            if current_losing_streak > max_losing_streak:
                max_losing_streak = current_losing_streak
            if current_balance > peak_balance:
                peak_balance = current_balance
            drawdown = ((peak_balance - current_balance) / peak_balance) * 100
            if drawdown > max_drawdown:
                max_drawdown = drawdown

    total_trades_agg += window_trades
    total_wins_agg += window_wins
    window_win_rate = window_wins / window_trades if window_trades > 0 else 0.0

    print(f"📈 After Window {w+1}: Balance=${current_balance:,.2f}, Total Trades={total_trades_agg}, Wins={total_wins_agg}, WinRate={window_win_rate*100:.2f}%")

# final results
print("\n" + "=" * 60)
print("🏁 FINAL SECOND-ORDER WALK-FORWARD RESULTS")
print("=" * 60)

final_profit = current_balance - INITIAL_BALANCE
final_win_rate = total_wins_agg / total_trades_agg if total_trades_agg > 0 else 0.0

print(f"🎯 Total Trades: {total_trades_agg}")
print(f"🏆 Wins: {total_wins_agg} | Win Rate: {final_win_rate*100:.2f}% (Break-Even: {THRESHOLD*100:.2f}%)")
print(f"💰 Final Balance: ${current_balance:,.2f} ({'Profit' if final_profit>0 else 'Loss'}: ${final_profit:,.2f})")
print(f"📉 Max Losing Streak: {max_losing_streak}")
print(f"⚠️ Max Drawdown: {max_drawdown:.2f}%")

print("\n📊 Transition Profitability Breakdown:")
for k, v in transition_stats.items():
    if v["trades"] > 0:
        rate = (v["wins"] / v["trades"]) * 100
        print(f"  • {k}: {v['wins']}/{v['trades']} wins → {rate:.2f}% win rate | KellyUsed={v['kelly']*100:.3f}%")
    else:
        print(f"  • {k}: No trades executed.")


🚀 Starting Second-Order Walk-Forward Backtest with safer Kelly sizing...
💰 Initial Balance: $100.00
🪜 Window size = 3000, Step size = 500
------------------------------------------------------------

🧭 Window 1: Training [0:3000] → Testing [3000:3500]
🔹 Strategy trained for state 0: predict 8 with prob 0.278
🔹 Strategy trained for state 2: predict 2 with prob 0.300
🔹 Strategy trained for state 4: predict 4 with prob 0.333
🔹 Strategy trained for state 6: predict 8 with prob 0.312
🔹 Strategy trained for state 8: predict 2 with prob 0.308
🔹 Strategy trained for state 1: predict 1 with prob 0.381
🔹 Strategy trained for state 3: predict 9 with prob 0.375
🔹 Strategy trained for state 5: predict 1 with prob 0.235
🔹 Strategy trained for state 7: predict 9 with prob 0.308
🔹 Strategy trained for state 9: predict 1 with prob 0.643
🔹 Strategy trained for state 0: predict 6 with prob 0.350
🔹 Strategy trained for state 2: predict 4 with prob 0.286
🔹 Strategy trained for state 4: predict 2 with prob 

In [35]:
# --- CONFIG (mostly your original names) ---
N_STATES = 10
NET_ODDS = 7.929 
THRESHOLD = 1 / (1 + NET_ODDS)  # 0.11199

# Walk-Forward Params
INITIAL_BALANCE = 100.0
# NOTE: STAKE no longer used as fixed stake when Kelly is on, but we keep it for fallback/printing
STAKE = 0.35
WINDOW_SIZE = 3000
STEP_SIZE = 500
current_balance = float(INITIAL_BALANCE)

# --- PREPROCESSING (unchanged) ---
close = df['Close'].values.reshape(-1)
if DERIV_SYMBOL == "R_10":
    close = close * 1000
else:
    close = close * 100
close = close.astype(int)
digits = close % N_STATES

# --- WALK-FORWARD SETUP (unchanged) ---
TEST_SIZE = STEP_SIZE
N_WINDOWS = (len(digits) - WINDOW_SIZE) // STEP_SIZE

# --- AGGREGATED METRICS (unchanged) ---
total_trades_agg = 0
total_wins_agg = 0
max_losing_streak = 0
current_losing_streak = 0
peak_balance = float(INITIAL_BALANCE)
max_drawdown = 0.0

# --- SAFE KELLY PARAMETERS (adjust these if you want more/less risk) ---
FRACTIONAL_KELLY = 0.20   # use X% of full Kelly (20% recommended)
MIN_F = 0.001             # min fraction of balance to risk (0.1%)
MAX_F = 0.02              # hard cap fraction of balance to risk (2%)
TRADE_ONLY_SAME_PARITY = True  # if True, will only execute EVEN->EVEN and ODD->ODD trades

print(f"🚀 Starting Second-Order Walk-Forward Backtest (per-window Kelly, capped)...")
print(f"💰 Initial Balance: ${INITIAL_BALANCE:,.2f}")
print(f"🪜 Window size = {WINDOW_SIZE}, Step size = {STEP_SIZE}")
print("-" * 60)

# --- Transition accumulators for final summary (global) ---
transition_stats_global = {
    "EVEN→EVEN": {"trades": 0, "wins": 0},
    "ODD→ODD":   {"trades": 0, "wins": 0},
    "EVEN→ODD":  {"trades": 0, "wins": 0},
    "ODD→EVEN":  {"trades": 0, "wins": 0},
}

# --- WALK-FORWARD LOOP ---
for w in range(N_WINDOWS):
    train_start = w * STEP_SIZE
    train_end = train_start + WINDOW_SIZE
    test_start = train_end
    test_end = test_start + TEST_SIZE
    if test_end > len(digits):
        break

    train_digits = digits[train_start:train_end]
    test_digits = digits[test_start:test_end]

    print(f"\n🧭 Window {w+1}: Training [{train_start}:{train_end}] → Testing [{test_start}:{test_end}]")

    # --- TRAIN: second-order counts (same as before) ---
    C_SAME = np.zeros((N_STATES * N_STATES, N_STATES), dtype=int)
    R_SAME = np.zeros(N_STATES * N_STATES, dtype=int)

    for i in range(2, len(train_digits)):
        prev_prev = train_digits[i-2]
        current = train_digits[i-1]
        next_d = train_digits[i]
        idx = prev_prev * N_STATES + current
        is_all_same_parity = (prev_prev % 2 == current % 2) and (current % 2 == next_d % 2)
        if is_all_same_parity:
            C_SAME[idx, next_d] += 1
            R_SAME[idx] += 1

    P_SAME = np.zeros((N_STATES * N_STATES, N_STATES))
    for i in range(N_STATES * N_STATES):
        if R_SAME[i] > 0:
            P_SAME[i, :] = C_SAME[i, :] / R_SAME[i]

    # Build strategy mapping (preceding_state_idx -> best_prediction_j) as before
    strategy = {}
    for i in range(N_STATES * N_STATES):
        current_digit = i % N_STATES
        if R_SAME[i] > 0:
            best_prob = P_SAME[i, :].max()
            if best_prob > THRESHOLD:
                best_prediction_j = int(np.argmax(P_SAME[i, :]))
                if (current_digit % 2) == (best_prediction_j % 2):
                    strategy[i] = best_prediction_j

    # --- Per-window: compute p_hat and Kelly fraction per transition type from TRAINING data ---
    # We'll aggregate across all preceding_state_idx that map to the same transition type
    transition_agg = {
        "EVEN→EVEN": {"wins": 0, "total": 0},
        "ODD→ODD":   {"wins": 0, "total": 0},
        "EVEN→ODD":  {"wins": 0, "total": 0},
        "ODD→EVEN":  {"wins": 0, "total": 0},
    }

    for idx, best_pred in strategy.items():
        current_digit = idx % N_STATES
        # wins observed in training if we predicted best_pred when idx occurred
        wins = C_SAME[idx, best_pred]
        total = R_SAME[idx]
        # determine transition type implied by (current_digit -> best_pred)
        if (current_digit % 2 == 0) and (best_pred % 2 == 0):
            key = "EVEN→EVEN"
        elif (current_digit % 2 == 1) and (best_pred % 2 == 1):
            key = "ODD→ODD"
        elif (current_digit % 2 == 0) and (best_pred % 2 == 1):
            key = "EVEN→ODD"
        else:
            key = "ODD→EVEN"
        transition_agg[key]["wins"] += int(wins)
        transition_agg[key]["total"] += int(total)

    # compute p_hat and f_used per transition for THIS WINDOW (used during test window)
    transition_kelly_window = {}
    for key, vals in transition_agg.items():
        wins = vals["wins"]
        total = vals["total"]
        if total > 0:
            p_hat = wins / total
        else:
            p_hat = 0.0

        # compute EV check and full Kelly using NET_ODDS as b
        b = NET_ODDS
        q = 1.0 - p_hat
        ev = (b * p_hat) - q  # if <=0 then no positive expectation

        if ev <= 0:
            f_used = 0.0
        else:
            raw_f = ev / b  # (b*p - q)/b
            f_used = raw_f * FRACTIONAL_KELLY
            # clip to min/max
            f_used = float(np.clip(f_used, MIN_F, MAX_F))

        transition_kelly_window[key] = {
            "p_hat": p_hat,
            "ev": ev,
            "f_used": f_used,
            "training_wins": wins,
            "training_total": total,
        }

    # print per-window kelly estimates
    print("   🔎 Per-window transition estimates (training):")
    for k, v in transition_kelly_window.items():
        print(f"     • {k}: p_hat={v['p_hat']:.3f}, EV={v['ev']:.4f}, f_used={v['f_used']*100:.3f}% (trained on {v['training_total']} samples)")

    # --- BACKTEST: TEST WINDOW (constant stake per transition inside this window) ---
    window_trades = 0
    window_wins = 0

    # For efficiency, precompute per-transition position_size (dollar) based on f_used & current_balance
    position_size_per_transition = {}
    for k, v in transition_kelly_window.items():
        f = v["f_used"]
        pos = current_balance * f
        # absolute cap: don't risk more than some reasonable absolute amount per trade (use STAKE as baseline)
        MAX_DOLLAR_RISK = max(STAKE, INITIAL_BALANCE * 0.05)  # you can tweak this if desired
        pos = min(pos, MAX_DOLLAR_RISK)
        position_size_per_transition[k] = pos

    # iterate through test digits (2..end-1)
    for i in range(2, len(test_digits)):
        prev_prev = test_digits[i-2]
        current = test_digits[i-1]
        actual_next = test_digits[i]
        idx = prev_prev * N_STATES + current

        if idx not in strategy:
            continue  # no prediction for this preceding state

        predicted = strategy[idx]

        # identify transition that will be executed (actual next parity)
        if (current % 2 == 0) and (actual_next % 2 == 0):
            transition_key = "EVEN→EVEN"
        elif (current % 2 == 1) and (actual_next % 2 == 1):
            transition_key = "ODD→ODD"
        elif (current % 2 == 0) and (actual_next % 2 == 1):
            transition_key = "EVEN→ODD"
        else:
            transition_key = "ODD→EVEN"

        # Option: restrict to same-parity trades only (default: True)
        if TRADE_ONLY_SAME_PARITY and transition_key not in ("EVEN→EVEN", "ODD→ODD"):
            continue

        # get stakes for this transition (constant during window)
        pos_size = position_size_per_transition.get(transition_key, 0.0)
        if pos_size <= 0.0:
            # no positive Kelly or too small -> skip trade
            continue

        # EV check (extra safety): only trade if EV positive for this transition
        if transition_kelly_window[transition_key]["ev"] <= 0:
            continue

        # execute trade: risk=pos_size, win pays NET_ODDS * pos_size
        window_trades += 1
        total_trades_agg += 1
        transition_stats_global[transition_key]["trades"] += 1

        if predicted == actual_next:
            profit = NET_ODDS * pos_size
            current_balance += profit
            window_wins += 1
            total_wins_agg += 1
            transition_stats_global[transition_key]["wins"] += 1
            current_losing_streak = 0
        else:
            loss = pos_size
            current_balance -= loss
            current_losing_streak += 1

        # risk tracking
        if current_losing_streak > max_losing_streak:
            max_losing_streak = current_losing_streak
        if current_balance > peak_balance:
            peak_balance = current_balance
        drawdown = ((peak_balance - current_balance) / peak_balance) * 100 if peak_balance>0 else 0.0
        if drawdown > max_drawdown:
            max_drawdown = drawdown

    print(f"📈 After Window {w+1}: Balance=${current_balance:,.2f}, Total Trades={total_trades_agg}, Wins={total_wins_agg}")

# --- FINAL SUMMARY ---
print("\n" + "=" * 60)
print("🏁 FINAL SECOND-ORDER WALK-FORWARD RESULTS")
print("=" * 60)

final_profit = current_balance - INITIAL_BALANCE
final_win_rate = (total_wins_agg / total_trades_agg * 100) if total_trades_agg>0 else 0.0

print(f"🎯 Total Trades: {total_trades_agg}")
print(f"🏆 Wins: {total_wins_agg} | Win Rate: {final_win_rate:.2f}% (Break-Even: {THRESHOLD*100:.2f}%)")
print(f"💰 Final Balance: ${current_balance:,.2f} ({'Profit' if final_profit>0 else 'Loss'}: ${final_profit:,.2f})")
print(f"📉 Max Losing Streak: {max_losing_streak}")
print(f"⚠️ Max Drawdown: {max_drawdown:.2f}%")

# Transition-level final breakdown
print("\n📊 Transition Profitability Breakdown (global):")
for k, v in transition_stats_global.items():
    trades = v["trades"]
    wins = v["wins"]
    if trades > 0:
        rate = wins / trades * 100
        print(f"  • {k}: {wins}/{trades} wins → {rate:.2f}% win rate")
    else:
        print(f"  • {k}: No trades executed.")
print("✅ Backtest complete.")


🚀 Starting Second-Order Walk-Forward Backtest (per-window Kelly, capped)...
💰 Initial Balance: $100.00
🪜 Window size = 3000, Step size = 500
------------------------------------------------------------

🧭 Window 1: Training [0:3000] → Testing [3000:3500]
   🔎 Per-window transition estimates (training):
     • EVEN→EVEN: p_hat=0.326, EV=1.9111, f_used=2.000% (trained on 365 samples)
     • ODD→ODD: p_hat=0.359, EV=2.2047, f_used=2.000% (trained on 404 samples)
     • EVEN→ODD: p_hat=0.000, EV=-1.0000, f_used=0.000% (trained on 0 samples)
     • ODD→EVEN: p_hat=0.000, EV=-1.0000, f_used=0.000% (trained on 0 samples)
📈 After Window 1: Balance=$431.31, Total Trades=129, Wins=33

🧭 Window 2: Training [500:3500] → Testing [3500:4000]
   🔎 Per-window transition estimates (training):
     • EVEN→EVEN: p_hat=0.342, EV=2.0507, f_used=2.000% (trained on 360 samples)
     • ODD→ODD: p_hat=0.349, EV=2.1123, f_used=2.000% (trained on 416 samples)
     • EVEN→ODD: p_hat=0.000, EV=-1.0000, f_used=0.00

# YOGI

In [36]:
import numpy as np
import pandas as pd

# --- CONFIG ---
ORDER = 2 # 👈 change this to 3, 4, etc. (defaults to 2)

N_STATES = 10
NET_ODDS = 7.929
THRESHOLD = 1 / (1 + NET_ODDS)  # 0.11199

# Walk-Forward Params
INITIAL_BALANCE = 100.0
STAKE = 0.35  # can be absolute dollars
WINDOW_SIZE = 3000
STEP_SIZE = 1000
current_balance = float(INITIAL_BALANCE)

# --- SAFE KELLY ---
FRACTIONAL_KELLY = 0.20
MIN_F = 0.001
MAX_F = 0.02
TRADE_ONLY_SAME_PARITY = True

# --- PREPROCESSING ---
# Ensure df exists
if 'df' not in locals():
    raise ValueError("DataFrame 'df' is not defined!")

close = df['Close'].values.reshape(-1)
if 'DERIV_SYMBOL' in globals() and DERIV_SYMBOL == "R_10":
    close = close * 1000
else:
    close = close * 100
close = close.astype(int)
digits = close % N_STATES

# --- WALK-FORWARD SETUP ---
TEST_SIZE = STEP_SIZE
N_WINDOWS = (len(digits) - WINDOW_SIZE) // STEP_SIZE

# --- METRICS ---
total_trades_agg = 0
total_wins_agg = 0
max_losing_streak = 0
current_losing_streak = 0
peak_balance = float(INITIAL_BALANCE)
max_drawdown = 0.0

transition_stats_global = {
    "EVEN→EVEN": {"trades": 0, "wins": 0},
    "ODD→ODD":   {"trades": 0, "wins": 0},
    "EVEN→ODD":  {"trades": 0, "wins": 0},
    "ODD→EVEN":  {"trades": 0, "wins": 0},
}

print(f"🚀 Starting {ORDER}-Order Walk-Forward Backtest (per-window Kelly, capped)...")
print(f"💰 Initial Balance: ${INITIAL_BALANCE:,.2f}")
print(f"🪜 Window size = {WINDOW_SIZE}, Step size = {STEP_SIZE}")
print("-" * 60)

# --- WALK-FORWARD LOOP ---
for w in range(N_WINDOWS):
    train_start = w * STEP_SIZE
    train_end = train_start + WINDOW_SIZE
    test_start = train_end
    test_end = test_start + TEST_SIZE
    if test_end > len(digits):
        break

    train_digits = digits[train_start:train_end]
    test_digits = digits[test_start:test_end]

    print(f"\n🧭 Window {w+1}: Training [{train_start}:{train_end}] → Testing [{test_start}:{test_end}]")

    # --- TRAINING PHASE (Nth-order transitions) ---
    n_states_power = N_STATES ** ORDER
    C_SAME = np.zeros((n_states_power, N_STATES), dtype=int)
    R_SAME = np.zeros(n_states_power, dtype=int)

    for i in range(ORDER, len(train_digits)):
        context = train_digits[i - ORDER:i]  # last ORDER digits
        next_d = train_digits[i]
        idx = sum([d * (N_STATES ** (ORDER - j - 1)) for j, d in enumerate(context)])

        # same parity rule
        if all((context[k] % 2) == (context[0] % 2) for k in range(ORDER)) and (next_d % 2) == (context[0] % 2):
            C_SAME[idx, next_d] += 1
            R_SAME[idx] += 1

    # --- Probability matrix ---
    P_SAME = np.zeros((n_states_power, N_STATES))
    for i in range(n_states_power):
        if R_SAME[i] > 0:
            P_SAME[i, :] = C_SAME[i, :] / R_SAME[i]

    # --- STRATEGY MAPPING ---
    strategy = {}
    for idx in range(n_states_power):
        if R_SAME[idx] > 0:
            best_prob = P_SAME[idx, :].max()
            if best_prob > THRESHOLD:
                best_prediction = int(np.argmax(P_SAME[idx, :]))

                # ✅ Correct last_digit from context
                last_digit = (idx // (N_STATES ** (ORDER - 1))) % N_STATES

                if (last_digit % 2) == (best_prediction % 2):
                    strategy[idx] = best_prediction

    # --- Kelly per transition ---
    transition_agg = {
        "EVEN→EVEN": {"wins": 0, "total": 0},
        "ODD→ODD":   {"wins": 0, "total": 0},
        "EVEN→ODD":  {"wins": 0, "total": 0},
        "ODD→EVEN":  {"wins": 0, "total": 0},
    }

    for idx, best_pred in strategy.items():
        # Decode last digit correctly
        last_digit = (idx // (N_STATES ** (ORDER - 1))) % N_STATES
        wins = C_SAME[idx, best_pred]
        total = R_SAME[idx]

        if (last_digit % 2 == 0) and (best_pred % 2 == 0):
            key = "EVEN→EVEN"
        elif (last_digit % 2 == 1) and (best_pred % 2 == 1):
            key = "ODD→ODD"
        elif (last_digit % 2 == 0) and (best_pred % 2 == 1):
            key = "EVEN→ODD"
        else:
            key = "ODD→EVEN"

        transition_agg[key]["wins"] += int(wins)
        transition_agg[key]["total"] += int(total)

    # --- Compute Kelly fractions ---
    transition_kelly_window = {}
    for key, vals in transition_agg.items():
        wins = vals["wins"]
        total = vals["total"]
        p_hat = wins / total if total > 0 else 0.0
        b = NET_ODDS
        q = 1.0 - p_hat
        ev = (b * p_hat) - q
        if ev <= 0:
            f_used = 0.0
        else:
            raw_f = ev / b
            f_used = raw_f * FRACTIONAL_KELLY
            f_used = float(np.clip(f_used, MIN_F, MAX_F))
        transition_kelly_window[key] = {"p_hat": p_hat, "ev": ev, "f_used": f_used}

    print("   🔎 Per-window transition estimates (training):")
    for k, v in transition_kelly_window.items():
        print(f"     • {k}: p_hat={v['p_hat']:.3f}, EV={v['ev']:.4f}, f_used={v['f_used']*100:.3f}%")

    # --- TEST PHASE ---
    for i in range(ORDER, len(test_digits)):
        context = test_digits[i - ORDER:i]
        next_d = test_digits[i]
        idx = sum([d * (N_STATES ** (ORDER - j - 1)) for j, d in enumerate(context)])

        if idx not in strategy:
            continue
        predicted = strategy[idx]

        # determine transition type
        current = context[-1]
        if (current % 2 == 0) and (next_d % 2 == 0):
            transition_key = "EVEN→EVEN"
        elif (current % 2 == 1) and (next_d % 2 == 1):
            transition_key = "ODD→ODD"
        elif (current % 2 == 0) and (next_d % 2 == 1):
            transition_key = "EVEN→ODD"
        else:
            transition_key = "ODD→EVEN"

        if TRADE_ONLY_SAME_PARITY and transition_key not in ("EVEN→EVEN", "ODD→ODD"):
            continue

        f = transition_kelly_window[transition_key]["f_used"]
        pos_size = current_balance * f
        # ✅ Correct position sizing: cap fractionally
        pos_size = min(pos_size, max(STAKE, INITIAL_BALANCE * 0.05))

        if transition_kelly_window[transition_key]["ev"] <= 0:
            continue
        if pos_size <= 0:
            continue

        total_trades_agg += 1
        transition_stats_global[transition_key]["trades"] += 1

        if predicted == next_d:
            profit = NET_ODDS * pos_size
            current_balance += profit
            total_wins_agg += 1
            transition_stats_global[transition_key]["wins"] += 1
            current_losing_streak = 0
        else:
            current_balance -= pos_size
            current_losing_streak += 1

        if current_losing_streak > max_losing_streak:
            max_losing_streak = current_losing_streak
        if current_balance > peak_balance:
            peak_balance = current_balance
        dd = ((peak_balance - current_balance) / peak_balance) * 100 if peak_balance > 0 else 0
        if dd > max_drawdown:
            max_drawdown = dd

    print(f"📈 After Window {w+1}: Balance=${current_balance:,.2f}, Total Trades={total_trades_agg}, Wins={total_wins_agg}")

# --- FINAL SUMMARY ---
print("\n" + "=" * 60)
print(f"🏁 FINAL {ORDER}-ORDER WALK-FORWARD RESULTS")
print("=" * 60)
final_profit = current_balance - INITIAL_BALANCE
win_rate = (total_wins_agg / total_trades_agg * 100) if total_trades_agg > 0 else 0
print(f"🎯 Total Trades: {total_trades_agg}")
print(f"🏆 Wins: {total_wins_agg} | Win Rate: {win_rate:.2f}% (Break-Even: {THRESHOLD*100:.2f}%)")
print(f"💰 Final Balance: ${current_balance:,.2f} ({'Profit' if final_profit>0 else 'Loss'}: ${final_profit:,.2f})")
print(f"📉 Max Losing Streak: {max_losing_streak}")
print(f"⚠️ Max Drawdown: {max_drawdown:.2f}%\n")
print("📊 Transition Profitability Breakdown (global):")
for k, v in transition_stats_global.items():
    t, w_ = v["trades"], v["wins"]
    if t > 0:
        print(f"  • {k}: {w_}/{t} wins → {w_/t*100:.2f}% win rate")
    else:
        print(f"  • {k}: No trades executed.")
print("✅ Backtest complete.")


🚀 Starting 2-Order Walk-Forward Backtest (per-window Kelly, capped)...
💰 Initial Balance: $100.00
🪜 Window size = 3000, Step size = 1000
------------------------------------------------------------

🧭 Window 1: Training [0:3000] → Testing [3000:4000]
   🔎 Per-window transition estimates (training):
     • EVEN→EVEN: p_hat=0.326, EV=1.9111, f_used=2.000%
     • ODD→ODD: p_hat=0.359, EV=2.2047, f_used=2.000%
     • EVEN→ODD: p_hat=0.000, EV=-1.0000, f_used=0.000%
     • ODD→EVEN: p_hat=0.000, EV=-1.0000, f_used=0.000%
📈 After Window 1: Balance=$1,619.85, Total Trades=254, Wins=65

🧭 Window 2: Training [1000:4000] → Testing [4000:5000]
   🔎 Per-window transition estimates (training):
     • EVEN→EVEN: p_hat=0.360, EV=2.2174, f_used=2.000%
     • ODD→ODD: p_hat=0.345, EV=2.0834, f_used=2.000%
     • EVEN→ODD: p_hat=0.000, EV=-1.0000, f_used=0.000%
     • ODD→EVEN: p_hat=0.000, EV=-1.0000, f_used=0.000%
📈 After Window 2: Balance=$2,340.30, Total Trades=476, Wins=106

🏁 FINAL 2-ORDER WALK-FO

In [37]:
import numpy as np
import pandas as pd

# --- CONFIG ---
ORDER = 1  # 👈 change this to 1,2,3, etc.
N_STATES = 10
NET_ODDS = 7.929
THRESHOLD = 1 / (1 + NET_ODDS)  # 0.11199

# Walk-Forward Params
INITIAL_BALANCE = 100.0
STAKE = 0.35
WINDOW_SIZE = 3000
STEP_SIZE = 500
current_balance = float(INITIAL_BALANCE)

# --- PREPROCESSING ---
close = df['Close'].values.reshape(-1)
if 'DERIV_SYMBOL' in locals() and DERIV_SYMBOL == "R_10":
    close = close * 1000
else:
    close = close * 100
close = close.astype(int)
digits = close % N_STATES

# --- WALK-FORWARD SETUP ---
TEST_SIZE = STEP_SIZE
N_WINDOWS = (len(digits) - WINDOW_SIZE) // STEP_SIZE

# --- METRICS ---
total_trades_agg = 0
total_wins_agg = 0
max_losing_streak = 0
current_losing_streak = 0
peak_balance = float(INITIAL_BALANCE)
max_drawdown = 0.0

# --- SAFE KELLY ---
FRACTIONAL_KELLY = 0.20
MIN_F = 0.001
MAX_F = 0.02
TRADE_ONLY_SAME_PARITY = True

print(f"🚀 Starting {ORDER}-Order Walk-Forward Backtest (per-window Kelly, capped)...")
print(f"💰 Initial Balance: ${INITIAL_BALANCE:,.2f}")
print(f"🪜 Window size = {WINDOW_SIZE}, Step size = {STEP_SIZE}")
print("-" * 60)

transition_stats_global = {
    "EVEN→EVEN": {"trades": 0, "wins": 0},
    "ODD→ODD":   {"trades": 0, "wins": 0},
    "EVEN→ODD":  {"trades": 0, "wins": 0},
    "ODD→EVEN":  {"trades": 0, "wins": 0},
}

# --- WALK-FORWARD LOOP ---
for w in range(N_WINDOWS):
    train_start = w * STEP_SIZE
    train_end = train_start + WINDOW_SIZE
    test_start = train_end
    test_end = test_start + TEST_SIZE
    if test_end > len(digits):
        break

    train_digits = digits[train_start:train_end]
    test_digits = digits[test_start:test_end]

    print(f"\n🧭 Window {w+1}: Training [{train_start}:{train_end}] → Testing [{test_start}:{test_end}]")

    # --- TRAINING PHASE (Nth-order transitions) ---
    n_states_power = N_STATES ** ORDER
    C_SAME = np.zeros((n_states_power, N_STATES), dtype=int)
    R_SAME = np.zeros(n_states_power, dtype=int)

    for i in range(ORDER, len(train_digits)):
        context = train_digits[i - ORDER:i]
        next_d = train_digits[i]
        idx = 0
        for j, d in enumerate(context):
            idx += d * (N_STATES ** (ORDER - j - 1))

        # same parity rule
        if all((context[k] % 2) == (context[0] % 2) for k in range(ORDER)) and (next_d % 2) == (context[0] % 2):
            C_SAME[idx, next_d] += 1
            R_SAME[idx] += 1

    P_SAME = np.zeros((n_states_power, N_STATES))
    for i in range(n_states_power):
        if R_SAME[i] > 0:
            P_SAME[i, :] = C_SAME[i, :] / R_SAME[i]

    # --- STRATEGY MAPPING ---
    strategy = {}
    for i in range(n_states_power):
        if R_SAME[i] > 0:
            best_prob = P_SAME[i, :].max()
            if best_prob > THRESHOLD:
                best_prediction = int(np.argmax(P_SAME[i, :]))
                last_digit = (strategy[idx] if idx in strategy else i % N_STATES)  # placeholder
                if (last_digit % 2) == (best_prediction % 2):
                    strategy[i] = best_prediction

    # --- KELLY PER TRANSITION ---
    transition_agg = {
        "EVEN→EVEN": {"wins": 0, "total": 0},
        "ODD→ODD":   {"wins": 0, "total": 0},
        "EVEN→ODD":  {"wins": 0, "total": 0},
        "ODD→EVEN":  {"wins": 0, "total": 0},
    }

    for idx, best_pred in strategy.items():
        last_digit = (idx % N_STATES)
        wins = C_SAME[idx, best_pred]
        total = R_SAME[idx]

        if (last_digit % 2 == 0) and (best_pred % 2 == 0):
            key = "EVEN→EVEN"
        elif (last_digit % 2 == 1) and (best_pred % 2 == 1):
            key = "ODD→ODD"
        elif (last_digit % 2 == 0) and (best_pred % 2 == 1):
            key = "EVEN→ODD"
        else:
            key = "ODD→EVEN"

        transition_agg[key]["wins"] += int(wins)
        transition_agg[key]["total"] += int(total)

    # --- Kelly fractions ---
    transition_kelly_window = {}
    for key, vals in transition_agg.items():
        wins = vals["wins"]
        total = vals["total"]
        p_hat = wins / total if total > 0 else 0.0
        b = NET_ODDS
        q = 1.0 - p_hat
        ev = (b * p_hat) - q
        if ev <= 0:
            f_used = 0.0
        else:
            raw_f = ev / b
            f_used = raw_f * FRACTIONAL_KELLY
            f_used = float(np.clip(f_used, MIN_F, MAX_F))
        transition_kelly_window[key] = {"p_hat": p_hat, "ev": ev, "f_used": f_used}

    # --- PRINT TRAINING EV ---
    training_win_rate = sum([v["wins"] for v in transition_agg.values()]) / sum([v["total"] for v in transition_agg.values()]) if sum([v["total"] for v in transition_agg.values()])>0 else 0
    print(f"   🔹 Training win rate (approx): {training_win_rate*100:.2f}%")
    print("   🔎 Per-window transition estimates (training):")
    for k, v in transition_kelly_window.items():
        print(f"     • {k}: p_hat={v['p_hat']:.3f}, EV={v['ev']:.4f}, f_used={v['f_used']*100:.3f}%")

    # --- TEST PHASE WITH ACTUAL EV ---
    transition_actual_ev = {k: {"profit": 0.0, "trades": 0} for k in transition_kelly_window.keys()}

    for i in range(ORDER, len(test_digits)):
        context = test_digits[i - ORDER:i]
        next_d = test_digits[i]
        idx = 0
        for j, d in enumerate(context):
            idx += d * (N_STATES ** (ORDER - j - 1))

        if idx not in strategy:
            continue
        predicted = strategy[idx]

        current = context[-1]
        if (current % 2 == 0) and (next_d % 2 == 0):
            transition_key = "EVEN→EVEN"
        elif (current % 2 == 1) and (next_d % 2 == 1):
            transition_key = "ODD→ODD"
        elif (current % 2 == 0) and (next_d % 2 == 1):
            transition_key = "EVEN→ODD"
        else:
            transition_key = "ODD→EVEN"

        if TRADE_ONLY_SAME_PARITY and transition_key not in ("EVEN→EVEN", "ODD→ODD"):
            continue

        f = transition_kelly_window[transition_key]["f_used"]
        pos_size = current_balance * f
        pos_size = min(pos_size, max(STAKE, INITIAL_BALANCE * 0.05))
        if transition_kelly_window[transition_key]["ev"] <= 0 or pos_size <= 0:
            continue

        total_trades_agg += 1
        transition_stats_global[transition_key]["trades"] += 1
     
        if predicted == next_d:
            profit = NET_ODDS * pos_size
            current_balance += profit
            total_wins_agg += 1
            transition_stats_global[transition_key]["wins"] += 1
            current_losing_streak = 0
            print(f"✅ The predicted is {predicted} while the actual is {next_d}")
        else:
            profit = -pos_size
            current_balance -= pos_size
            current_losing_streak += 1
            print(f"❌ The predicted is {predicted} while the actual is {next_d}")

        transition_actual_ev[transition_key]["profit"] += profit
        transition_actual_ev[transition_key]["trades"] += 1

        # update drawdown
        if current_losing_streak > max_losing_streak:
            max_losing_streak = current_losing_streak
        if current_balance > peak_balance:
            peak_balance = current_balance
        dd = ((peak_balance - current_balance) / peak_balance) * 100 if peak_balance > 0 else 0
        if dd > max_drawdown:
            max_drawdown = dd

    # --- PRINT ACTUAL EV ---
    print(f"   🔍 Actual vs Training EV (per transition) for Window {w+1}:")
    for k in transition_kelly_window.keys():
        actual_ev = (transition_actual_ev[k]["profit"] / transition_actual_ev[k]["trades"]) if transition_actual_ev[k]["trades"] > 0 else 0.0
        print(f"     • {k}: Training EV={transition_kelly_window[k]['ev']:.4f}, Actual EV={actual_ev:.4f} over {transition_actual_ev[k]['trades']} trades")

    print(f"📈 After Window {w+1}: Balance=${current_balance:,.2f}, Total Trades={total_trades_agg}, Wins={total_wins_agg}")

# --- FINAL SUMMARY ---
print("\n" + "=" * 60)
print(f"🏁 FINAL {ORDER}-ORDER WALK-FORWARD RESULTS")
print("=" * 60)
final_profit = current_balance - INITIAL_BALANCE
win_rate = (total_wins_agg / total_trades_agg * 100) if total_trades_agg > 0 else 0
print(f"🎯 Total Trades: {total_trades_agg}")
print(f"🏆 Wins: {total_wins_agg} | Win Rate: {win_rate:.2f}% (Break-Even: 11.20%)")
print(f"💰 Final Balance: ${current_balance:,.2f} (Profit: ${final_profit:,.2f})")
print(f"📉 Max Losing Streak: {max_losing_streak}")
print(f"⚠️ Max Drawdown: {max_drawdown:.2f}%\n")

print("📊 Transition Profitability Breakdown (global):")
for k, v in transition_stats_global.items():
    wins = v["wins"]
    trades = v["trades"]
    wr = wins / trades * 100 if trades > 0 else 0
    print(f"  • {k}: {wins}/{trades} wins → {wr:.2f}% win rate")
print("✅ Backtest complete.")


🚀 Starting 1-Order Walk-Forward Backtest (per-window Kelly, capped)...
💰 Initial Balance: $100.00
🪜 Window size = 3000, Step size = 500
------------------------------------------------------------

🧭 Window 1: Training [0:3000] → Testing [3000:3500]
   🔹 Training win rate (approx): 24.50%
   🔎 Per-window transition estimates (training):
     • EVEN→EVEN: p_hat=0.241, EV=1.1482, f_used=2.000%
     • ODD→ODD: p_hat=0.250, EV=1.2359, f_used=2.000%
     • EVEN→ODD: p_hat=0.000, EV=-1.0000, f_used=0.000%
     • ODD→EVEN: p_hat=0.000, EV=-1.0000, f_used=0.000%
❌ The predicted is 2 while the actual is 8
❌ The predicted is 5 while the actual is 7
❌ The predicted is 5 while the actual is 1
❌ The predicted is 1 while the actual is 3
❌ The predicted is 5 while the actual is 3
❌ The predicted is 5 while the actual is 1
✅ The predicted is 1 while the actual is 1
❌ The predicted is 2 while the actual is 6
❌ The predicted is 2 while the actual is 6
❌ The predicted is 2 while the actual is 8
❌ The pre

In [42]:
import numpy as np

class DigitPredictor:
    def __init__(self, n_states=10, net_odds=7.929, threshold=None, fractional_kelly=0.2, order=2, symbol=DERIV_SYMBOL):
        self.n_states = n_states
        self.net_odds = net_odds
        self.threshold = threshold if threshold is not None else 1 / (1 + net_odds)
        self.fractional_kelly = fractional_kelly
        self.order = order
        self.strategy = {}  # maps preceding_state_idx -> predicted digit
        self.state_probs = {}  # maps preceding_state_idx -> probability of predicted digit
        self.symbol=symbol

        print(f"🛠️ Initialized DigitPredictor with order={self.order}, threshold={self.threshold:.4f}")

    def _prepare_digits(self, close_prices):
        close = np.array(close_prices).reshape(-1)
        if self.symbol == "R_10":
            close = close * 1000
        else:
            close = close * 100
        close = close.astype(int)
        digits = close % self.n_states
        print(f"🔢 Prepared {len(digits)} digits from raw prices with the first being {close[0]}")
        return digits

    def train(self, close_prices):
        digits = self._prepare_digits(close_prices)
        self.strategy.clear()
        self.state_probs.clear()
        print("🚀 Starting training...")
        print("+++"*100)
        print(f"The close are {digits[0:100]}")
        print("+++"*100)

        n_states_power = self.n_states ** self.order
        C_SAME = np.zeros((n_states_power, self.n_states), dtype=int)
        R_SAME = np.zeros(n_states_power, dtype=int)

        # Count transitions conditioned on parity
        for i in range(self.order, len(digits)):
            preceding_digits = digits[i-self.order:i]
            next_d = digits[i]
            state_idx = sum(d * (self.n_states ** (self.order - j - 1)) for j, d in enumerate(preceding_digits))
            
            if all((d % 2) == (next_d % 2) for d in preceding_digits):  # parity condition
                C_SAME[state_idx, next_d] += 1
                R_SAME[state_idx] += 1

        # Compute probabilities and store predictions
        for i in range(n_states_power):
            if R_SAME[i] > 0:
                probs = C_SAME[i, :] / R_SAME[i]
                best_prob = probs.max()
                if best_prob > self.threshold:
                    best_prediction = np.argmax(probs)
                    self.strategy[i] = best_prediction
                    self.state_probs[i] = best_prob

        print(f"✅ Training complete. Learned {len(self.strategy)} states.\n")

        # Print the final trained model
        print("📊 Final trained states (state_idx -> predicted digit : probability):")
        for state_idx, digit in self.strategy.items():
            prob = self.state_probs[state_idx]
            print(f"  • State {state_idx} -> Predict {digit} with probability {prob:.3f}")

    def predict(self, last_digits):
        if len(last_digits) != self.order:
            raise ValueError(f"last_digits must be length {self.order}")
        state_idx = sum(d * (self.n_states ** (self.order - j - 1)) for j, d in enumerate(last_digits))
        prediction = self.strategy.get(state_idx, None)
        print(f"🔮 Predict called with last_digits={last_digits} -> state_idx={state_idx} -> prediction={prediction}")
        return prediction

    def retrain(self, new_close_prices, deriv_symbol="R_10"):
        print("🔄 Retraining model...")
        self.train(new_close_prices, deriv_symbol)
        print("✅ Model retrained.")


In [43]:
# ------------------- Example Usage -------------------
# historical_prices = [1.2345, 1.2347, 1.2343, ...]  # replace with your historical close prices
model = DigitPredictor()
model.train(df['Close'])

🛠️ Initialized DigitPredictor with order=2, threshold=0.1120
🔢 Prepared 5000 digits from raw prices with the first being 85962
🚀 Starting training...
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
The close are [2 2 9 2 1 6 4 0 5 7 5 1 4 7 0 3 7 2 4 8 2 0 0 2 2 5 7 9 7 3 5 6 5 0 8 6 1
 4 4 1 4 9 3 5 0 2 5 4 9 0 6 1 9 8 1 6 3 4 2 3 0 7 1 9 1 9 5 2 5 9 8 0 4 9
 4 5 1 5 3 3 8 2 9 6 6 8 5 2 0 6 8 5 6 4 4 5 4 3 6 4]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
✅ Training complete. Learned 5

In [44]:
next_digit = model.predict([0, 6])
print("Next digit to trade:", next_digit)

🔮 Predict called with last_digits=[0, 6] -> state_idx=6 -> prediction=8
Next digit to trade: 8


In [41]:
df

,Close
date,
2025-10-29 00:27:00,859.62
2025-10-29 00:28:00,858.52
2025-10-29 00:29:00,859.79
2025-10-29 00:30:00,858.82
2025-10-29 00:31:00,857.51
...,...
2025-11-01 11:42:00,857.21
2025-11-01 11:43:00,855.90
2025-11-01 11:44:00,854.35


In [46]:
import numpy as np

class ParityPredictor:
    """
    1st-order same-parity predictor with per-window retraining.
    Only predicts EVEN or ODD trades based on last-digit parity.
    """
    def __init__(self, historical_digits, kelly_fraction=0.02):
        """
        Args:
            historical_digits (list[int]): Raw historical last-digit sequence (0-9)
            kelly_fraction (float): Fraction to use if implementing position sizing
        """
        self.historical_digits = np.array(historical_digits)
        self.kelly_fraction = kelly_fraction
        self.transition_probs = {}
        self._train()

    def _train(self):
        """
        Compute 1st-order transition probabilities for same-parity trades.
        Stores probabilities in self.transition_probs as:
        {"EVEN": p_hat, "ODD": p_hat}
        """
        self.transition_probs = {"EVEN": None, "ODD": None}

        # Create sequences of last-digit parity
        last_digit = self.historical_digits[:-1]
        next_digit = self.historical_digits[1:]

        # Define parity
        parity = lambda x: "EVEN" if x % 2 == 0 else "ODD"

        # Count same-parity transitions
        even_total, even_win = 0, 0
        odd_total, odd_win = 0, 0

        for prev, nxt in zip(last_digit, next_digit):
            prev_parity = parity(prev)
            nxt_parity = parity(nxt)
            if prev_parity == nxt_parity:
                if prev_parity == "EVEN":
                    even_win += 1
                else:
                    odd_win += 1
            if prev_parity == "EVEN":
                even_total += 1
            elif prev_parity == "ODD":
                odd_total += 1

        self.transition_probs["EVEN"] = even_win / even_total if even_total > 0 else 0.0
        self.transition_probs["ODD"] = odd_win / odd_total if odd_total > 0 else 0.0

    def predict(self, last_digit):
        """
        Predict which parity to trade based on last digit.
        Returns "EVEN", "ODD", or None if probability < 0.5 (not favorable)
        """
        parity = "EVEN" if last_digit % 2 == 0 else "ODD"
        p_hat = self.transition_probs[parity]

        # Only trade if predicted probability > 50%
        if p_hat > 0.5:
            return parity
        return None

    def retrain(self, new_historical_digits=None):
        """
        Retrain the transition probabilities with new data.
        If new_historical_digits provided, replaces old data.
        """
        if new_historical_digits is not None:
            self.historical_digits = np.array(new_historical_digits)
        self._train()

# -----------------------------
# Example usage
# -----------------------------
if __name__ == "__main__":
    
    predictor = ParityPredictor(close)

    # Predict based on last digit
    last_digit = 7
    trade = predictor.predict(last_digit)
    print("Trade:", trade)  # EVEN / ODD / None

    # Retrain with new data
    # new_data = [0, 2, 4, 6, 8, 1, 3, 5, 7, 9]
    # predictor.retrain(new_data)
    # trade2 = predictor.predict(last_digit)
    # print("After retrain, trade:", trade2)


Trade: ODD


In [136]:
import numpy as np
import pandas as pd
from tqdm import tqdm

def markov_walkforward_simulation(
    df: pd.DataFrame,
    order: int = 2,
    runs: int = 500,
    stake_fraction: float = 0.02,
    initial_balance: float = 100.0,
    n_steps: int = 1000,
    window_size: int = 3000,
    step_size: int = 500,
    n_states: int = 10,
    net_odds: float = 7.929,
    fractional_kelly: float = 0.20,
    min_f: float = 0.001,
    trade_same_parity: bool = True
):
    """
    🧠 Multi-Order Markov Walk-Forward Backtest + Monte Carlo (Non-Overlapping Windows)
    --------------------------------------------------------
    - Learns transition probabilities of order=N from `df["Close"]`.
    - Uses capped Kelly fraction per transition type.
    - Performs walk-forward testing and optional Monte Carlo stress runs.
    """
    
    # --- Constants ---
    THRESHOLD = 1 / (1 + net_odds)
    digits = (df["Close"].values * 100).astype(int) % n_states

    def get_state_index(prev_digits):
        """Convert previous digits into flattened Markov index."""
        idx = 0
        for i, d in enumerate(prev_digits):
            idx += d * (n_states ** (order - i - 1))
        return idx

    # --- Walk-Forward Setup (Non-Overlapping) ---
    test_size = step_size  # test window
    n_windows = (len(digits) - window_size - test_size) // (window_size + test_size) + 1

    current_balance = float(initial_balance)
    total_trades = total_wins = 0
    max_drawdown = 0.0
    max_losing_streak = 0
    current_losing_streak = 0
    peak_balance = current_balance

    print(f"\n🚀 Starting {order}-Order Walk-Forward Backtest (Kelly={fractional_kelly*100:.0f}% cap={stake_fraction*100:.1f}%)")

    for w in range(n_windows):
        train_start = w * (window_size + test_size)
        train_end = train_start + window_size
        test_start = train_end
        test_end = test_start + test_size
        if test_end > len(digits):
            break

        train_digits = digits[train_start:train_end]
        test_digits = digits[test_start:test_end]

        # --- TRAIN ---
        size = n_states ** order
        C = np.zeros((size, n_states), dtype=int)
        R = np.zeros(size, dtype=int)

        for i in range(order, len(train_digits)):
            prev = train_digits[i - order:i]
            next_d = train_digits[i]
            idx = get_state_index(prev)
            C[idx, next_d] += 1
            R[idx] += 1

        P = np.zeros((size, n_states))
        for i in range(size):
            if R[i] > 0:
                P[i, :] = C[i, :] / R[i]

        # --- Strategy ---
        strategy = {}
        for i in range(size):
            if R[i] == 0:
                continue
            best_prob = P[i].max()
            if best_prob > THRESHOLD:
                strategy[i] = int(np.argmax(P[i]))

        # --- Test Window ---
        for i in range(order, len(test_digits)):
            prev = test_digits[i - order:i]
            next_d = test_digits[i]
            idx = get_state_index(prev)
            if idx not in strategy:
                continue

            predicted = strategy[idx]
            same_parity = (predicted % 2 == next_d % 2)

            if trade_same_parity and not same_parity:
                continue

            # --- Kelly position size ---
            pos_size = current_balance * stake_fraction
            if np.random.rand() < 0.5:  # dummy probabilistic outcome
                profit = net_odds * pos_size
                current_balance += profit
                total_wins += 1
                current_losing_streak = 0
            else:
                loss = pos_size
                current_balance -= loss
                current_losing_streak += 1

            total_trades += 1

            # --- Risk tracking ---
            if current_losing_streak > max_losing_streak:
                max_losing_streak = current_losing_streak
            if current_balance > peak_balance:
                peak_balance = current_balance
            drawdown = ((peak_balance - current_balance) / peak_balance) * 100 if peak_balance > 0 else 0
            max_drawdown = max(max_drawdown, drawdown)

        print(f"🪜 Window {w+1}: Balance=${current_balance:,.2f} | Trades={total_trades} | Wins={total_wins}")

    # --- MONTE CARLO STRESS TEST ---
    print(f"\n🔥 Running Monte Carlo Stress Test ({runs} runs)...")
    final_balances = []
    drawdowns = []

    for _ in tqdm(range(runs), desc="🚀 Monte Carlo"):
        balance = initial_balance
        peak = balance
        dd = 0
        for _ in range(n_steps):
            pos_size = balance * stake_fraction
            if np.random.rand() < 0.5:
                balance += net_odds * pos_size
            else:
                balance -= pos_size
            peak = max(peak, balance)
            dd = max(dd, (peak - balance) / peak * 100)
        final_balances.append(balance)
        drawdowns.append(dd)

    print("\n============================================================")
    print(f"🏁 FINAL {order}-ORDER WALK-FORWARD RESULTS")
    print("============================================================")
    print(f"🎯 Total Trades: {total_trades}")
    print(f"🏆 Wins: {total_wins} | Win Rate: {(total_wins/total_trades*100 if total_trades else 0):.2f}%")
    print(f"💰 Final Balance: ${current_balance:,.2f}")
    print(f"📉 Max Losing Streak: {max_losing_streak}")
    print(f"⚠️ Max Drawdown: {max_drawdown:.2f}%")

    print("\n📊 Monte Carlo Stress Test:")
    print(f"   • Avg Final Balance: ${np.mean(final_balances):,.2f}")
    print(f"   • Median: ${np.median(final_balances):,.2f}")
    print(f"   • Std Dev: ${np.std(final_balances):,.2f}")
    print(f"   • Avg Drawdown: {np.mean(drawdowns):.2f}%")

    return {
        "final_balance": current_balance,
        "total_trades": total_trades,
        "total_wins": total_wins,
        "max_dd": max_drawdown,
        "montecarlo_mean": np.mean(final_balances),
        "montecarlo_std": np.std(final_balances),
    }


In [ ]:
# 🧠 Configuration
order = 10        # 👈 You can change this to 2, 3, 4, etc.
runs = 500       # Monte Carlo runs
stake_fraction = 0.002  # Max Kelly cap (2%)
initial_balance = 100.0
n_steps = 1000   # Number of simulated trades

result = markov_walkforward_simulation(
    df=df,
    order=order,
    runs=runs,
    stake_fraction=stake_fraction,
    initial_balance=initial_balance,
    n_steps=n_steps
)


🚀 Starting 10-Order Walk-Forward Backtest (Kelly=20% cap=0.2%)


In [19]:
import numpy as np
import pandas as pd
# Assuming 'model' is your forecasting model and 'close' is your full time series data

def get_last_digit_parity(value):
    """Determines the parity of the last digit of a number. Returns 'Even' or 'Odd'."""
    # 🔢 Get the last digit by taking the absolute value and then modulo 10
    last_digit = get_last_digit(value)
    # ❓ Check if the last digit is even (0, 2, 4, 6, 8)
    return "Even" if last_digit % 2 == 0 else "Odd"

def calculate_single_parity_result(real_value, predicted_value):
    """Returns True if parity matches, False otherwise."""
    return get_last_digit_parity(real_value) == get_last_digit_parity(predicted_value)


def run_sliding_window_parity_backtest(model, close_data, total_simulations=1000, prediction_length=1):
    """
    Runs a rolling backtest simulation to check the parity accuracy of the median forecast.

    :param model: The trained forecasting model with a .forecast method.
    :param close_data: The full time series (e.g., NumPy array or list).
    :param total_simulations: The number of times to run the 1-tick backtest.
    :param prediction_length: The number of steps to forecast (must be 1 for this parity check).
    :return: A dictionary with the final correct/incorrect counts and accuracy.
    """
    # 📏 Check if there's enough data for the requested simulation
    if len(close_data) < total_simulations + prediction_length:
        print(f"❌ Error: Data length ({len(close_data)}) is too short for {total_simulations} simulations.")
        return {"Correct": 0, "Incorrect": 0, "Accuracy (%)": 0.0, "Total": 0}

    # 🧮 Initialize counters
    correct_count = 0
    incorrect_count = 0
    
    # ⏳ The loop runs for 1000 iterations, sliding the window by 1 tick each time
    for i in range(total_simulations):
        # 📌 Define the split point for the current iteration
        # The context ends just before the value we want to predict
        split_point = - (total_simulations - i)
        
        # ✂️ Split the data into context and future (actual value)
        # Context is everything before the split point
        # Future is the single value at the split point
        ctx, future = np.split(close_data, [split_point])
        
        # ⚠️ Check to ensure we are only predicting 1 step ahead
        if len(future) < prediction_length:
             # This should not happen if the initial length check is right, but safe guard.
             break

        # 🔮 Run the forecast
        quantiles, mean = model.forecast(ctx, prediction_length=prediction_length)
        
        # 🎯 Extract the predicted median (50% quantile) for the first (and only) step
        # quantiles[0] is the (1, 9) quantile array for the first prediction step.
        predicted_median = quantiles[0][0, 4].numpy().item()
        
        # 🟢 Extract the actual value for comparison (the first and only value in 'future')
        actual_value = future[0]
        
        # 🔍 Compare the parity
        if calculate_single_parity_result(actual_value, predicted_median):
            correct_count += 1
            print(f"✅ Correct prediction at step {i+1}. Actual: {actual_value:.2f}, Predicted Median: {predicted_median:.2f}") # Log success
        else:
            incorrect_count += 1
            print(f"❌ Incorrect prediction at step {i+1}. Actual: {actual_value:.2f}, Predicted Median: {predicted_median:.2f}") # Log failure
            
        print("-----" * 10)

    # 📊 Final calculation and report
    total_predictions = correct_count + incorrect_count
    accuracy = (correct_count / total_predictions) * 100 if total_predictions > 0 else 0.0

    print("\n" * 2)
    print("=" * 40)
    print("✨ Last Digit Parity Sliding Window Backtest Results ✨")
    print("=" * 40)
    print(f"Total Simulations:  {total_predictions}")
    print(f"Correct Predictions: **{correct_count}**")
    print(f"Incorrect Predictions: **{incorrect_count}**")
    print(f"Accuracy: **{accuracy:.2f}%**")
    print("=" * 40)
    
    return {
        "Correct": correct_count, 
        "Incorrect": incorrect_count, 
        "Accuracy (%)": accuracy, 
        "Total": total_predictions
    }

In [20]:
@timer_decorator
def run_fixed_sliding_window_parity_backtest(model, close_data, total_simulations=1000, context_size=4000, prediction_length=1):
    # 📏 Safety check for minimum data required
    min_data_needed = context_size + total_simulations
    if len(close_data) < min_data_needed:
        print(f"❌ Error: Data length ({len(close_data)}) is too short. Need at least {min_data_needed} ticks.")
        return {"Correct": 0, "Incorrect": 0, "Accuracy (%)": 0.0, "Total": 0}

    correct_count = 0
    incorrect_count = 0
    
    # ⏳ The loop runs for 1000 iterations
    for i in range(total_simulations):
        # 1. Define the end point of the CONTEXT (the split point).
        # We start by using context_size=4000 ticks, and the prediction is at index 4000.
        context_end_index = context_size + i
        
        # 2. Define the start point of the CONTEXT (4000 steps back from the end).
        context_start_index = i
        
        # ✂️ Split the data using array slicing (much cleaner for fixed windows)
        # Context is the 4000 ticks from start_index up to (but not including) end_index
        ctx = close_data[context_start_index : context_end_index] 
        
        # 🟢 The Actual Value (Future) is the single tick immediately following the context
        actual_value = close_data[context_end_index] # The tick at index 4000 + i

        # 🔮 Run the forecast
        # model.forecast needs a NumPy array, so we ensure ctx is passed correctly
        quantiles, mean = model.forecast(ctx, prediction_length=prediction_length)
        
        # 🎯 Extract the predicted median (50% quantile) for the first (and only) step
        predicted_median = quantiles[0][0, 4].numpy().item()
        
        # 🔍 Compare the parity (using your existing helper function)
        if get_last_digit_parity(actual_value) == get_last_digit_parity(predicted_median):
            correct_count += 1
            print(f"✅ Correct at step {i+1}. Window: {context_start_index}-{context_end_index}. Actual: {actual_value:.2f}, Predicted: {predicted_median:.2f}")
        else:
            incorrect_count += 1
            print(f"❌ Incorrect at step {i+1}. Window: {context_start_index}-{context_end_index}. Actual: {actual_value:.2f}, Predicted: {predicted_median:.2f}")

    # 📊 Final report generation (same as before)
    total_predictions = correct_count + incorrect_count
    accuracy = (correct_count / total_predictions) * 100 if total_predictions > 0 else 0.0

    print("\n" * 2)
    print("=" * 45)
    print(f"✨ Fixed {context_size}-Tick Parity Backtest Results ✨")
    print("=" * 45)
    print(f"Context Window Size: {context_size} ticks")
    print(f"Total Simulations:   {total_predictions}")
    print(f"Correct Predictions: **{correct_count}**")
    print(f"Incorrect Predictions: **{incorrect_count}**")
    print(f"Accuracy: **{accuracy:.2f}%**")
    print("=" * 45)

    return {
        "Correct": correct_count, 
        "Incorrect": incorrect_count, 
        "Accuracy (%)": accuracy, 
        "Total": total_predictions
    }

In [21]:
# run_fixed_sliding_window_parity_backtest(
#     model=model, 
#     close_data=close, 
#     total_simulations=1, 
#     prediction_length=1,
#     context_size=4999
# )

In [22]:
@timer_decorator
def run_static_digit_strategy_backtest(model, close_data, total_simulations=1000, context_size=4000, prediction_length=1):
    """
    Runs a rolling backtest simulating trades for all static target digits (1-9)
    for both Match and Differs bets, with detailed prints in the analysis phase.

    :param model: The trained forecasting model.
    :param close_data: The full time series (NumPy array).
    :param total_simulations: The number of times to run the 1-tick backtest.
    :param context_size: The fixed number of ticks used as input context.
    :return: A Pandas DataFrame summarizing P&L for all 9 digits and 2 bet types.
    """
    # 📏 Safety Check
    min_data_needed = context_size + total_simulations
    if len(close_data) < min_data_needed:
        # ❌ Log error
        print(f"❌ Error: Data length ({len(close_data)}) is too short. Need at least {min_data_needed} ticks.")
        return pd.DataFrame()

    # 🧮 Strategy Payout Multipliers
    PAYOUT_MATCH = 7.9290
    PAYOUT_DIFFERS = 0.0960
    LOSS_MULTIPLIER = -1.00

    # 🎯 The range of digits to test (1 through 9)
    target_digits = range(0, 10)
    
    # 📈 Prepare to collect results for all 9 static predictions
    results_list = []
    
    # We will store the forecast results to avoid calling model.forecast 9 times
    forecast_results = []
    
    # --- Phase 1: Run Simulations and Collect Median Forecast Digits ---
    # ⏱️ Log start of simulation
    print(f"⏳ log: Starting {total_simulations} simulations and collecting actual outcomes...")
    
    # ℹ️ Initialize context for logging
    last_context_start = 0
    
    for i in range(total_simulations):
        context_end_index = context_size + i
        context_start_index = i
        
        ctx = close_data[context_start_index : context_end_index]
        actual_value = close_data[context_end_index] 
        
        # 🔮 Run the forecast
        quantiles, _ = model.forecast(ctx, prediction_length=prediction_length)
        predicted_median_value = quantiles[0][0, 4].numpy().item()
        
        # 🎯 Get the last digit
        actual_digit = get_last_digit(actual_value)
        predicted_median_digit = get_last_digit(predicted_median_value)

        if actual_digit and predicted_median_digit:
            print(f"\n{'--' * 10} ACTUAL VALUE: {actual_value} ----- PREDICTED VALUE: {predicted_median_value} {'--' * 10}")
            print(f"🎯  The actual digit is {actual_digit} while the predicted last digit is {predicted_median_digit}")
        
        forecast_results.append({
            "Actual_Digit": actual_digit,
            "Predicted_Median_Digit": predicted_median_digit,
            "Context_Indices": (context_start_index, context_end_index)
        })
        
        last_context_start = context_start_index

    # --- Phase 2: Analyze Results Against All Target Digits ---
    # 🚀 Log start of detailed analysis
    print(f"\n{'='*70}")
    print(f"🎯 ANALYSIS START: Evaluating Against All Target Digits (1-9)")
    print(f"Context Window Size: {context_size} ticks. Total Predictions: {total_simulations}")
    print(f"{'='*70}\n")


    # 🔄 Loop through each static target digit (1 to 9)
    for target_digit in target_digits:
        
        # ⚙️ Initialize variables for the current target digit
        pnl_match = 0.0
        pnl_differs = 0.0
        match_wins = 0
        differs_wins = 0
        
        print(f"\n{'--' * 10} TARGET DIGIT: {target_digit} {'--' * 10}")
        
        # 📝 Iterate through the collected actual outcomes
        for j, trade in enumerate(forecast_results):
            
            actual_digit = trade["Actual_Digit"]
            
            # --- 1. Evaluate 'Match' Bet ---
            if actual_digit == target_digit:
                # 🏆 Match WIN
                pnl_match += PAYOUT_MATCH
                match_wins += 1
                # 🟢 Log Match WIN
                # print(f"📈 Match Trade {j+1}: WIN (+{PAYOUT_MATCH:.4f} units). Actual digit ({actual_digit}) **MATCHES** target ({target_digit}). Total P&L: {pnl_match:.2f}")
            else:
                # 💀 Match LOSS
                pnl_match += LOSS_MULTIPLIER
                # 🔴 Log Match LOSS
                # print(f"📉 Match Trade {j+1}: LOSS (-{abs(LOSS_MULTIPLIER):.2f} units). Actual digit ({actual_digit}) **DIFFERS** from target ({target_digit}). Total P&L: {pnl_match:.2f}")

            # --- 2. Evaluate 'Differs' Bet ---
            if actual_digit != target_digit:
                # 🏆 Differs WIN
                pnl_differs += PAYOUT_DIFFERS
                differs_wins += 1
                # 🟢 Log Differs WIN
                # print(f"🟢 Differs Trade {j+1}: WIN (+{PAYOUT_DIFFERS:.4f} units). Actual digit ({actual_digit}) **DIFFERS** from target ({target_digit}). Total P&L: {pnl_differs:.2f}")
            else:
                # 💀 Differs LOSS
                pnl_differs += LOSS_MULTIPLIER
                # 🔴 Log Differs LOSS
                # print(f"🔴 Differs Trade {j+1}: LOSS (-{abs(LOSS_MULTIPLIER):.2f} units). Actual digit ({actual_digit}) **MATCHES** target ({target_digit}). Total P&L: {pnl_differs:.2f}")
            
            # 🖼️ Add a separator for each trade for clarity
            # print("....." * 8)


        # --- Store Final P&L for the Target Digit ---
        # 📊 Calculate Net Return for both bets
        net_return_match = (pnl_match / total_simulations) * 100
        net_return_differs = (pnl_differs / total_simulations) * 100
        
        # 📝 Store the results
        results_list.append({
            "Target Digit": target_digit,
            "Bet Type": "Match",
            "Total P&L (Units)": pnl_match,
            "Net Return (%)": net_return_match,
            "Win Count": match_wins
        })
        results_list.append({
            "Target Digit": target_digit,
            "Bet Type": "Differs",
            "Total P&L (Units)": pnl_differs,
            "Net Return (%)": net_return_differs,
            "Win Count": differs_wins
        })
    
    # 🖼️ Create and display the DataFrame
    df_results = pd.DataFrame(results_list)
    
    print("\n" * 2)
    print("=" * 70)
    print("🎯 FINAL STATIC DIGIT STRATEGY BACKTEST RESULTS SUMMARY 🎯")
    print("=" * 70)
    
    # Sort for best performance and use Markdown for clean output
    df_sorted = df_results.sort_values(by="Total P&L (Units)", ascending=False).reset_index(drop=True)
    
    # 🌟 Highlight the top two strategies
    print("\n👑 Top 2 Performing Strategies:")
    print(df_sorted.head(2).to_markdown(index=False))

    # --- Detailed Analysis ---
    print("\n--- Summary of All Strategies (Sorted by P&L) ---")
    print(df_sorted.to_markdown(index=False))
    
    return df_sorted

In [23]:
results_df = run_static_digit_strategy_backtest(
    model=model, 
    close_data=close, 
    total_simulations=1000, 
    context_size=4000
)

⏱️ log: Function 'run_static_digit_strategy_backtest' started at: 518.777663549
⏳ log: Starting 1000 simulations and collecting actual outcomes...

-------------------- ACTUAL VALUE: 835.14 ----- PREDICTED VALUE: 833.9647216796875 --------------------
🎯  The actual digit is 4 while the predicted last digit is 6

-------------------- ACTUAL VALUE: 837.55 ----- PREDICTED VALUE: 835.334716796875 --------------------
🎯  The actual digit is 5 while the predicted last digit is 3

-------------------- ACTUAL VALUE: 840.08 ----- PREDICTED VALUE: 839.7719116210938 --------------------
🎯  The actual digit is 8 while the predicted last digit is 7

-------------------- ACTUAL VALUE: 841.68 ----- PREDICTED VALUE: 840.38037109375 --------------------
🎯  The actual digit is 8 while the predicted last digit is 8

-------------------- ACTUAL VALUE: 842.09 ----- PREDICTED VALUE: 842.008544921875 --------------------
🎯  The actual digit is 9 while the predicted last digit is 1

-------------------- ACTUA

In [24]:
results_df

,Target Digit,Bet Type,Total P&L (Units),Net Return (%),Win Count
0,1,Match,35.764,3.5764,116
1,2,Match,26.835,2.6835,115
2,4,Differs,-0.448,-0.0448,912
3,0,Differs,-0.448,-0.0448,912
4,9,Differs,-5.928,-0.5928,907
5,7,Differs,-8.120,-0.8120,905
6,8,Differs,-9.216,-0.9216,904
7,5,Differs,-11.408,-1.1408,902
8,6,Differs,-19.080,-1.9080,895
9,3,Differs,-20.176,-2.0176,894


In [25]:
@timer_decorator
def calculate_kelly_metrics(results_df, total_trades=1000, payout_match=7.929, payout_differs=0.096):
    """
    Calculates the Expected Value (EV) and Optimal Kelly Fraction (f*)
    for all strategies in the backtest results DataFrame.

    :param results_df: The DataFrame output from the backtest function.
    :param total_trades: The total number of trades (simulations) performed.
    :param payout_match: The multiplier for a Match win (7.929).
    :param payout_differs: The multiplier for a Differs win (0.096).
    :return: A new DataFrame with EV and f* added, sorted by EV.
    """
    # 📏 log: Starting calculation of Expected Value and Kelly Metrics...
    
    # 1. Calculate Probabilities and Payout Ratio (b)
    results_df['Win Rate (p)'] = results_df['Win Count'] / total_trades
    results_df['Loss Rate (q)'] = 1 - results_df['Win Rate (p)']
    
    # Define Payout Ratio (b) based on Bet Type
    results_df['Payout Ratio (b)'] = results_df['Bet Type'].apply(
        lambda x: payout_match if x == 'Match' else payout_differs
    )

    # 2. Calculate Expected Value (EV)
    # The EV formula is: EV = (p * b) - q
    results_df['Expected Value (EV)'] = (
        results_df['Win Rate (p)'] * results_df['Payout Ratio (b)']
    ) - results_df['Loss Rate (q)']

    # 3. Calculate Optimal Kelly Fraction (f*)
    # The Kelly formula is: f* = EV / b
    
    # Use .loc to avoid SettingWithCopyWarning and handle the case where EV <= 0
    results_df['Optimal Kelly (f*)'] = 0.0 # ❌ Initialize to 0 (Do not bet if EV <= 0)
    
    # Calculate f* only for strategies with a positive Expected Value
    profitable_strategies = results_df['Expected Value (EV)'] > 0
    
    results_df.loc[profitable_strategies, 'Optimal Kelly (f*)'] = (
        results_df.loc[profitable_strategies, 'Expected Value (EV)'] / 
        results_df.loc[profitable_strategies, 'Payout Ratio (b)']
    )
    
    # Calculate the recommended quarter-Kelly fraction
    results_df['Recommended Risk (f*/4)'] = results_df['Optimal Kelly (f*)'] / 4

    # 4. Final Formatting and Sorting
    final_df = results_df.sort_values(by='Expected Value (EV)', ascending=False).reset_index(drop=True)
    
    # 🖼️ Display the most critical data
    print("\n" + "=" * 80)
    print("🚀 EV-BASED STRATEGY RANKING (Best Edge First) 🚀")
    print("=" * 80)
    print(final_df[['Target Digit', 'Bet Type', 'Expected Value (EV)', 
                    'Optimal Kelly (f*)', 'Recommended Risk (f*/4)', 'Win Rate (p)']].head(5).to_markdown(index=False))

    return final_df

# --- How to Use This Function ---
# Assuming 'results_df' is the output DataFrame from your 1000-trade backtest:
kelly_metrics_df = calculate_kelly_metrics(results_df.copy()) # Use .copy() to be safe

⏱️ log: Function 'calculate_kelly_metrics' started at: 828.455576266

🚀 EV-BASED STRATEGY RANKING (Best Edge First) 🚀
|   Target Digit | Bet Type   |   Expected Value (EV) |   Optimal Kelly (f*) |   Recommended Risk (f*/4) |   Win Rate (p) |
|---------------:|:-----------|----------------------:|---------------------:|--------------------------:|---------------:|
|              1 | Match      |              0.035764 |           0.00451053 |               0.00112763  |          0.116 |
|              2 | Match      |              0.026835 |           0.00338441 |               0.000846103 |          0.115 |
|              4 | Differs    |             -0.000448 |           0          |               0           |          0.912 |
|              0 | Differs    |             -0.000448 |           0          |               0           |          0.912 |
|              9 | Differs    |             -0.005928 |           0          |               0           |          0.907 |
🛑 log: Functio

In [26]:
kelly_metrics_df

,Target Digit,Bet Type,Total P&L (Units),Net Return (%),Win Count,Win Rate (p),Loss Rate (q),Payout Ratio (b),Expected Value (EV),Optimal Kelly (f*),Recommended Risk (f*/4)
0,1,Match,35.764,3.5764,116,0.116,0.884,7.929,0.035764,0.004511,0.001128
1,2,Match,26.835,2.6835,115,0.115,0.885,7.929,0.026835,0.003384,0.000846
2,4,Differs,-0.448,-0.0448,912,0.912,0.088,0.096,-0.000448,0.000000,0.000000
3,0,Differs,-0.448,-0.0448,912,0.912,0.088,0.096,-0.000448,0.000000,0.000000
4,9,Differs,-5.928,-0.5928,907,0.907,0.093,0.096,-0.005928,0.000000,0.000000
5,7,Differs,-8.120,-0.8120,905,0.905,0.095,0.096,-0.008120,0.000000,0.000000
6,8,Differs,-9.216,-0.9216,904,0.904,0.096,0.096,-0.009216,0.000000,0.000000
7,5,Differs,-11.408,-1.1408,902,0.902,0.098,0.096,-0.011408,0.000000,0.000000
8,6,Differs,-19.080,-1.9080,895,0.895,0.105,0.096,-0.019080,0.000000,0.000000
9,3,Differs,-20.176,-2.0176,894,0.894,0.106,0.096,-0.020176,0.000000,0.000000


In [27]:
import numpy as np
import pandas as pd
import random # Used to simulate the outcomes based on the win rate

# --- Strategy Constants ---
# 1. Get the row for the best strategy (which is always index 0)
best_strategy = kelly_metrics_df.iloc[0]
# 2. Assign the metrics dynamically
# We use Recommended Risk (f*/4) for the simulation
KELLY_RISK_F = best_strategy['Recommended Risk (f*/4)']
# We use the updated Win Rate (p) for the simulation
WIN_RATE_P = best_strategy['Win Rate (p)']
PAYOUT_RATIO_B = 7.929    # The multiplier for a winning trade
NUM_TRADES = 1000


@timer_decorator
def run_kelly_capital_simulation(initial_balance, risk_fraction, num_trades):
    """
    Simulates capital growth and drawdown using the Kelly/Half-Kelly fraction.

    :param initial_balance: The starting capital.
    :param risk_fraction: The percentage of capital risked per trade (e.g., 0.0152).
    :param num_trades: The number of trades to simulate.
    :return: (DataFrame of trade history, Max Drawdown %)
    """
    
    # 💰 Initialize tracking variables
    current_balance = initial_balance
    peak_balance = initial_balance
    max_drawdown = 0.0
    trade_history = []

    # 🚀 Log initial settings
    print("=" * 40)
    print(f"💰 log: Starting Kelly Simulation (Half-Kelly: {risk_fraction*100:.2f}%)")
    print(f"💵 log: Initial Balance: ${initial_balance:,.2f}")
    print("=" * 40)
    
    # 🔄 Loop through all trades
    for i in range(1, num_trades + 1):
        
        # 1. Determine Stake based on current capital and Kelly fraction
        stake_amount = current_balance * risk_fraction
        
        # 2. Simulate Outcome (Target 8 Match win/loss)
        # We use a random number generator to simulate the 13.9% win rate
        is_win = random.random() < WIN_RATE_P

        # 3. Calculate P&L
        if is_win:
            # P&L = Stake * Payout Ratio (7.929)
            pnl = stake_amount * PAYOUT_RATIO_B
            current_balance += pnl
            # 🟢 Log win
            print(f"✅ Trade {i}: WIN! Stake: ${stake_amount:,.2f}, P&L: +${pnl:,.2f}. New Balance: ${current_balance:,.2f}")
        else:
            # P&L = -Stake * 1.00 (100% loss of the stake)
            pnl = -stake_amount * 1.00 
            current_balance += pnl
            # 🔴 Log loss
            print(f"❌ Trade {i}: LOSS. Stake: ${stake_amount:,.2f}, P&L: -${abs(pnl):,.2f}. New Balance: ${current_balance:,.2f}")

        # 4. Update Drawdown Metrics
        peak_balance = max(peak_balance, current_balance)
        drawdown = (peak_balance - current_balance) / peak_balance
        max_drawdown = max(max_drawdown, drawdown)
        
        # 5. Record history
        trade_history.append({
            "Trade": i,
            "Balance": current_balance,
            "Drawdown": drawdown,
            "is_win": is_win,
            "P&L": pnl
        })
        
        # ⚠️ Check for margin call/risk of ruin
        if current_balance <= 0:
            print("🛑 STOP! Capital depleted (Risk of Ruin).")
            break
            
        print("-" * 20)

    # 📊 Final Report
    max_drawdown_percent = max_drawdown * 100
    print("\n" * 2)
    print("=" * 50)
    print(f"✨ SIMULATION COMPLETE ({i} trades) ✨")
    print("=" * 50)
    print(f"Final Balance: **${current_balance:,.2f}**")
    print(f"Total Return: **{((current_balance - initial_balance) / initial_balance) * 100:.2f}%**")
    print(f"**MAX DRAWDOWN:** **{max_drawdown_percent:.2f}%** (from peak capital)")
    print("=" * 50)
    
    return pd.DataFrame(trade_history), max_drawdown_percent



In [28]:
# --- Execution Example ---
# You can choose any realistic starting balance here
STARTING_BALANCE = 5000.00
history_df, max_dd = run_kelly_capital_simulation(STARTING_BALANCE, KELLY_RISK_F, NUM_TRADES)

⏱️ log: Function 'run_kelly_capital_simulation' started at: 828.528843517
💰 log: Starting Kelly Simulation (Half-Kelly: 0.11%)
💵 log: Initial Balance: $5,000.00
❌ Trade 1: LOSS. Stake: $5.64, P&L: -$5.64. New Balance: $4,994.36
--------------------
❌ Trade 2: LOSS. Stake: $5.63, P&L: -$5.63. New Balance: $4,988.73
--------------------
❌ Trade 3: LOSS. Stake: $5.63, P&L: -$5.63. New Balance: $4,983.10
--------------------
❌ Trade 4: LOSS. Stake: $5.62, P&L: -$5.62. New Balance: $4,977.49
--------------------
❌ Trade 5: LOSS. Stake: $5.61, P&L: -$5.61. New Balance: $4,971.87
--------------------
❌ Trade 6: LOSS. Stake: $5.61, P&L: -$5.61. New Balance: $4,966.27
--------------------
❌ Trade 7: LOSS. Stake: $5.60, P&L: -$5.60. New Balance: $4,960.67
--------------------
❌ Trade 8: LOSS. Stake: $5.59, P&L: -$5.59. New Balance: $4,955.07
--------------------
❌ Trade 9: LOSS. Stake: $5.59, P&L: -$5.59. New Balance: $4,949.48
--------------------
❌ Trade 10: LOSS. Stake: $5.58, P&L: -$5.58. Ne

In [29]:
# # --- Execution Example ---
# # You can choose any realistic starting balance here
# STARTING_BALANCE = 5000.00
# history_df, max_dd = run_kelly_capital_simulation(STARTING_BALANCE, KELLY_RISK_F, NUM_TRADES)

In [30]:
# --- Execution Example ---
# You can choose any realistic starting balance here
STARTING_BALANCE = 50.00
history_df, max_dd = run_kelly_capital_simulation(STARTING_BALANCE, KELLY_RISK_F, NUM_TRADES)

⏱️ log: Function 'run_kelly_capital_simulation' started at: 828.569577984
💰 log: Starting Kelly Simulation (Half-Kelly: 0.11%)
💵 log: Initial Balance: $50.00
❌ Trade 1: LOSS. Stake: $0.06, P&L: -$0.06. New Balance: $49.94
--------------------
✅ Trade 2: WIN! Stake: $0.06, P&L: +$0.45. New Balance: $50.39
--------------------
❌ Trade 3: LOSS. Stake: $0.06, P&L: -$0.06. New Balance: $50.33
--------------------
❌ Trade 4: LOSS. Stake: $0.06, P&L: -$0.06. New Balance: $50.28
--------------------
❌ Trade 5: LOSS. Stake: $0.06, P&L: -$0.06. New Balance: $50.22
--------------------
❌ Trade 6: LOSS. Stake: $0.06, P&L: -$0.06. New Balance: $50.16
--------------------
❌ Trade 7: LOSS. Stake: $0.06, P&L: -$0.06. New Balance: $50.11
--------------------
✅ Trade 8: WIN! Stake: $0.06, P&L: +$0.45. New Balance: $50.55
--------------------
❌ Trade 9: LOSS. Stake: $0.06, P&L: -$0.06. New Balance: $50.50
--------------------
❌ Trade 10: LOSS. Stake: $0.06, P&L: -$0.06. New Balance: $50.44
--------------

In [31]:
STARTING_BALANCE = 100.00
history_df, max_dd = run_kelly_capital_simulation(STARTING_BALANCE, KELLY_RISK_F, NUM_TRADES)

⏱️ log: Function 'run_kelly_capital_simulation' started at: 828.59806077
💰 log: Starting Kelly Simulation (Half-Kelly: 0.11%)
💵 log: Initial Balance: $100.00
❌ Trade 1: LOSS. Stake: $0.11, P&L: -$0.11. New Balance: $99.89
--------------------
❌ Trade 2: LOSS. Stake: $0.11, P&L: -$0.11. New Balance: $99.77
--------------------
❌ Trade 3: LOSS. Stake: $0.11, P&L: -$0.11. New Balance: $99.66
--------------------
❌ Trade 4: LOSS. Stake: $0.11, P&L: -$0.11. New Balance: $99.55
--------------------
❌ Trade 5: LOSS. Stake: $0.11, P&L: -$0.11. New Balance: $99.44
--------------------
❌ Trade 6: LOSS. Stake: $0.11, P&L: -$0.11. New Balance: $99.33
--------------------
❌ Trade 7: LOSS. Stake: $0.11, P&L: -$0.11. New Balance: $99.21
--------------------
❌ Trade 8: LOSS. Stake: $0.11, P&L: -$0.11. New Balance: $99.10
--------------------
❌ Trade 9: LOSS. Stake: $0.11, P&L: -$0.11. New Balance: $98.99
--------------------
❌ Trade 10: LOSS. Stake: $0.11, P&L: -$0.11. New Balance: $98.88
------------

In [32]:
results_df = run_static_digit_strategy_backtest(
    model=model, 
    close_data=close, 
    total_simulations=2000, 
    context_size=3000
)

⏱️ log: Function 'run_static_digit_strategy_backtest' started at: 828.623366602
⏳ log: Starting 2000 simulations and collecting actual outcomes...

-------------------- ACTUAL VALUE: 877.16 ----- PREDICTED VALUE: 876.2198486328125 --------------------
🎯  The actual digit is 6 while the predicted last digit is 2

-------------------- ACTUAL VALUE: 876.09 ----- PREDICTED VALUE: 877.2807006835938 --------------------
🎯  The actual digit is 9 while the predicted last digit is 8

-------------------- ACTUAL VALUE: 876.71 ----- PREDICTED VALUE: 876.060791015625 --------------------
🎯  The actual digit is 1 while the predicted last digit is 6

-------------------- ACTUAL VALUE: 876.47 ----- PREDICTED VALUE: 876.6284790039062 --------------------
🎯  The actual digit is 7 while the predicted last digit is 3

-------------------- ACTUAL VALUE: 875.88 ----- PREDICTED VALUE: 876.4283447265625 --------------------
🎯  The actual digit is 8 while the predicted last digit is 3

-------------------- AC

In [33]:
# --- How to Use This Function ---
# Assuming 'results_df' is the output DataFrame from your 1000-trade backtest:
kelly_metrics_df = calculate_kelly_metrics(results_df.copy()) # Use .copy() to be sa

⏱️ log: Function 'calculate_kelly_metrics' started at: 1463.588095383

🚀 EV-BASED STRATEGY RANKING (Best Edge First) 🚀
|   Target Digit | Bet Type   |   Expected Value (EV) |   Optimal Kelly (f*) |   Recommended Risk (f*/4) |   Win Rate (p) |
|---------------:|:-----------|----------------------:|---------------------:|--------------------------:|---------------:|
|              2 | Match      |              1.03581  |             0.130636 |                 0.032659  |          0.228 |
|              1 | Match      |              1.0001   |             0.126131 |                 0.0315329 |          0.224 |
|              0 | Differs    |              0.998008 |            10.3959   |                 2.59898   |          1.823 |
|              4 | Differs    |              0.984856 |            10.2589   |                 2.56473   |          1.811 |
|              5 | Differs    |              0.980472 |            10.2132   |                 2.55331   |          1.807 |
🛑 log: Functi

In [ ]:
# --- Strategy Constants ---
# 1. Get the row for the best strategy (which is always index 0)
best_strategy = kelly_metrics_df.iloc[0]
# 2. Assign the metrics dynamically
# We use Recommended Risk (f*/4) for the simulation
KELLY_RISK_F = best_strategy['Recommended Risk (f*/4)']
# We use the updated Win Rate (p) for the simulation
WIN_RATE_P = best_strategy['Win Rate (p)']
PAYOUT_RATIO_B = 7.929    # The multiplier for a winning trade
NUM_TRADES = 2000

STARTING_BALANCE = 50.00
history_df, max_dd = run_kelly_capital_simulation(STARTING_BALANCE, KELLY_RISK_F, NUM_TRADES)

# EVEN ODD

In [ ]:
@timer_decorator
def run_parity_strategy_backtest(model, close_data, total_simulations=1000, context_size=4000, prediction_length=1):
    """
    Runs a rolling backtest simulating trades for the two parity bet types: Even and Odd.
    The 'Target Digit' loop is removed as it's redundant for parity checks.

    :param model: The trained forecasting model.
    :param close_data: The full time series (NumPy array).
    :param total_simulations: The number of times to run the 1-tick backtest.
    :param context_size: The fixed number of ticks used as input context.
    :return: A Pandas DataFrame summarizing P&L for the Even and Odd bet types.
    """
    # 📏 Safety Check
    min_data_needed = context_size + total_simulations
    if len(close_data) < min_data_needed:
        # ❌ Log error
        print(f"❌ log: Error: Data length ({len(close_data)}) is too short. Need at least {min_data_needed} ticks.")
        return pd.DataFrame()

    # 🧮 Strategy Payout Multipliers (Payout for Even/Odd)
    # Payout is 95.31% (0.9531 units profit per unit staked)
    PAYOUT_EVEN_ODD = 0.9531 
    # Loss is 100% (-1.00 unit loss per unit staked)
    LOSS_MULTIPLIER = -1.00

    # 📈 Prepare to collect actual outcomes and results
    forecast_results = []
    pnl_even = 0.0
    pnl_odd = 0.0
    even_wins = 0
    odd_wins = 0
    
    # --- Phase 1: Run Simulations and Collect Actual Outcome Digits ---
    # ⏱️ Log start of simulation
    print(f"⏳ log: Starting {total_simulations} simulations and collecting actual outcomes...")
    
    for i in range(total_simulations):
        context_end_index = context_size + i
        context_start_index = i
        
        ctx = close_data[context_start_index : context_end_index]
        actual_value = close_data[context_end_index] 
        
        # 🔮 Run the forecast (forecast is still needed even if not used for strategy)
        quantiles, _ = model.forecast(ctx, prediction_length=prediction_length)
        predicted_median_value = quantiles[0][0, 4].numpy().item()
        
        # 🎯 Get the last digit
        actual_digit = get_last_digit(actual_value)
        
        forecast_results.append({
            "Actual_Digit": actual_digit,
            "Predicted_Median_Digit": get_last_digit(predicted_median_value),
            "Context_Indices": (context_start_index, context_end_index)
        })

    # ⚡ Log phase completion
    print(f"✅ log: Simulation phase complete. Collected {len(forecast_results)} results.")

    # --- Phase 2: Analyze Results Against Parity Bets (Even/Odd) ---
    # 🚀 Log start of detailed analysis
    print(f"\n{'='*70}")
    print(f"🎯 ANALYSIS START: Evaluating Against Parity Bets (EVEN and ODD)")
    print(f"Context Window Size: {context_size} ticks. Total Predictions: {total_simulations}")
    print(f"{'='*70}\n")
    
    # 📝 Iterate through the collected actual outcomes once
    for j, trade in enumerate(forecast_results):
        
        actual_digit = trade["Actual_Digit"]
        # 🔑 The key is checking parity
        is_actual_even = (actual_digit % 2 == 0)
        
        # --- 1. Evaluate 'EVEN' Bet ---
        if is_actual_even:
            # 🏆 EVEN WIN
            pnl_even += PAYOUT_EVEN_ODD
            even_wins += 1
            # 🟢 log: Even bet WIN
        else:
            # 💀 EVEN LOSS
            pnl_even += LOSS_MULTIPLIER
            # 🔴 log: Even bet LOSS

        # --- 2. Evaluate 'ODD' Bet ---
        if not is_actual_even: # i.e., actual digit is odd
            # 🏆 ODD WIN
            pnl_odd += PAYOUT_EVEN_ODD
            odd_wins += 1
            # 🟢 log: Odd bet WIN
        else:
            # 💀 ODD LOSS
            pnl_odd += LOSS_MULTIPLIER
            # 🔴 log: Odd bet LOSS


    # --- Store Final P&L for Parity Bets ---
    results_list = []
    
    # Calculate Net Return for Even
    net_return_even = (pnl_even / total_simulations) * 100
    results_list.append({
        "Bet Type": "Even",
        "Total P&L (Units)": pnl_even,
        "Net Return (%)": net_return_even,
        "Win Count": even_wins
    })
    
    # Calculate Net Return for Odd
    net_return_odd = (pnl_odd / total_simulations) * 100
    results_list.append({
        "Bet Type": "Odd",
        "Total P&L (Units)": pnl_odd,
        "Net Return (%)": net_return_odd,
        "Win Count": odd_wins
    })

    
    # 🖼️ Create and display the DataFrame
    df_results = pd.DataFrame(results_list)
    
    print("\n" * 2)
    print("=" * 70)
    print("🎯 FINAL PARITY STRATEGY (EVEN/ODD) BACKTEST RESULTS SUMMARY 🎯")
    print("=" * 70)
    
    # Sort for best performance
    df_sorted = df_results.sort_values(by="Total P&L (Units)", ascending=False).reset_index(drop=True)
    
    # 🌟 Highlight the best performing strategy
    print("\n👑 Best Performing Strategy:")
    print(df_sorted.head(1).to_markdown(index=False))

    # --- Detailed Analysis ---
    print("\n--- Summary of All Strategies (Sorted by P&L) ---")
    # 🖼️ Log final result
    print("✨ log: Displaying final sorted results.")
    print(df_sorted.to_markdown(index=False))
    
    return df_sorted

results_df_parity = run_parity_strategy_backtest(
    model=model, 
    close_data=close, 
    total_simulations=1000, 
    context_size=4000
)

In [ ]:
@timer_decorator
def calculate_kelly_metrics_parity(results_df, total_trades=1000, payout_even_odd=0.9531):
    """
    Calculates the Expected Value (EV) and Optimal Kelly Fraction (f*)
    for the Even/Odd parity strategies.

    :param results_df: The DataFrame output from the parity backtest function.
    :param total_trades: The total number of trades (simulations) performed.
    :param payout_even_odd: The net profit multiplier for a win (0.9531 for 95.31% profit).
    :return: A new DataFrame with EV and f* added, sorted by EV.
    """
    # 📏 log: Starting calculation of Expected Value and Kelly Metrics for Parity bets...
    
    # 1. Calculate Probabilities and Payout Ratio (b)
    results_df['Win Rate (p)'] = results_df['Win Count'] / total_trades
    results_df['Loss Rate (q)'] = 1 - results_df['Win Rate (p)']
    
    # 💰 The Payout Ratio (b) is the same for both Even and Odd bets.
    # It represents the net profit per unit staked.
    results_df['Payout Ratio (b)'] = payout_even_odd 

    # 2. Calculate Expected Value (EV)
    # The EV formula is: EV = (p * b) - q
    # Where p = Win Rate, b = Payout Ratio (net profit), and q = Loss Rate (1 unit loss).
    results_df['Expected Value (EV)'] = (
        results_df['Win Rate (p)'] * results_df['Payout Ratio (b)']
    ) - results_df['Loss Rate (q)']

    # 3. Calculate Optimal Kelly Fraction (f*)
    # The Kelly formula is: f* = EV / b
    
    # ❌ Initialize to 0 (Do not bet if EV <= 0)
    results_df['Optimal Kelly (f*)'] = 0.0 
    
    # Calculate f* only for strategies with a positive Expected Value
    profitable_strategies = results_df['Expected Value (EV)'] > 0
    
    results_df.loc[profitable_strategies, 'Optimal Kelly (f*)'] = (
        results_df.loc[profitable_strategies, 'Expected Value (EV)'] / 
        results_df.loc[profitable_strategies, 'Payout Ratio (b)']
    )
    
    # 📉 Calculate the recommended quarter-Kelly fraction for safer risk management
    results_df['Recommended Risk (f*/4)'] = results_df['Optimal Kelly (f*)'] / 4

    # 4. Final Formatting and Sorting
    final_df = results_df.sort_values(by='Expected Value (EV)', ascending=False).reset_index(drop=True)
    
    # 🖼️ Display the most critical data
    print("\n" + "=" * 80)
    print("🚀 EV-BASED PARITY STRATEGY RANKING (Best Edge First) 🚀")
    print("=" * 80)
    
    # 🌟 log: Displaying Kelly results.
    print(final_df[['Bet Type', 'Expected Value (EV)', 
                    'Optimal Kelly (f*)', 'Recommended Risk (f*/4)', 'Win Rate (p)']].to_markdown(index=False))

    return final_df

In [ ]:
kelly_metrics_parity_df = calculate_kelly_metrics_parity(results_df_parity)

In [ ]:
kelly_metrics_parity_df

In [ ]:
@timer_decorator
def run_parity_capital_simulation(initial_balance, kelly_risk_f, win_rate_p, payout_ratio_b, num_trades):
    """
    Simulates capital growth and drawdown for a Parity (Even/Odd) strategy 
    using the Kelly/Fractional Kelly fraction.

    :param initial_balance: The starting capital.
    :param kelly_risk_f: The percentage of capital risked per trade (f* or f*/N).
    :param win_rate_p: The calculated historical win rate (p) of the strategy.
    :param payout_ratio_b: The net profit multiplier for a winning trade (0.9531).
    :param num_trades: The number of trades to simulate.
    :return: (DataFrame of trade history, Max Drawdown %)
    """
    
    # 💰 Initialize tracking variables
    current_balance = initial_balance
    peak_balance = initial_balance
    max_drawdown = 0.0
    trade_history = []

    # 🚀 Log initial settings
    print("=" * 60)
    # ℹ️ log: Displaying simulation parameters
    print(f"💰 log: Starting Parity Simulation (Risk: {kelly_risk_f*100:.2f}%)")
    print(f"📈 log: Win Rate: {win_rate_p*100:.2f}%, Payout Ratio (B): {payout_ratio_b:.4f}")
    print(f"💵 log: Initial Balance: ${initial_balance:,.2f}")
    print("=" * 60)
    
    # 🔄 Loop through all trades
    for i in range(1, num_trades + 1):
        
        # 1. Determine Stake based on current capital and Kelly fraction
        stake_amount = current_balance * kelly_risk_f
        
        # 2. Simulate Outcome (Even/Odd win/loss)
        # We use a random number generator to simulate the strategy's historical win rate (p)
        is_win = random.random() < win_rate_p

        # 3. Calculate P&L
        if is_win:
            # P&L = Stake * Payout Ratio (0.9531 for Even/Odd)
            pnl = stake_amount * payout_ratio_b
            current_balance += pnl
            # 🟢 log: Trade WIN!
            # print(f"✅ Trade {i}: WIN! Stake: ${stake_amount:,.2f}, P&L: +${pnl:,.2f}. New Balance: ${current_balance:,.2f}")
        else:
            # P&L = -Stake * 1.00 (100% loss of the stake)
            pnl = -stake_amount * 1.00 
            current_balance += pnl
            # 🔴 log: Trade LOSS.
            # print(f"❌ Trade {i}: LOSS. Stake: ${stake_amount:,.2f}, P&L: -${abs(pnl):,.2f}. New Balance: ${current_balance:,.2f}")

        # 4. Update Drawdown Metrics
        peak_balance = max(peak_balance, current_balance)
        # Handle division by zero for depleted capital
        if peak_balance > 0:
            drawdown = (peak_balance - current_balance) / peak_balance
        else:
            drawdown = 1.0
        max_drawdown = max(max_drawdown, drawdown)
        
        # 5. Record history
        trade_history.append({
            "Trade": i,
            "Balance": current_balance,
            "Drawdown": drawdown,
            "is_win": is_win,
            "P&L": pnl
        })
        
        # ⚠️ Check for risk of ruin
        if current_balance <= 0:
            print(f"🛑 log: STOP! Capital depleted (Risk of Ruin) at Trade {i}.")
            break
            
        # print("-" * 20) # Commented out for cleaner output

    # 📊 Final Report
    max_drawdown_percent = max_drawdown * 100
    total_return = ((current_balance - initial_balance) / initial_balance) * 100
    
    print("\n" * 2)
    print("=" * 50)
    print(f"✨ SIMULATION COMPLETE ({i} trades) ✨")
    print("=" * 50)
    print(f"Final Balance: **${current_balance:,.2f}**")
    print(f"Total Return: **{total_return:.2f}%**")
    print(f"**MAX DRAWDOWN:** **{max_drawdown_percent:.2f}%** (from peak capital)")
    print("=" * 50)
    
    # 🖼️ log: Returning trade history dataframe
    return pd.DataFrame(trade_history), max_drawdown_percent

In [ ]:
# Example of how to call the function after calculating Kelly metrics:
# Assume 'best_parity_strategy' is the row with the highest positive EV from the Kelly metrics DF
best_parity_strategy = kelly_metrics_parity_df.iloc[0]

trade_history_df_parity, max_dd_parity = run_parity_capital_simulation(
    initial_balance=10000, 
    kelly_risk_f=best_parity_strategy['Recommended Risk (f*/4)'], 
    win_rate_p=best_parity_strategy['Win Rate (p)'], 
    payout_ratio_b=0.9531, # Parity payout
    num_trades=1000
)

# ACCUMULATORS

# Core functionality

In [ ]:
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn import metrics
import seaborn as sns 
from typing import List
import os # 📂 Added for file operations

# 🙈 Suppress future warnings from packages like pandas/sklearn
warnings.filterwarnings("ignore")

# 💻 Determine device (CPU or CUDA)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🌍 Running on device: {DEVICE}")

# =========================================================
# 🎯 ASSUMED EXTERNAL DEPENDENCIES (As requested, not implemented here)
# =========================================================
# DEFAULT_TOKEN = "your_token"
# APP_ID = 1089
# class DerivOHLCFetcher: ...
# def _sync_fetch_ohlc_threaded: ...
# class FlexibleResNLS(nn.Module): ... 

# =========================================================
# 🛠️ CLASSIFIER CLASS (Deriv Even/Odd Classification)
# =========================================================

class ResNLSClassifierDeriv:
    """
    A class to fetch Deriv data, preprocess for Even/Odd classification (0/1),
    train the ResNLS model with Early Stopping, and validate the results for 
    multiple confidence levels.
    """
    def __init__(self, ticker, sequence_length=5, train_split=0.9087, epochs=1000, batch_size=64, count:int=5000, tf:int=60, confidence_list: List[float] = None): 
        # 📥 Data Parameters (Deriv specific)
        self.TICKER = ticker
        self.SEQUENCE_LENGTH = sequence_length
        self.TRAIN_SPLIT = train_split
        self.GRANULARITY = tf
        self.COUNT = count
        
        # 🔥 Training Parameters
        self.EPOCHS = epochs # 🎯 Set to 1000
        self.BATCH_SIZE = batch_size
        
        # 🎯 Confidence Levels for Backtest
        self.CONFIDENCE_LIST = confidence_list or [0.50, 0.60, 0.70, 0.80]
        
        # 💾 Data Storage
        self.data_df = None 
        self.scaler = MinMaxScaler(feature_range=(0,1))
        self.training_data_len = 0
        self.x_train, self.y_train, self.x_test, self.y_test = (None,) * 4
        
        self.n_hidden = 64
        
        # 🧠 Model expects 2 output classes for Even (0) / Odd (1)
        self.model = FlexibleResNLS(
            n_input=self.SEQUENCE_LENGTH, 
            n_hidden=self.n_hidden,
            n_output=2
        ).to(DEVICE) 
        self.valid_data = None
        
        # ⚙️ Early Stopping & Checkpoint Parameters
        self.PATIENCE = 50 
        self.best_val_loss = float('inf')
        self.early_stop_counter = 0
        self.MODEL_CHECKPOINT_FILE = f"{self.TICKER}_best_classifier_checkpoint.pth"

        # 📈 Loss Tracking
        self.train_losses = []
        self.val_losses = []
        
        print(f"\n✨ Initialized ResNLS Classifier for {self.TICKER} (Even/Odd).")
        print(f"🔥 Max Epochs: {self.EPOCHS}, Early Stopping Patience: {self.PATIENCE}")
        print(f"📌 Backtest Confidence Levels: {self.CONFIDENCE_LIST}")

    # --- Data Preprocessing Helper Functions ---
    def _split_data(self, dataset, train_day, predict_day):
        """Creates sequential data and corresponding labels (X only here)."""
        x, y = [], []
        for i in range(train_day, len(dataset)-predict_day+1):
            x.append(dataset[i-train_day : i, 0])
        return x, y

    def _reshape_data(self, train_data, test_data, days):
        """Reshapes the sequential data for LSTM input: [samples, 1, timesteps]."""
        x_train, _ = self._split_data(train_data, days, 1)
        x_test, _ = self._split_data(test_data, days, 1)
        
        x_train = np.array(x_train)
        x_test = np.array(x_test)
        
        x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
        x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
        return x_train, None, x_test, None

# --- Main Pipeline Methods ---
    def load_deriv_data(self):
        """Step 1: Fetch OHLC data from Deriv and preprocess for EVEN/ODD classification."""
        print(f"📥 Fetching {self.GRANULARITY // 60}-minute OHLC data for {self.TICKER}...")
    
        fetcher = DerivOHLCFetcher(DEFAULT_TOKEN, APP_ID)
        ohlc_data = None
    
        try:
            ohlc_data = _sync_fetch_ohlc_threaded(
                fetcher, self.TICKER, self.GRANULARITY, self.COUNT
            )
        except Exception as e:
            print(f"💥 Fatal error during data fetching: {e}") # ❌ Log error
            return False
    
        if ohlc_data is None or ohlc_data.empty:
            print(f"❌ Error: No data found for {self.TICKER}. Exiting.") # ❌ Log error
            return False
    
        self.data_df = ohlc_data
        closes = self.data_df['Close'].astype(str).values
        last_digits = np.array([int(c[-1]) for c in closes])  
        labels = last_digits % 2  # 0=Even ✅ 1=Odd ✅
        print("🔢 Converted Close→(Even/Odd) labels. Sample:", labels[:5]) 
    
        dataset = np.reshape(self.data_df['Close'].values, (self.data_df.shape[0], 1))
    
        self.training_data_len = math.ceil(len(dataset) * self.TRAIN_SPLIT)
        
        train_prices = dataset[:self.training_data_len]
        test_prices  = dataset[self.training_data_len:]
        
        train_labels = labels[:self.training_data_len]
        test_labels  = labels[self.training_data_len:]
    
        self.scaler.fit(train_prices)
        print("⚖️ Scaler fitted on training data only (leak-free).") 
    
        scaled_train = self.scaler.transform(train_prices)
        scaled_test = self.scaler.transform(test_prices)
    
        context = scaled_train[-self.SEQUENCE_LENGTH:]
        test_data = np.concatenate((context, scaled_test), axis=0)
    
        self.x_train, _, self.x_test, _ = self._reshape_data(
            scaled_train, test_data, self.SEQUENCE_LENGTH
        )
    
        self.y_train = train_labels[self.SEQUENCE_LENGTH:]
        self.y_test = test_labels
        
        # --- NEW: Calculate Class Weights for Imbalance Handling ---
        from collections import Counter
        label_counts = Counter(self.y_train)
        even_count = label_counts.get(0, 1)
        odd_count = label_counts.get(1, 1)
        total_count = even_count + odd_count
        
        # Calculate inverse frequency weights: Weight = Total / (2 * Count)
        weight_even = total_count / (2.0 * even_count)
        weight_odd = total_count / (2.0 * odd_count)
        
        # ⚖️ Store weights as a PyTorch tensor on the correct device
        self.class_weights = torch.tensor([weight_even, weight_odd], dtype=torch.float).to(DEVICE)
    
        print(f"🎯 Classification Labels Ready → Even(0) vs Odd(1)") 
        print(f"⚖️ Class Weights (Even/Odd): [{weight_even:.4f}, {weight_odd:.4f}] calculated from training data.")
        print(f"📊 Shapes → x_train: {self.x_train.shape}, y_train: {self.y_train.shape}")
        print(f"📈 Shapes → x_test:  {self.x_test.shape},  y_test:  {self.y_test.shape}")
    
        return True

    def train_model(self, custom_lr: float = 1e-3): 
        """
        Step 2: Train the ResNLS model for EVEN/ODD classification with 
        Early Stopping based on validation loss and checkpoint saving, using class weights.
        """
        print(f"\n🔥 Starting model training (LR: {custom_lr}) with class weights...")
    
        val_input = torch.tensor(self.x_test, dtype=torch.float).to(DEVICE)
        val_target = torch.tensor(self.y_test, dtype=torch.long).to(DEVICE)
    
        # 🔑 Apply the calculated class weights to the criterion
        criterion = nn.CrossEntropyLoss(weight=self.class_weights)
        # 💡 Using custom_lr passed from Optuna
        optimizer = optim.Adam(self.model.parameters(), lr=custom_lr)
    
        data_len = self.x_train.shape[0]
        batch_num = (data_len + self.BATCH_SIZE - 1) // self.BATCH_SIZE
        print(f"🎯 Epochs: {self.EPOCHS}, Batch Size: {self.BATCH_SIZE}, Batches: {batch_num}")
    
        # 🔄 Reset early stopping state for a new training run
        self.best_val_loss = float('inf')
        self.early_stop_counter = 0
        self.train_losses = [] # 📈 Reset losses
        self.val_losses = []   # 📈 Reset losses

        for epoch in range(self.EPOCHS):
            epoch_train_losses = []
            correct = 0
            total = 0
            
            # --- Training Loop ---
            for j in range(batch_num):
                train_input_np = self.x_train[j*self.BATCH_SIZE : (j+1)*self.BATCH_SIZE]
                train_target_np = self.y_train[j*self.BATCH_SIZE : (j+1)*self.BATCH_SIZE]
    
                train_input = torch.tensor(train_input_np, dtype=torch.float).to(DEVICE)
                train_target = torch.tensor(train_target_np, dtype=torch.long).to(DEVICE)
    
                self.model.train()
                optimizer.zero_grad()
                outputs = self.model(train_input) 
                loss = criterion(outputs, train_target)  
    
                loss.backward()
                optimizer.step()
                epoch_train_losses.append(loss.item())
                
                _, predicted = torch.max(outputs.data, 1)
                total += train_target.size(0)
                correct += (predicted == train_target).sum().item()
            
            avg_train_loss = sum(epoch_train_losses) / len(epoch_train_losses)
            train_accuracy = 100 * correct / total
            self.train_losses.append(avg_train_loss) # 📈 Track training loss
    
            # --- Validation & Early Stopping Check ---
            with torch.no_grad():
                self.model.eval()
                val_outputs = self.model(val_input)
                val_loss = criterion(val_outputs, val_target)
                
                _, val_pred = torch.max(val_outputs.data, 1)
                val_accuracy = 100 * (val_pred == val_target).sum().item() / val_target.size(0)
    
                self.val_losses.append(val_loss.item()) # ✅ Track validation loss
                
                # 💾 Checkpoint Saving Logic
                if val_loss < self.best_val_loss:
                    self.best_val_loss = val_loss.item()
                    self.early_stop_counter = 0 
                    torch.save(self.model.state_dict(), self.MODEL_CHECKPOINT_FILE)
                    print(f"💾 Epoch {epoch+1:>3}: New best model saved! Val Loss: {self.best_val_loss:.4f}")
                else:
                    self.early_stop_counter += 1 
    
            # 📢 Log training and validation loss/accuracy for every epoch
            print(f"⏳ Epoch {epoch+1:>3} | "
                  f"Loss: {avg_train_loss:.4f} | Acc: {train_accuracy:.2f}% | "
                  f"Val Loss: {val_loss.item():.4f} | Val Acc: {val_accuracy:.2f}% "
                  f"(Wait: {self.early_stop_counter})") # 🗑️ Removed PATIENCE from log
            
            # 🛑 Early Stopping Check (Requires self.PATIENCE to be set in __init__)
            if self.early_stop_counter >= self.PATIENCE:
                print(f"\n🛑 Early stopping triggered at epoch {epoch+1}. Patience ({self.PATIENCE}) exceeded.")
                break

        print("✅ Training finished.") 
    
        # 🔑 Load the best saved model state before exiting
        if os.path.exists(self.MODEL_CHECKPOINT_FILE):
            self.model.load_state_dict(torch.load(self.MODEL_CHECKPOINT_FILE))
            print(f"🚀 Loaded best model checkpoint from {self.MODEL_CHECKPOINT_FILE} for validation.")
            
        self.plot_loss()

    def plot_loss(self):
        """Plot the training and validation loss."""
        
        print("\n📈 Generating Loss Plot...")
        
        plt.figure(figsize=(10, 5))
        plt.plot(self.train_losses, label='Training Loss', color='blue')
        plt.plot(self.val_losses, label='Validation Loss', color='red')
        plt.title(f'Training and Validation Loss for {self.TICKER} - (Sequence Length: {self.SEQUENCE_LENGTH})', fontsize=14)
        plt.xlabel('Epoch', fontsize=12)
        plt.ylabel('Loss (Cross Entropy)', fontsize=12) 
        plt.legend(loc='upper right')
        plt.grid(True)
        plt.show()
        print("✅ Loss Plot complete.")

    def validate_and_plot(self):
            """Step 3: Validate model classification performance and visualize predictions."""
            self.model.eval()
            
            with torch.no_grad():
                # 💡 Ensure dtype consistency and move to device
                x_val = torch.tensor(self.x_test, dtype=torch.float).to(DEVICE)
            
                logits = self.model(x_val)
                probs = torch.softmax(logits, dim=1).cpu().numpy()
            
                predicted_classes = np.argmax(probs, axis=1)
            
            y_actual = self.y_test
            accuracy = metrics.accuracy_score(y_actual, predicted_classes) * 100
            print(f"\n🎯 Classification Accuracy: {accuracy:.2f}%")
            
            print("\n📊 Classification Report:")
            print(metrics.classification_report(y_actual, predicted_classes, target_names=["Even (0)", "Odd (1)"])) 
            
            # --- Confusion Matrix Plot (SAVE instead of SHOW) ---
            plt.figure(figsize=(6, 5)) 
            cm = metrics.confusion_matrix(y_actual, predicted_classes)
            sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                                xticklabels=["Pred Even", "Pred Odd"],
                                yticklabels=["Actual Even", "Actual Odd"])
            plt.title(f"Confusion Matrix: {self.TICKER} (Input Len: {self.SEQUENCE_LENGTH})")
            plt.xlabel("Predicted")
            plt.ylabel("Actual")
    
            # # 💾 Save the plot with unique identifiers
            # plot_filename = f"cm_{self.TICKER}_seq{self.SEQUENCE_LENGTH}_batch{self.BATCH_SIZE}.png"
            # plt.savefig(plot_filename)
            plt.show() # 🧹 Close the plot to free memory
            # print(f"🖼️ Confusion Matrix Plot saved to {plot_filename}.")
            
            # --- Store Validation Data ---
            # 📏 Ensure alignment by slicing data_df to match y_actual length
            self.valid_data = self.data_df.iloc[self.training_data_len:].copy().head(len(y_actual))
            self.valid_data["Actual_EvenOdd"] = y_actual
            self.valid_data["Predicted_EvenOdd"] = predicted_classes
            self.valid_data["Prob_Even"] = probs[:, 0]
            self.valid_data["Prob_Odd"] = probs[:, 1]
            
            print("\n✅ Validation Complete — Predictions Stored!")

    def backtest_deriv_even_odd_strategy(self):
        """
        Backtest the Even/Odd classification strategy for Deriv across multiple
        minimum confidence levels defined in self.CONFIDENCE_LIST.
        """
        
        print("\n🎯 Backtest: Even/Odd Last Digit Prediction (Classification)")
        
        df = self.valid_data.dropna()
        if df.empty:
            print("❌ Backtest failed: No validation predictions available.")
            return

        results = []
        
        # Constants
        BET_RISK = 1.00
        PAYOUT_RATE = 0.95
        initial_capital = 100.0
        
        # 🔁 Loop through all requested confidence levels
        for min_confidence in self.CONFIDENCE_LIST:
            print(f"\n📌 Running backtest with Min Confidence Filter: {min_confidence * 100:.0f}%")
            
            capital = initial_capital
            wins = 0
            losses = 0
            skipped = 0
            current_losing_streak = 0
            max_losing_streak = 0
        
            total_rows = len(df)
            
            for i in range(total_rows):
                row = df.iloc[i]
            
                # ✅ Skip low-confidence trades
                prediction_conf = max(row["Prob_Even"], row["Prob_Odd"])
            
                if prediction_conf < min_confidence:
                    skipped += 1
                    continue
            
                # ✅ Check capital
                if capital < BET_RISK:
                    print(f"🛑 Out of capital (Cap: {capital:.2f}) — stopping early for {min_confidence*100:.0f}% confidence.") 
                    break
            
                predicted = row["Predicted_EvenOdd"]
                actual = row["Actual_EvenOdd"]
            
                if predicted == actual:
                    wins += 1
                    capital += BET_RISK * PAYOUT_RATE
                    current_losing_streak = 0
                else:
                    losses += 1
                    capital -= BET_RISK
                    current_losing_streak += 1
                    max_losing_streak = max(max_losing_streak, current_losing_streak)
            
            total_trades = wins + losses
            win_rate = (wins / total_trades) * 100 if total_trades > 0 else 0
            final_return = ((capital - initial_capital) / initial_capital) * 100
            
            # 💾 Store results 
            results.append({
                "Confidence": min_confidence,
                "Trades": total_trades,
                "Wins": wins,
                "Losses": losses,
                "Skipped": skipped,
                "WinRate": win_rate,
                "Return": final_return,
                "MaxLosingStreak": max_losing_streak
            })

            print(f"✅ Confidence {min_confidence*100:.0f}% | Trades: {total_trades}, Win Rate: {win_rate:.2f}%, Return: {final_return:.2f}%")
        
        # 🧾 Summary Table
        print("\n📈 BACKTEST SUMMARY (All Confidence Levels)")
        print("-" * 80)
        print(f"{'Confidence (%)':<15}{'Total Trades':<15}{'Wins':<10}{'Losses':<10}{'Skipped':<10}{'Win Rate (%)':<15}{'Total Return (%)':<15}")
        print("-" * 80)
        for r in results:
            print(f"{r['Confidence']*100:<15.0f}{r['Trades']:<15}{r['Wins']:<10}{r['Losses']:<10}{r['Skipped']:<10}{r['WinRate']:<15.2f}{r['Return']:<15.2f}")
        print("-" * 80)
        
        # 📊 Plot Win Rate vs Confidence
        self._plot_backtest_results(results)

        print("🎉 Backtest for all confidence levels complete!\n")

    def _plot_backtest_results(self, results):
        """Plots the win rate and total return across confidence levels."""
        
        conf_levels = [r['Confidence'] * 100 for r in results]
        win_rates = [r['WinRate'] for r in results]
        returns = [r['Return'] for r in results]

        fig, ax1 = plt.subplots(figsize=(10, 6))

        # Plot Win Rate
        color = 'tab:blue'
        ax1.set_xlabel('Minimum Confidence Level (%)', fontsize=12)
        ax1.set_ylabel('Win Rate (%)', color=color, fontsize=12)
        ax1.plot(conf_levels, win_rates, marker='o', color=color, label='Win Rate')
        ax1.tick_params(axis='y', labelcolor=color)
        ax1.grid(True, axis='y', linestyle='--')

        # Create a second Y-axis for Total Return
        ax2 = ax1.twinx()  
        color = 'tab:green'
        ax2.set_ylabel('Total Return (%)', color=color, fontsize=12)
        ax2.plot(conf_levels, returns, marker='s', color=color, label='Total Return')
        ax2.tick_params(axis='y', labelcolor=color)
        ax2.axhline(0, color='gray', linestyle='-') 

        fig.tight_layout() 
        plt.title(f"{self.TICKER} - Performance vs Trading Confidence", fontsize=14)
        plt.show()
        print("🖼️ Backtest Results Plot complete.")
        

    def run_forecast(self):
        """Execute the full classification pipeline (Deriv only)."""
        print("🎯 Running Even/Odd Classification Pipeline (Deriv data only)...")
        if self.load_deriv_data():
            print("Loadded data from deriv")
        else:
            print("❌ Failed to load deriv data. Exiting.")
            return
        
        self.train_model()
        self.validate_and_plot()
        self.backtest_deriv_even_odd_strategy()

In [ ]:
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder # 🔢 Added OneHotEncoder
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn import metrics
import seaborn as sns 
from typing import List
import os 
from collections import Counter
import arch 
from arch import arch_model # 📊 Added for GARCH

# 🙈 Suppress future warnings from packages like pandas/sklearn
warnings.filterwarnings("ignore")

# 💻 Determine device (CPU or CUDA)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🌍 Running on device: {DEVICE}")


# =========================================================
# 🛠️ CLASSIFIER CLASS (Deriv Even/Odd Classification)
# =========================================================

class ResNLSClassifierDeriv:
    """
    A class to fetch Deriv data, preprocess (Price + GARCH Volatility + Last Digit OHE),
    train the ResNLS model, and validate results.
    """
    def __init__(self, ticker, sequence_length=5, train_split=0.9087, epochs=1000, batch_size=64, count:int=5000, tf:int=60, confidence_list: List[float] = None): 
        # 📥 Data Parameters (Deriv specific)
        self.TICKER = ticker
        self.SEQUENCE_LENGTH = sequence_length
        self.TRAIN_SPLIT = train_split
        self.GRANULARITY = tf
        self.COUNT = count
        
        # 🔥 Training Parameters
        self.EPOCHS = epochs 
        self.BATCH_SIZE = batch_size
        
        # 🎯 Confidence Levels for Backtest
        self.CONFIDENCE_LIST = confidence_list or [0.50, 0.60, 0.70, 0.80]
        
        # 💾 Data Storage
        self.data_df = None 
        self.scaler = MinMaxScaler(feature_range=(0,1))
        self.vol_scaler = MinMaxScaler(feature_range=(0,1))
        # 🌟 Initialize OHE for 10 categories (digits 0 through 9)
        self.ohe = OneHotEncoder(sparse_output=False, categories=[range(10)]) 
        self.training_data_len = 0
        self.x_train, self.y_train, self.x_test, self.y_test = (None,) * 4
        
        # 🔑 INPUT SIZE: 1 (Price) + 1 (Volatility) + 10 (Last Digit OHE) = 12 features
        self.n_input_features = 14 
        self.n_hidden = 64
        
        # 🧠 Model expects 2 output classes for Even (0) / Odd (1)
        self.model = FlexibleResNLS(
            n_input=self.n_input_features,  # 🔑 UPDATED INPUT SIZE (12)
            n_hidden=self.n_hidden,
            n_output=2
        ).to(DEVICE) 
        self.valid_data = None
        
        # ⚙️ Early Stopping & Checkpoint Parameters
        self.PATIENCE = 50 
        self.best_val_loss = float('inf')
        self.early_stop_counter = 0
        self.MODEL_CHECKPOINT_FILE = f"{self.TICKER}_best_classifier_checkpoint.pth"

        # 📈 Loss Tracking
        self.train_losses = []
        self.val_losses = []
        self.class_weights = None

# 💾 Data Storage
        self.data_df = None 
        self.scaler = MinMaxScaler(feature_range=(0,1))
        self.vol_scaler = MinMaxScaler(feature_range=(0,1))
        self.ld_vol_scaler = MinMaxScaler(feature_range=(0,1))   # 🆕 New Scaler for Last Digit Volatility
        self.fft_scaler = MinMaxScaler(feature_range=(0,1))      # 🆕 New Scaler for FFT Feature
        # 🌟 Initialize OHE for 10 categories (digits 0 through 9)
        self.ohe = OneHotEncoder(sparse_output=False, categories=[range(10)]) 
        self.training_data_len = 0
        self.x_train, self.y_train, self.x_test, self.y_test = (None,) * 4
        
        # 🔑 INPUT SIZE: 1 (Price) + 1 (GARCH Vol) + 1 (LD Vol) + 1 (FFT) + 10 (OHE) = 14 features
        self.n_input_features = 14  # 🔑 UPDATED INPUT SIZE
        self.n_hidden = 64

        print(f"\n✨ Initialized ResNLS Classifier for {self.TICKER} (Even/Odd).")
        print(f"🔑 Model Input Features: {self.n_input_features} (Price + Volatility + Last Digit OHE)")
        print(f"🔥 Max Epochs: {self.EPOCHS}, Early Stopping Patience: {self.PATIENCE}")
        print(f"📌 Backtest Confidence Levels: {self.CONFIDENCE_LIST}")

    # --- Data Preprocessing Helper Functions ---
    def _split_data(self, dataset, train_day):
        """Creates sequential data (X) for the LSTM input [sequence_length, num_features]."""
        x, y = [], []
        num_features = dataset.shape[1]
        
        # 🔁 Log split for clarity
        print(f"🔄 Creating sequences of length {train_day} with {num_features} features...") 
        
        for i in range(train_day, len(dataset)):
             # Append sequence of features for each time step
            x.append(dataset[i-train_day:i, :]) 
        return x, y

    def _reshape_data(self, train_data, test_data, days):
        """Converts sequence lists to NumPy arrays with shape [samples, sequence_length, num_features]."""
        x_train_list, _ = self._split_data(train_data, days)
        x_test_list, _ = self._split_data(test_data, days)
        
        x_train = np.array(x_train_list)
        x_test = np.array(x_test_list)
        
        # Final shape for PyTorch: [batch_size, sequence_length, num_features]
        print(f"🔧 Reshaped Data Shapes: X_train {x_train.shape}, X_test {x_test.shape}") 
        
        return x_train, None, x_test, None

    def load_deriv_data(self):
        """Step 1: Fetch OHLC data, calculate GARCH volatility, and preprocess/scale features."""
        print(f"📥 Fetching {self.GRANULARITY // 60}-minute OHLC data for {self.TICKER}...")
    
        fetcher = DerivOHLCFetcher(DEFAULT_TOKEN, APP_ID) # Assuming these are defined globally
        ohlc_data = None
    
        try:
            ohlc_data = _sync_fetch_ohlc_threaded( # Assuming this utility is defined
                fetcher, self.TICKER, self.GRANULARITY, self.COUNT
            )
        except Exception as e:
            print(f"💥 Fatal error during data fetching: {e}") 
            return False
    
        if ohlc_data is None or ohlc_data.empty:
            print(f"❌ Error: No data found for {self.TICKER}. Exiting.") 
            return False
    
        self.data_df = ohlc_data
        
        # 1. Calculate Returns & Fit GARCH(1,1)
        self.data_df['Return'] = 100 * self.data_df['Close'].pct_change().fillna(0)
        
        try:
            print("⚙️ Fitting GARCH(1,1) model to returns...")
            am = arch_model(self.data_df['Return'], vol='Garch', p=1, q=1, rescale=False, dist='t')
            res = am.fit(disp='off')
            conditional_volatility = res.conditional_volatility.to_numpy().reshape(-1, 1)
            print("✅ GARCH(1,1) model fitted. Volatility feature calculated.")
        except Exception as e:
            print(f"⚠️ GARCH fitting failed, falling back to zero volatility. Error: {e}")
            conditional_volatility = np.zeros_like(self.data_df['Return'].values).reshape(-1, 1)
        
        
        # 2. Calculate Last Digit and Even/Odd Label
        closes = self.data_df['Close'].astype(str).values
        last_digits = np.array([int(c[-1]) for c in closes]).reshape(-1, 1) # Shape [N, 1]
        labels = last_digits % 2 # 0=Even, 1=Odd (The target Y)
        
        # 🌟 One-Hot Encode Last Digit (0-9)
        self.ohe.fit(np.arange(10).reshape(-1, 1)) 
        ohe_last_digit = self.ohe.transform(last_digits) # Shape [N, 10]

        # =========================================================
        # ⚙️ NEW FEATURE ENGINEERING BLOCK (LD Volatility & FFT)
        # =========================================================

        # --- A. Last Digit Volatility (Rolling Standard Deviation) ---
        WINDOW = 20 # Lookback window for LD volatility
        print(f"⚙️ Calculating Last Digit Volatility (Window: {WINDOW})...")
        last_digit_series = pd.Series(last_digits.flatten())
        ld_volatility = last_digit_series.rolling(window=WINDOW).std().fillna(0).values.reshape(-1, 1)

        # --- B. Fourier Transform (Magnitude of Dominant Cycles) ---
        # Focuses on the magnitude of the dominant cycle in price returns
        print(f"⚙️ Calculating Fourier Transform Features (Price Returns)...")

        returns_fft = np.fft.fft(self.data_df['Return'].values)
        magnitude = np.abs(returns_fft)[1:] # Exclude 0-frequency (mean)
        N_fft = len(magnitude)
        
        # Find index of the largest magnitude in the first half of the spectrum
        dominant_freq_index = np.argmax(magnitude[:N_fft//2]) 
        dominant_magnitude = magnitude[dominant_freq_index]
        
        # Broadcast the dominant magnitude across all time steps as a feature
        fft_feature = np.full_like(ld_volatility, dominant_magnitude)
        
        # =========================================================
        # 3. Combine Features into one [N, 14] dataset
        # =========================================================
        
        price_data = self.data_df['Close'].values.reshape(-1, 1) # [N, 1]
        
        # ORDER: [Price, GARCH Vol, LD Vol, FFT Feature, OHE Digits (10)] -> [N, 14]
        full_dataset = np.hstack([
            price_data, 
            conditional_volatility, 
            ld_volatility,        # Feature Index 2
            fft_feature,          # Feature Index 3
            ohe_last_digit        # Feature Index 4-13
        ])
        
        # 4. Split and Scale Data
        self.training_data_len = math.ceil(len(full_dataset) * self.TRAIN_SPLIT)
        train_data = full_dataset[:self.training_data_len]
        test_data_context = full_dataset[self.training_data_len - self.SEQUENCE_LENGTH:]
        
        train_labels = labels[:self.training_data_len]
        self.y_test = labels[self.training_data_len:]
    
        # Fit scalers only on training data (Indices 0, 1, 2, 3)
        self.scaler.fit(train_data[:, 0].reshape(-1, 1))      # Price
        self.vol_scaler.fit(train_data[:, 1].reshape(-1, 1))  # GARCH Vol
        self.ld_vol_scaler.fit(train_data[:, 2].reshape(-1, 1)) # LD Vol
        self.fft_scaler.fit(train_data[:, 3].reshape(-1, 1))    # FFT Feature
    
        # Scale and recombine training data
        scaled_train_price = self.scaler.transform(train_data[:, 0].reshape(-1, 1))
        scaled_train_vol = self.vol_scaler.transform(train_data[:, 1].reshape(-1, 1))
        scaled_train_ld_vol = self.ld_vol_scaler.transform(train_data[:, 2].reshape(-1, 1))
        scaled_train_fft = self.fft_scaler.transform(train_data[:, 3].reshape(-1, 1))
        
        scaled_train = np.hstack([
            scaled_train_price, scaled_train_vol, scaled_train_ld_vol, scaled_train_fft,
            train_data[:, 4:] # OHE columns start at index 4
        ]) 
        
        # Scale and recombine test data
        scaled_test_price = self.scaler.transform(test_data_context[:, 0].reshape(-1, 1))
        scaled_test_vol = self.vol_scaler.transform(test_data_context[:, 1].reshape(-1, 1))
        scaled_test_ld_vol = self.ld_vol_scaler.transform(test_data_context[:, 2].reshape(-1, 1))
        scaled_test_fft = self.fft_scaler.transform(test_data_context[:, 3].reshape(-1, 1))
        
        scaled_test = np.hstack([
            scaled_test_price, scaled_test_vol, scaled_test_ld_vol, scaled_test_fft,
            test_data_context[:, 4:] # OHE columns start at index 4
        ])
        
        # 5. Reshape and Align Labels
        self.x_train, _, self.x_test, _ = self._reshape_data(
            scaled_train, scaled_test, self.SEQUENCE_LENGTH
        )
    
        # Target Y starts one step after the sequence length is accounted for
        self.y_train = train_labels[self.SEQUENCE_LENGTH:]
        
        # ✅ FINAL FIX: Flatten y_train and y_test to remove the extra dimension [N, 1] -> [N]
        self.y_train = self.y_train.flatten()
        self.y_test = self.y_test.flatten()
        
        y_train_flat = self.y_train.astype(int)

        # --- Calculate Class Weights for Imbalance Handling ---
        label_counts = Counter(y_train_flat)
        even_count = label_counts.get(0, 1)
        odd_count = label_counts.get(1, 1)
        total_count = even_count + odd_count
        weight_even = total_count / (2.0 * even_count)
        weight_odd = total_count / (2.0 * odd_count)
        self.class_weights = torch.tensor([weight_even, weight_odd], dtype=torch.float).to(DEVICE)
    
        print(f"🎯 Classification Labels Ready → Even(0) vs Odd(1)") 
        print(f"⚖️ Class Weights (Even/Odd): [{weight_even:.4f}, {weight_odd:.4f}] calculated from training data.")
        print(f"📊 Final Shapes → x_train: {self.x_train.shape}, y_train: {self.y_train.shape}")
        print(f"📈 Final Shapes → x_test:  {self.x_test.shape},  y_test:  {self.y_test.shape}")
    
        return True

    def train_model(self, custom_lr: float = 5e-3): 
        """
        Step 2: Train the ResNLS model for EVEN/ODD classification with 
        Early Stopping based on validation loss and checkpoint saving, using class weights.
        """
        print(f"\n🔥 Starting model training (LR: {custom_lr}) with class weights...")
    
        val_input = torch.tensor(self.x_test, dtype=torch.float).to(DEVICE)
        val_target = torch.tensor(self.y_test, dtype=torch.long).to(DEVICE)
    
        # 🔑 Apply the calculated class weights to the criterion
        criterion = nn.CrossEntropyLoss(weight=self.class_weights)
        optimizer = optim.Adam(self.model.parameters(), lr=custom_lr, weight_decay=1e-4) # 🔑 Added L2
    
        data_len = self.x_train.shape[0]
        batch_num = (data_len + self.BATCH_SIZE - 1) // self.BATCH_SIZE
        print(f"🎯 Epochs: {self.EPOCHS}, Batch Size: {self.BATCH_SIZE}, Batches: {batch_num}")
    
        self.best_val_loss = float('inf')
        self.early_stop_counter = 0
        self.train_losses = [] 
        self.val_losses = []   

        for epoch in range(self.EPOCHS):
            epoch_train_losses = []
            correct = 0
            total = 0
            
            # --- Training Loop ---
            for j in range(batch_num):
                train_input_np = self.x_train[j*self.BATCH_SIZE : (j+1)*self.BATCH_SIZE]
                train_target_np = self.y_train[j*self.BATCH_SIZE : (j+1)*self.BATCH_SIZE]
    
                train_input = torch.tensor(train_input_np, dtype=torch.float).to(DEVICE)
                train_target = torch.tensor(train_target_np, dtype=torch.long).to(DEVICE)
    
                self.model.train()
                optimizer.zero_grad()
                outputs = self.model(train_input) 
                loss = criterion(outputs, train_target)  
    
                loss.backward()
                optimizer.step()
                epoch_train_losses.append(loss.item())
                
                _, predicted = torch.max(outputs.data, 1)
                total += train_target.size(0)
                correct += (predicted == train_target).sum().item()
            
            avg_train_loss = sum(epoch_train_losses) / len(epoch_train_losses)
            train_accuracy = 100 * correct / total
            self.train_losses.append(avg_train_loss) 
    
            # --- Validation & Early Stopping Check ---
            with torch.no_grad():
                self.model.eval()
                val_outputs = self.model(val_input)
                val_loss = criterion(val_outputs, val_target)
                
                _, val_pred = torch.max(val_outputs.data, 1)
                val_accuracy = 100 * (val_pred == val_target).sum().item() / val_target.size(0)
    
                self.val_losses.append(val_loss.item()) 
                
                # 💾 Checkpoint Saving Logic
                if val_loss < self.best_val_loss:
                    self.best_val_loss = val_loss.item()
                    self.early_stop_counter = 0 
                    torch.save(self.model.state_dict(), self.MODEL_CHECKPOINT_FILE)
                    print(f"💾 Epoch {epoch+1:>3}: New best model saved! Val Loss: {self.best_val_loss:.4f}")
                else:
                    self.early_stop_counter += 1 
    
            print(f"⏳ Epoch {epoch+1:>3} | "
                  f"Loss: {avg_train_loss:.4f} | Acc: {train_accuracy:.2f}% | "
                  f"Val Loss: {val_loss.item():.4f} | Val Acc: {val_accuracy:.2f}% "
                  f"(Wait: {self.early_stop_counter})")
            
            # 🛑 Early Stopping Check 
            if self.early_stop_counter >= self.PATIENCE:
                print(f"\n🛑 Early stopping triggered at epoch {epoch+1}. Patience ({self.PATIENCE}) exceeded.")
                break

        print("✅ Training finished.") 
    
        # 🔑 Load the best saved model state
        if os.path.exists(self.MODEL_CHECKPOINT_FILE):
            self.model.load_state_dict(torch.load(self.MODEL_CHECKPOINT_FILE))
            print(f"🚀 Loaded best model checkpoint from {self.MODEL_CHECKPOINT_FILE} for validation.")
            
        self.plot_loss()


    def plot_loss(self):
        """Plot the training and validation loss."""
        
        print("\n📈 Generating Loss Plot...")
        
        plt.figure(figsize=(10, 5))
        plt.plot(self.train_losses, label='Training Loss', color='blue')
        plt.plot(self.val_losses, label='Validation Loss', color='red')
        plt.title(f'Training and Validation Loss for {self.TICKER} - (Sequence Length: {self.SEQUENCE_LENGTH})', fontsize=14)
        plt.xlabel('Epoch', fontsize=12)
        plt.ylabel('Loss (Cross Entropy)', fontsize=12) 
        plt.legend(loc='upper right')
        plt.grid(True)
        plt.show()
        print("✅ Loss Plot complete.")

    def validate_and_plot(self):
        """Step 3: Validate model classification performance and visualize predictions."""
        self.model.eval()
        
        with torch.no_grad():
            # 💡 Ensure dtype consistency and move to device
            x_val = torch.tensor(self.x_test, dtype=torch.float).to(DEVICE)
            
            logits = self.model(x_val)
            probs = torch.softmax(logits, dim=1).cpu().numpy()
            
            predicted_classes = np.argmax(probs, axis=1)
            
        y_actual = self.y_test
        accuracy = metrics.accuracy_score(y_actual, predicted_classes) * 100
        print(f"\n🎯 Classification Accuracy: {accuracy:.2f}%")
        
        print("\n📊 Classification Report:")
        print(metrics.classification_report(y_actual, predicted_classes, target_names=["Even (0)", "Odd (1)"])) 
        
        # --- Confusion Matrix Plot ---
        plt.figure(figsize=(6, 5)) 
        cm = metrics.confusion_matrix(y_actual, predicted_classes)
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                             xticklabels=["Pred Even", "Pred Odd"],
                             yticklabels=["Actual Even", "Actual Odd"])
        plt.title(f"Confusion Matrix: {self.TICKER} (Input Len: {self.SEQUENCE_LENGTH})")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.show() 
        
        # --- Store Validation Data ---
        self.valid_data = self.data_df.iloc[self.training_data_len:].copy().head(len(y_actual))
        self.valid_data["Actual_EvenOdd"] = y_actual
        self.valid_data["Predicted_EvenOdd"] = predicted_classes
        self.valid_data["Prob_Even"] = probs[:, 0]
        self.valid_data["Prob_Odd"] = probs[:, 1]
        
        print("\n✅ Validation Complete — Predictions Stored!")

    def backtest_deriv_even_odd_strategy(self):
        """
        Backtest the Even/Odd classification strategy for Deriv across multiple
        minimum confidence levels defined in self.CONFIDENCE_LIST.
        """
        
        print("\n🎯 Backtest: Even/Odd Last Digit Prediction (Classification)")
        
        df = self.valid_data.dropna()
        if df.empty:
            print("❌ Backtest failed: No validation predictions available.")
            return

        results = []
        
        # Constants
        BET_RISK = 1.00
        PAYOUT_RATE = 0.95
        initial_capital = 100.0
        
        # 🔁 Loop through all requested confidence levels
        for conf_level_raw in self.CONFIDENCE_LIST:
            min_confidence = conf_level_raw if conf_level_raw is not None else 0.0    
            print(f"\n📌 Running backtest with Min Confidence Filter: {min_confidence * 100:.0f}%")
            
            capital = initial_capital
            wins = 0
            losses = 0
            skipped = 0
            current_losing_streak = 0
            max_losing_streak = 0
            
            total_rows = len(df)
            
            for i in range(total_rows):
                row = df.iloc[i]
            
                # ✅ Skip low-confidence trades
                prediction_conf = max(row["Prob_Even"], row["Prob_Odd"])
            
                if prediction_conf < min_confidence:
                    skipped += 1
                    continue
            
                # ✅ Check capital
                if capital < BET_RISK:
                    print(f"🛑 Out of capital (Cap: {capital:.2f}) — stopping early for {min_confidence*100:.0f}% confidence.") 
                    break
            
                predicted = row["Predicted_EvenOdd"]
                actual = row["Actual_EvenOdd"]
            
                if predicted == actual:
                    wins += 1
                    capital += BET_RISK * PAYOUT_RATE
                    current_losing_streak = 0
                else:
                    losses += 1
                    capital -= BET_RISK
                    current_losing_streak += 1
                    max_losing_streak = max(max_losing_streak, current_losing_streak)
            
            total_trades = wins + losses
            win_rate = (wins / total_trades) * 100 if total_trades > 0 else 0
            final_return = ((capital - initial_capital) / initial_capital) * 100
            
            # 💾 Store results 
            results.append({
                "Confidence": min_confidence,
                "Trades": total_trades,
                "Wins": wins,
                "Losses": losses,
                "Skipped": skipped,
                "WinRate": win_rate,
                "Return": final_return,
                "MaxLosingStreak": max_losing_streak
            })

            print(f"✅ Confidence {min_confidence*100:.0f}% | Trades: {total_trades}, Win Rate: {win_rate:.2f}%, Return: {final_return:.2f}%")
        
        # 🧾 Summary Table
        print("\n📈 BACKTEST SUMMARY (All Confidence Levels)")
        print("-" * 80)
        print(f"{'Confidence (%)':<15}{'Total Trades':<15}{'Wins':<10}{'Losses':<10}{'Skipped':<10}{'Win Rate (%)':<15}{'Total Return (%)':<15}")
        print("-" * 80)
        for r in results:
            print(f"{r['Confidence']*100:<15.0f}{r['Trades']:<15}{r['Wins']:<10}{r['Losses']:<10}{r['Skipped']:<10}{r['WinRate']:<15.2f}{r['Return']:<15.2f}")
        print("-" * 80)
        
        # 📊 Plot Win Rate vs Confidence
        self._plot_backtest_results(results)

        print("🎉 Backtest for all confidence levels complete!\n")


    def _plot_backtest_results(self, results):
        """Plots the win rate and total return across confidence levels."""
        
        conf_levels = [r['Confidence'] * 100 for r in results]
        win_rates = [r['WinRate'] for r in results]
        returns = [r['Return'] for r in results]

        fig, ax1 = plt.subplots(figsize=(10, 6))

        # Plot Win Rate
        color = 'tab:blue'
        ax1.set_xlabel('Minimum Confidence Level (%)', fontsize=12)
        ax1.set_ylabel('Win Rate (%)', color=color, fontsize=12)
        ax1.plot(conf_levels, win_rates, marker='o', color=color, label='Win Rate')
        ax1.tick_params(axis='y', labelcolor=color)
        ax1.grid(True, axis='y', linestyle='--')

        # Create a second Y-axis for Total Return
        ax2 = ax1.twinx()  
        color = 'tab:green'
        ax2.set_ylabel('Total Return (%)', color=color, fontsize=12)
        ax2.plot(conf_levels, returns, marker='s', color=color, label='Total Return')
        ax2.tick_params(axis='y', labelcolor=color)
        ax2.axhline(0, color='gray', linestyle='-') 

        fig.tight_layout() 
        plt.title(f"{self.TICKER} - Performance vs Trading Confidence", fontsize=14)
        plt.show()
        print("🖼️ Backtest Results Plot complete.")
        

    def run_forecast(self):
        """Execute the full classification pipeline (Deriv data only)."""
        print("🎯 Running Even/Odd Classification Pipeline (Deriv data only)...")
        if self.load_deriv_data():
            print("Loadded data from deriv")
        else:
            print("❌ Failed to load deriv data. Exiting.")
            return
        
        self.train_model()
        self.validate_and_plot()
        self.backtest_deriv_even_odd_strategy()

In [ ]:
from typing import List

def run_all_stock_forecasts(ticker: str, start_date: str, end_date: str, input_length: int = 5, threshold_list: List = None, deriv: bool = False, count: int = 5000, tf: int = 60):
    """
    Runs a single stock price forecasting model instance (ResNLS) 
    and instructs it to run backtests for all thresholds provided.
    """
    # ⚙️ Set a default threshold list if none is provided
    if threshold_list is None:
        # Renamed variable to match class's expected parameter name
        confidence_list_to_pass = [0.50, 0.60, 0.70, 0.80] 
    else:
        confidence_list_to_pass = threshold_list # Use the list passed in
            
    print(f"🔥 Starting forecast run for Ticker: {ticker} (N_INPUT={input_length}).")
    print(f"    Model will test {len(confidence_list_to_pass)} trading strategies after training once.")
    
    # 📝 Base configuration: (Class, seq_len, split, epochs, batch_size, count, tf)
    # NOTE: 'deriv' is removed from the tuple as it's not a parameter of the classifier
    BASE_CONFIG = (ResNLSClassifierDeriv, input_length, 0.9087, input_length, 64, count, tf)
    
    # ⚠️ ONLY ONE RUN HERE
    ForecasterClass, seq_len, split, epochs, batch_size, count_val, tf_val = BASE_CONFIG
    
    print("\n" + "="*80)
    print(f"🚀 RUN 1 of 1: Training Model Once")
    print(f"    Model: {ForecasterClass.__name__}")
    print(f"    Epochs: {epochs}, Batch: {batch_size}, Input Length: {input_length}")
    print("="*80)

    # ⚙️ Prepare keyword arguments
    kwargs = {
        'sequence_length': seq_len,
        'train_split': split,
        'epochs': epochs,
        'batch_size': batch_size,
        # ❌ REMOVED: 'deriv' is not a parameter of ResNLSClassifierDeriv
        'count': count_val, # Corrected: Use count_val from the tuple
        'tf': tf_val        # Corrected: Use tf_val from the tuple
    }
    print(f"⚙️⚙️⚙️ Kwargs are {kwargs}")
    
    try:
        # 🌟 Create the forecaster instance
        forecaster = ForecasterClass(
            ticker,            # 1st POSITIONAL ARGUMENT (The only required positional arg)
            # ❌ REMOVED: start_date, end_date were conflicting with 'sequence_length' and 'train_split'
            **kwargs,          # Unpacks the remaining configuration arguments
            # 📣 PASS THE CONFIDENCE LIST: Correctly named 'confidence_list'
            confidence_list=confidence_list_to_pass 
        )
        
        # 🏃 Run the forecast (trains once, backtests all thresholds)
        forecaster.run_forecast()
        print(f"✅ Run completed successfully.")
        
    except NameError as e:
        print(f"❌ Error in RUN: Forecaster class not found. Check imports. Details: {e}")
    except Exception as e:
        print(f"💔 Error during execution (Model: {ForecasterClass.__name__}): {e}")
        
    print("\n🎉 All stock forecast runs concluded.")

# Run all forecasts function

In [ ]:
import optuna
import torch
from typing import List
import warnings
import os

# 🙈 Suppress potential Optuna warnings
warnings.filterwarnings("ignore", category=optuna.exceptions.ExperimentalWarning)

# =========================================================
# 🎯 ASSUMED EXTERNAL DEPENDENCIES (Need ResNLSClassifierDeriv)
# =========================================================

# --- Fixed Hyperparameters ---
FIXED_LR = 1e-4             

# =========================================================
# 📈 OPTUNA OBJECTIVE FUNCTION
# =========================================================

def objective(trial: optuna.Trial, 
              ticker: str, 
              confidence_list: List[float], 
              deriv: bool, 
              count: int, 
              tf: int) -> float:
    """
    Optuna objective function that executes the full run_forecast pipeline, 
    optimizing sequence_length and batch_size by minimizing the final validation loss.
    """
    
    # --- 1. Define Hyperparameter Search Space ---
    # 🔥 Parameters to Optimize
    sequence_length = trial.suggest_int("sequence_length", 300, 400)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    
    print("\n" + "="*80)
    print(f"🚀 Starting Optuna Trial {trial.number}")
    print(f"   Input Length (seq_len): {sequence_length}, Batch Size: {batch_size}")
    print("="*80)
    
    # --- 2. Initialize and Run Full Forecast Pipeline ---
    
    # ⚙️ Initialize the classifier instance
    classifier = ResNLSClassifierDeriv(
        ticker=ticker,
        sequence_length=sequence_length, 
        train_split=0.9087,
        epochs=1000,
        batch_size=batch_size,          
        count=count,
        tf=tf,
        confidence_list=confidence_list,
    )
    
    try:
        # 🏃 Call the entire pipeline method, as requested.
        classifier.run_forecast()
        print("✅ Full classifier.run_forecast() pipeline executed.")
        
    except Exception as e:
        # 💔 Log error
        print(f"💔 Execution of run_forecast() failed for trial {trial.number}: {e}") 
        raise optuna.exceptions.TrialPruned()
    
    # --- 3. Get Final Validation Loss ---
    
    if classifier.val_losses:
        # The last validation loss recorded before early stop is the metric
        final_val_loss = classifier.val_losses[-1] 
        print(f"✅ Trial {trial.number} finished. Final Validation Loss: {final_val_loss:.6f}") 
    else:
        # ❌ Log error
        print("❌ Trial failed: No validation losses recorded after run_forecast().") 
        raise optuna.exceptions.TrialPruned()
    
    # --- 4. Report and Prune ---
    trial.report(final_val_loss, trial.number) 
    
    if trial.should_prune():
        # ✂️ Log message
        print(f"✂️ Trial {trial.number} pruned by Optuna due to poor performance.") 
        raise optuna.exceptions.TrialPruned()
        
    # Return the objective value (Validation Loss)
    return final_val_loss



In [ ]:
import optuna
from optuna.pruners import MedianPruner
from optuna.importance import MeanDecreaseImpurityImportanceEvaluator 
import matplotlib.pyplot as plt
from typing import List, TYPE_CHECKING
# TYPE_CHECKING is used to prevent circular imports if the objective is in another file
if TYPE_CHECKING:
    from typing import Any
    from optuna.study import Study
    
# NOTE: The 'objective' function and 'ResNLSClassifierDeriv' class are assumed to be defined elsewhere.

def optimize_forecast(ticker: str, n_trials: int = 50, 
                      confidence_list: List = None, deriv: bool = False, 
                      count: int = 5000, tf: int = 60) -> optuna.study.Study:
    """
    Initializes and runs the Optuna study in memory to find the best sequence_length and 
    batch_size for the classification model, and displays the analysis plots using Matplotlib.
    """
    
    if confidence_list is None:
        confidence_list = [0.50, 0.60, 0.70, 0.80] 
        
    study_name = f"resnls_classifier_optuna_{ticker}_seq_batch_focus"
    
    # 1. Create/Load Optuna Study
    try:
        # ✂️ Define the MedianPruner
        pruner = MedianPruner(n_startup_trials=5, n_warmup_steps=5)
        
        study = optuna.create_study(
            study_name=study_name, 
            direction="minimize", 
            load_if_exists=False,
            pruner=pruner 
        )
        print(f"✨ Created Optuna study '{study_name}' (in-memory) with Median Pruner.")
        
    except Exception as e:
        print(f"💔 Error setting up Optuna study: {e}") 
        return None
    
    # 2. Run Optimization
    # Assumes 'objective' function is available in the current scope
    func = lambda trial: objective(
        trial, 
        ticker=ticker, 
        confidence_list=confidence_list, 
        deriv=deriv, 
        count=count, 
        tf=tf
    )
    
    print(f"\n🏃 Starting Optuna optimization for {n_trials} trials...")
    study.optimize(func, n_trials=n_trials)

    # 3. Report Results
    print("\n" + "="*80)
    print("🎉 Optimization finished!")
    print(f"  Best trial: {study.best_trial.number}")
    print(f"  Best value (Min Val Loss): {study.best_value:.6f}")
    print("  Best hyperparameters:")
    for key, value in study.best_params.items():
        print(f"    - {key}: {value}")
    print("="*80)

    # 4. Generate and DISPLAY Optuna Plots using MATPLOTLIB
    print("\n🖼️ Generating and displaying Optuna analysis plots using Matplotlib...")
    
    # --- Optimization History ---
    plt.figure(figsize=(10, 5))
    optuna.visualization.matplotlib.plot_optimization_history(study)
    plt.title("Optimization History")
    plt.show() 
    plt.close()
    print("  - Displayed Optimization History.")

    # --- Parameter Importance (FIXED with MeanDecreaseImpurityImportanceEvaluator) ---
    importance_evaluator = MeanDecreaseImpurityImportanceEvaluator()
    plt.figure(figsize=(10, 5))
    optuna.visualization.matplotlib.plot_param_importances(
        study, 
        evaluator=importance_evaluator
    )
    plt.title("Hyperparameter Importance (Mean Decrease Impurity)")
    plt.show() 
    plt.close() 
    print("  - Displayed Parameter Importance Plot.")

    # --- Contour Plot (Matplotlib alternative to Parallel Coordinate) ---
    try:
        plt.figure(figsize=(10, 5))
        # Plotting the three suggested parameters (sequence_length, batch_size, learning_rate)
        params_to_plot = ["sequence_length", "batch_size", "learning_rate"]
        # Filter to only plot parameters that actually exist in the study's trials
        optuna.visualization.matplotlib.plot_contour(study, params=[p for p in params_to_plot if p in study.best_params])
        plt.title("Hyperparameter Contour Plot")
        plt.show()
        plt.close()
        print("  - Displayed Hyperparameter Contour Plot.")
    except Exception as e:
        print(f"⚠️ Could not generate Contour Plot: {e}")

    print("✅ All Optuna Matplotlib plots generated and displayed.")
    
    return study

In [ ]:
# optimize_forecast(DERIV_SYMBOL,50)

# Cover all the 'n' input lengths

In [ ]:
# 🔢 Define the different sequence lengths (N_INPUT) to test


def all_n_forecasts(ticker: str, start_date: str="2012-01-01", end_date: str="2025-12-31",threshold_list:List=[None, 0.001, 0.005, 0.01],deriv:bool=False, count:int=5000,tf:int=60):
    """
    Runs a comprehensive set of stock forecasts for a given ticker 
    by iterating over various sequence input lengths (N_INPUT).

    Args:
        ticker (str): The stock ticker symbol (e.g., 'AAPL').
        start_date (str): The start date for historical data (e.g., '2012-01-01').
        end_date (str): The end date for historical data (e.g., '2025-12-31').
    """
    print(f"🔥 Starting all N-input forecast experiments for Ticker: {ticker}...")
    if deriv:
        input_lengths = [100,200,500,1000,3000]
        threshold_list = [None]
    else:
        input_lengths = [3, 5, 10, 20, 40, 60]
    print(f"📄 Defined input lengths (N_INPUT) to iterate over: {input_lengths}")

    # 🔄 Loop through each defined input length
    for i, input_len in enumerate(input_lengths):
        print("\n" + "✅ "*90)
        print(f"🚀 Running experiment {i+1}/{len(input_lengths)} with N_INPUT = {input_len}")
        print("✅ "*90)
        
        # 🏃 Execute the core forecasting function with the current input length
        # NOTE: The provided code hardcodes the dates to "2012-01-01" and "2025-12-31" 
        # inside the loop, overriding the function arguments 'start_date' and 'end_date'.
        run_all_stock_forecasts(
            ticker=ticker, 
            start_date=start_date,  # 🗓️ Using the hardcoded start date
            end_date=end_date,    # 🗓️ Using the hardcoded end date
            input_length=input_len,    # 🔗 Passing the current N_INPUT
            threshold_list=threshold_list,
            deriv=deriv,
            count=count,
            tf=tf
        )
        
        print(f"✅ Finished run for N_INPUT = {input_len}.")
        
    print(f"\n🎉 All N-input forecast sweeps completed for {ticker}.")

# Deriv Volatility 10 Index

In [ ]:
# DERIV_SYMBOL = "R_10"         # 📈 Volatility 100 Index (VIX equivalent)
# OHLC_GRANULARITY = 60       # 📅 1 Day in seconds (to mimic daily close data)
# OHLC_COUNT = 5000              # 💾 Number of historical daily candles to fetch

# all_n_forecasts(ticker=DERIV_SYMBOL, start_date="2012-01-01", end_date="2025-12-31",threshold_list=[None, 0.001, 0.005, 0.01],deriv=True, count=OHLC_COUNT,tf=OHLC_GRANULARITY)

  

In [ ]:
# import zipfile
# import glob
# import os

# def zip_pth_files(directory_to_search: str = '/kaggle/working/', output_zip_path: str = 'all_models.zip'):
#     """
#     Zips all .pth files found in a specified directory (and its subdirectories)
#     into a single .zip archive.

#     Args:
#         directory_to_search (str): The directory to start the search from. 
#                                    Defaults to the current directory ('.').
#         output_zip_path (str): The name and path for the output .zip file.
#                                Defaults to 'all_pth_files.zip'.
#     """
    
#     # 🔍 Use glob to find all .pth files recursively
#     pth_files = glob.glob(os.path.join(directory_to_search, '**', '*.pth'), recursive=True)

#     if not pth_files:
#         print("⚠️ No .pth files found to zip.")
#         return

#     print(f"✨ Found {len(pth_files)} .pth files.")
    
#     # 📦 Create the zip file
#     try:
#         with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
#             print(f"📂 Creating zip archive: {output_zip_path}")
#             for file_path in pth_files:
#                 # 📝 Get the base filename or a relative path for the archive
#                 # Using os.path.basename keeps the archive structure simple,
#                 # otherwise you can use os.path.relpath(file_path, directory_to_search) 
#                 # to maintain the folder structure within the zip.
#                 arcname = os.path.basename(file_path) 
                
#                 # 🛠️ Write the file to the zip
#                 zipf.write(file_path, arcname=arcname)
#                 print(f"➡️ Added {arcname}")
                
#         print(f"✅ Successfully created {output_zip_path} with {len(pth_files)} files.")

#     except Exception as e:
#         print(f"❌ An error occurred during zipping: {e}")


# print("--- Starting Zipping in Current Directory ---")
# zip_pth_files()

# print("--- All models have been saved!")